In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy import linalg as scpla
import scipy
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq,minimize
import scipy.integrate
from math import tanh,cosh
import math
import time
import matplotlib.patches as mpatches
from functools import partial
import random

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# from sympy import *
from scipy.linalg import schur, eigvals
from scipy.special import comb, perm

extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [ ]:
#### structural connectivity 
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
    Amplit = gavgfix*typenum
    numsample = typenum**2
    Amplitg= np.zeros(numsample)
    idxc=0
    while (1):
      if idxc>=numsample:
        Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
        break
      p=np.random.random(1)
      Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
      if np.sum(Amplitg)>1.0:
        continue 
      elif np.sum(Amplitg)==1.0:
        break
      else:
        idxc +=1
        # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
    # Amplitg=0
    Amplitg*=Amplit
    Amplitg=np.sqrt(Amplitg)
    return Amplitg

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

def gradtanh(xorg):
    gradx = xorg.copy()
    nneuron,nknum=np.shape(gradx)[0],np.shape(gradx)[1]
    for i in range(nneuron):
        for j in range(nknum):
            gradx[i,j]=4/(np.exp(-xorg[i,j])+np.exp(xorg[i,j]))**2
    return gradx

In [ ]:
### dynamics
def iidGaussian(stats,shapem):
	mu,sig = stats[0],stats[1]
	nx,ny = shapem[0],shapem[1]
	return np.random.normal(mu,sig,(nx,ny))
def odeIntegral(x,t,J,I=0):
	x = np.squeeze(x)
	x = np.reshape(x,(len(x),1))
	# print('size:',np.shape(x),np.shape(J@np.tanh(x)))
	dxdt = -x+J@np.tanh(x)
	return np.squeeze(dxdt)
def odesimulation(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegral,J=Jpt,I=I),xinit,t)
## for single value 
def transferf(xorg):
	return np.tanh(xorg)
def transferdf(xorg):
    return np.log(np.cosh(xorg))
    # return 4/(np.exp(-xorg)+np.exp(xorg))**2
	# return 1/np.cosh(xorg)**2

# def decompunperturbedEI(Junperturb,nrank=1):
# 	eigvAm,eigveclAm,eigvecrAm=scpla.eig(Junperturb,left=True,right=True)
# 	N = len(eigvAm)
# 	meigvecAm=eigvecrAm.copy()
# 	neigvecAm=eigveclAm.copy()
# 	for i in range(nrank):
# 	    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
# 	    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
# 	for i in range(nrank,N):
# 	    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
# 	    neigvecAm[:,i]/=normalizecoe
# 	return meigvecAm,neigvecAm,eigvAm
# 	
def decompunperturbedEI(Junperturb,nrank=1):
	eigvAm,eigvecrAm=la.eig(Junperturb)
	meigvecAm=eigvecrAm.copy()
	neigvecAm=la.inv(eigvecrAm.copy())
	neigvecAm=neigvecAm.T
	for i in range(nrank):
		normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
		neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
	for i in range(nrank,N):
		normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
		neigvecAm[:,i]/=normalizecoe
	return meigvecAm,neigvecAm,eigvAm	

def decompNormalization(Jconn,refxvec,refyvec,xsign,ysign,nparams,sort=0,nrank=1):
	NE,NI= nparams[0],nparams[1]
	N = NE+NI
	eigvJ,eigrvecJ=la.eig(Jconn)
	inveigrvecJ=la.inv(eigrvecJ)
	meig = np.squeeze(eigrvecJ[:,:].copy())
	neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
	neig=neig.copy().T
	for i in range(nrank):
		neig[:,i]*=eigvJ[i]
	for j in range(nrank,N):
		neig[:,j]*=eigvJ[j]
	leig, reig = np.zeros((N,N)),np.zeros((N,N))
	'''    Sort Eigenvalue in ascending     '''
	if (sort ==1):
		eigenvalue_Rsort = np.squeeze(-np.abs(np.real(eigvJ.copy())))
		idxsort=np.argsort(eigenvalue_Rsort)
		eigenvalue_sort = np.squeeze(eigvJ.copy())
		eigvJ= eigvJ[idxsort]  #### >>>>>>>>>>>>>> resorting >>>>>>>>>
		## >>> for eigendecomposition 
		reig = meig[:,idxsort]
		leig = neig[:,idxsort]
		normval=np.sum(reig*leig.copy(),axis=0)
		normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
		leig=leig.copy()/normval.copy()   # error
	elif (sort==0):	
		reig=meig.copy()
		normval=np.sum(reig*neig.copy(),axis=0)
		normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
		leig=neig.copy()/normval.copy()
	## for reference
	tildex,tildey=np.reshape(reig[:,0].copy(),(N,1)),np.reshape(leig[:,0].copy(),(N,1))
	## make sure the E in y is positive, for negative change signs of x and y
	# if (np.mean(tildey[:NE,0])*refyvec[0,0])<0:
	#     tildex*=(-1)
	#     tildey*=(-1)
	if (np.mean(tildex[:NE,0])*xsign[0,0])<0:
	    tildex*=(-1)
	    tildey*=(-1)
	x,y=np.reshape(tildex,(N,1)),np.reshape(tildey,(N,1))
	x=np.sqrt(np.squeeze(tildey.T@refxvec)/np.squeeze(refyvec.T@tildex))*tildex
	y=np.sqrt(np.squeeze(tildex.T@refyvec)/np.squeeze(refxvec.T@tildey))*tildey
	return (eigvJ,leig,reig,x,y)

def decompUnnormalization(Jconn,refxvec,refyvec,xsign,ysign,nparams,sort=0,nrank=1):
	NE,NI= nparams[0],nparams[1]
	N = NE+NI
	eigvJ,eigrvecJ=la.eig(Jconn)
	inveigrvecJ=la.inv(eigrvecJ)
	meig = np.squeeze(eigrvecJ[:,:].copy())
	neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
	neig=neig.copy().T
	for i in range(nrank):
		neig[:,i]*=eigvJ[i]
	for j in range(nrank,N):
		neig[:,j]*=eigvJ[j]
	leig, reig = np.zeros((N,N)),np.zeros((N,N))
	'''    Sort Eigenvalue in ascending     '''
	if (sort ==1):
		eigenvalue_Rsort = np.squeeze(-np.abs(np.real(eigvJ.copy())))
		idxsort=np.argsort(eigenvalue_Rsort)
		eigenvalue_sort = np.squeeze(eigvJ.copy())
		eigvJ= eigvJ[idxsort]  #### >>>>>>>>>>>>>> resorting >>>>>>>>>
		## >>> for eigendecomposition 
		reig = meig[:,idxsort]
		leig = neig[:,idxsort]
		normval=np.sum(reig*leig.copy(),axis=0)
		normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
		leig=leig.copy()/normval.copy()   # error
	elif (sort==0):	
		reig=meig.copy()
		normval=np.sum(reig*neig.copy(),axis=0)
		normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
		leig=neig.copy()/normval.copy()
	## for reference
	tildex,tildey=np.reshape(reig[:,0].copy(),(N,1)),np.reshape(leig[:,0].copy(),(N,1))
	## make sure the E in y is positive, for negative change signs of x and y
	# if (np.mean(tildey[:NE,0])*refyvec[0,0])<0:
	#     tildex*=(-1)
	#     tildey*=(-1)
	if (np.mean(tildex[:NE,0])*xsign[0,0])<0:
		tildex*=(-1)
		tildey*=(-1)
	x,y=np.reshape(tildex,(N,1)),np.reshape(tildey,(N,1))
	return (eigvJ,leig,reig,x,y)

def Normalization(xvec,yvec,refxvec,refyvec,nparams,sort=0,nrank=1):
	NE,NI= nparams[0],nparams[1]
	N = NE+NI
	leig, reig = np.zeros((N,N)),np.zeros((N,N))
	reig=xvec.copy()
	normval=np.sum(reig*yvec.copy(),axis=0)
	# normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
	leig=yvec.copy()/normval.copy()
	## for reference
	tildex,tildey=np.reshape(reig[:,0].copy(),(N,1)),np.reshape(leig[:,0].copy(),(N,1))
	if (np.mean(tildex[:NE,0])*refxvec[0,0])<0:
		tildex*=(-1)
		tildey*=(-1)
	x,y=np.reshape(tildex,(N,1)),np.reshape(tildey,(N,1))
	x=np.sqrt(np.squeeze(tildey.T@refxvec)/np.squeeze(refyvec.T@tildex))*tildex
	y=np.sqrt(np.squeeze(tildex.T@refyvec)/np.squeeze(refxvec.T@tildey))*tildey
	return (leig,reig,x,y)

def numerical_stats(xrvec,ylvec,xrref,ylref,eigv,nrank,npop,ppercent):## ppercent might be useful
	nneuron = np.shape(xrvec)[0]
	nnpop = np.zeros(npop)
	for i in range(npop):
		nnpop[i] = int(ppercent[i]*nneuron)
	axrmu,aylmu = np.zeros((npop,nrank)),np.zeros((npop,nrank))
	sigxr,sigyl = np.zeros((npop,nrank)),np.zeros((npop,nrank))
	sigcov = np.zeros((npop,nrank,nrank))

	for irank in range(nrank):
		for ipop in range(npop):
			nns,nne = np.sum(nnpop[:ipop]),np.sum(nnpop[:ipop])+nnpop[ipop]
			nns = nns.astype(np.int32)
			nne = nne.astype(np.int32)
			axrmu[ipop,irank],aylmu[ipop,irank] = np.mean(xrvec[nns:nne,irank]),np.mean(ylvec[nns:nne,irank])
			sigxr[ipop,irank],sigyl[ipop,irank] = np.std(xrvec[nns:nne,irank])**2,np.std(ylvec[nns:nne,irank])**2

			# axrmu[ipop,irank],aylmu[ipop,irank] = np.mean(xrref[nns:nne,irank]),np.mean(ylref[nns:nne,irank])
			# sigxr[ipop,irank],sigyl[ipop,irank] = np.mean((xrvec[nns:nne,irank]-xrref[nns:nne,irank])**2),np.mean((ylvec[nns:nne,irank]-ylref[nns:nne,irank])**2)

	for ipop in range(npop):
		neuronpop = int(nnpop[ipop])
		nns,nne = np.sum(nnpop[:ipop]),np.sum(nnpop[:ipop])+nnpop[ipop]
		nns = nns.astype(np.int32)
		nne = nne.astype(np.int32)
		noisexr,noiseyl = np.zeros((neuronpop,nrank)),np.zeros((neuronpop,nrank))
		for irank in range(nrank):
			noisexr[:,irank],noiseyl[:,irank] = xrvec[nns:nne,irank],ylvec[nns:nne,irank]
			noisexr[:,irank]-=axrmu[irank]
			noiseyl[:,irank]-=aylmu[irank]
		sigcov[ipop,:,:] = noiseyl.T@noisexr/neuronpop

	return(axrmu,aylmu,sigxr,sigyl,sigcov)

In [ ]:
### Statistics of Gaussian random matrix 
### Gaussian Pparameters
gaussian_norm = (1/np.sqrt(np.pi))
gauss_points, gauss_weights = np.polynomial.hermite.hermgauss(300)
gauss_points = gauss_points*np.sqrt(2)

def Phi(mu, delta0):
    integrand = np.tanh(mu+np.sqrt(delta0)*gauss_points)
    return gaussian_norm * np.dot (integrand,gauss_weights)

def derPhi(mu,delta0):
    # integrand = np.log(np.cosh(mu+np.sqrt(delta0)*gauss_points))
    integrand = 2/(np.cosh(2*(mu+np.sqrt(delta0)*gauss_points))+1.0)
    return gaussian_norm * np.dot (integrand,gauss_weights)

def innerdeuxPhi(mu,delta0):
    # integrand = np.tanh(mu+np.sqrt(delta0)*gauss_points)
    # return gaussian_norm * np.dot(integrand,gauss_weights)
    integrand = np.tanh(mu+np.sqrt(delta0)*gauss_points)
    return gaussian_norm * np.dot (integrand**2,gauss_weights)

def innerdeuxderPhi(mu,delta0):
    # integrand = np.log(np.cosh(mu+np.sqrt(delta0)*gauss_points))
    integrand = 2/(np.cosh(2*(mu+np.sqrt(delta0)*gauss_points))+1.0)
    return gaussian_norm * np.dot (integrand**2,gauss_weights)

### no correlation
## solve the dynamics using the consistency of kappa
def iidperturbation(x,JE,JI,gmat,nparams):
    # JE,JI,g0 = args
    gee,gei,gie,gii = gmat[0],gmat[1],gmat[2],gmat[3]
    # print('gset:',gee,gei,gie,gii)
    NE,NI = nparams[0],nparams[1]
    N = NE+NI
    muphi,delta0phiE,delta0phiI = x, x**2*(gee**2*NE/N+gei**2*NI/N)/(JE-JI)**2, x**2*(gie**2*NE/N+gii**2*NI/N)/(JE-JI)**2
    delta_kappa = -x+(JE*Phi(muphi,delta0phiE)-JI*Phi(muphi,delta0phiI))
    # delta_kappa = -x+(JE-JI)*(Phi(muphi,delta0phiE)+Phi(muphi,delta0phiI))/2.0 ## ERROR!!!!!
    return delta_kappa

## solve the dynamics using the consistency of mu and sigma    
def iidfull_mudelta_consistency(x,JE,JI,gmat,nparams):
    gee,gei,gie,gii = gmat[0],gmat[1],gmat[2],gmat[3]
    muxE,muxI     = x[0],x[1]
    sigx2E,sigx2I = x[2],x[3]
    NE,NI = nparams[0],nparams[1]
    N = NE+NI
    inner2meanE,inner2meanI = innerdeuxPhi(muxE,sigx2E),innerdeuxPhi(muxI,sigx2I)
    PhimeanE,PhimeanI = Phi(muxE,sigx2E),Phi(muxI,sigx2I)
    consistency = [x[2]-(gee**2*inner2meanE*NE/N+gei**2*inner2meanI*NI/N),x[0]-(JE*PhimeanE-JI*PhimeanI),x[3]-(gie**2*inner2meanE*NE/N+gii**2*inner2meanI*NI/N),x[1]-(JE*PhimeanE-JI*PhimeanI)]
    return consistency
def iidR1_mudelta_consistency(x,JE,JI,gmat,nparams):
    gee,gei,gie,gii = gmat[0],gmat[1],gmat[2],gmat[3]
    muxE,muxI     = x[0],x[1]
    sigx2E,sigx2I = x[2],x[3]
    NE,NI = nparams[0],nparams[1]
    N = NE+NI
    PhimeanE,PhimeanI = Phi(muxE,sigx2E),Phi(muxI,sigx2I)
    consistency = [x[2]-(gee**2*NE/N+gei**2*NI/N)/(JE-JI)**2*(JE*PhimeanE-JI*PhimeanI)**2,x[0]-(JE*PhimeanE-JI*PhimeanI),x[3]-(gie**2*NE/N+gii**2*NI/N)/(JE-JI)**2*(JE*PhimeanE-JI*PhimeanI)**2,x[1]-(JE*PhimeanE-JI*PhimeanI)]
    return consistency

## solve the dynamics using the consistency of kappa (sym)
def symperturbation(x,JE,JI,g0,etaset):
    # JE,JI,g0 = args
    muphi,delta0phi = x, x**2*g0**2/(JE-JI)**2
    delta_kappa = -x+(JE-JI)*Phi(muphi,delta0phi)
    sigmaE_term = g0**2*(JE*etaset[0]-JI*etaset[1])/(JE-JI)**2*derPhi(muphi,delta0phi)*x/2.0
    sigmaI_term = g0**2*(JE*etaset[1]-JI*etaset[5])/(JE-JI)**2*derPhi(muphi,delta0phi)*x/2.0
    delta_kappa = delta_kappa+(sigmaE_term+sigmaI_term)
    return delta_kappa
## solve the dynamics using the consistency of mu and sigma (sym)
def symfull_mudelta_consistency(x,JE,JI,g0):
    mux,sigx2 = x[0],x[1]
    inner2mean = innerdeuxPhi(mux,sigx2)
    Phimean = Phi(mux,sigx2)
    consistency = [x[1]-g0**2*inner2mean,x[0]-(JE-JI)*Phimean]
    return consistency

def symR1_mudelta_consistency(x,JE,JI,g0):
    mux,sigx2 = x[0],x[1]
    outer2mean = Phi(mux,sigx2)**2
    Phimean = Phi(mux,sigx2)
    consistency = [x[1]-g0**2*outer2mean,x[0]-(JE-JI)*Phimean]
    return consistency


### functions compare/how variance and mean co-change under the condition with iid Random Perturbation
def iid_muCdelta_consistency(x,JE,JI,g0, sigx2):
    mux = x[0]
    Phimean = Phi(mux,sigx2)
    consistency = x[0]-(JE-JI)*Phimean
    return consistency


def randbin(M,N,P):  
    return np.random.choice([0, 1], size=(M,N), p=[P, 1-P])

#%%
def blocklikeEtasymmetry(Jmat,nparams,EPSP,IPSP):
    J,JT = Jmat.copy(),Jmat.copy().T
    NE,NI = nparams[0],nparams[1]
    K = int(NE/NI)
    N = NE+NI
    ## ee
    AEE,AEET = np.squeeze(J[:NE,:NE])/EPSP,np.squeeze(JT[:NE,:NE])/EPSP
    etaEE = np.sum((AEE*AEET).flatten())/NE
    etaEE = etaEE/(np.sum((AEE*AEE).flatten())/NE)
    ## ei set
    etaEIset = np.zeros(K)
    for i in range(K):
        subJIE,subJEI = J[NE:,i*NI:(i+1)*NI]/EPSP,JT[NE:,i*NI:(i+1)*NI]/IPSP
        etaEIset[i]   = np.sum((subJIE*subJEI).flatten())/NI

        etaEIset[i]   = etaEIset[i]/(np.sum((subJIE*subJIE).flatten())/NI)
    ## ii
    AII,AIIT = np.squeeze(J[NE:,NE:])/IPSP,np.squeeze(JT[NE:,NE:])/IPSP
    etaII = np.sum((AII*AIIT).flatten())/NI
    etaII = etaII/(np.sum((AII*AII).flatten())/NI)
    
    return etaEE,etaEIset,etaII
def blocklikeEtasymmetry_binary(Jmat,nparams):
    J,JT = Jmat.copy(),Jmat.copy().T
    NE,NI = nparams[0],nparams[1]
    K = int(NE/NI)
    N = NE+NI
    ## ee
    AEE,AEET = np.squeeze(J[:NE,:NE]),np.squeeze(JT[:NE,:NE])
    etaEE = np.sum((AEE*AEET).flatten())/NE
    etaEE = etaEE/(np.sum((AEE*AEE).flatten())/NE)
    ## ei set
    etaEIset = np.zeros(K)
    for i in range(K):
        subJIE,subJEI = J[NE:,i*NI:(i+1)*NI],JT[NE:,i*NI:(i+1)*NI]
        etaEIset[i]   = np.sum((subJIE*subJEI).flatten())/NI

        etaEIset[i]   = etaEIset[i]/(np.sum((subJIE*subJIE).flatten())/NI)
    ## ii
    AII,AIIT = np.squeeze(J[NE:,NE:]),np.squeeze(JT[NE:,NE:])
    etaII = np.sum((AII*AIIT).flatten())/NI
    etaII = etaII/(np.sum((AII*AII).flatten())/NI)
    
    return etaEE,etaEIset,etaII

## For symmetric properties

In [ ]:
def generate_SymSpr_Mat(eta,prob,nsize):
    n1,n2=nsize[0],nsize[1]
    subProb = randbin(1,n1*n2,1-prob)
    # print("mean,var",np.mean(subProb),np.std(subProb)**2,prob*(1-prob))
    Jbsub= np.reshape(subProb,(n1,n2))
    JbsubT = Jbsub.copy().T

    idxTsym,idyTsym = np.where(JbsubT>0)
    nnonzero=len(idxTsym)
    # print('nonzero:',nnonzero)
    cutIdx = int(nnonzero*eta)
    # print("fnum:",cutIdx)
    Idxuse=random.sample(range(0,nnonzero),cutIdx)
    idxT,idyT = idxTsym[Idxuse],idyTsym[Idxuse]
    JbsubT_=np.zeros((n2,n1))
    JbsubT_[idxT,idyT]=1
    ## compensate
    idxTinv,idyTinv = np.where(JbsubT<1)
    zeros=len(idxTinv)
    cutIdxinv = (nnonzero-cutIdx)
    print("should / available snum:",cutIdxinv, zeros)
    if zeros<cutIdxinv:
        cutIdxinv = zeros#int(nnonzero*(1-eta))
    # print("snum:",cutIdxinv)
    # print("sumnum:",cutIdx+cutIdxinv,n1*n2*prob)
    Idxinvuse=random.sample(range(0,zeros),cutIdxinv)
    idxinvT,idyinvT = idxTinv[Idxinvuse],idyTinv[Idxinvuse]
    JbsubT_[idxinvT,idyinvT]=1 ## complementary 
    # print("IJ_mean,var",np.mean(np.squeeze(JbsubT_).flatten()),np.std(np.squeeze(JbsubT_).flatten())**2,prob*(1-prob))
    return Jbsub,JbsubT_ 

def generate_SymSpr_Mat_trial(Jiid,eta,prob,nsize):
    n1,n2=nsize[0],nsize[1]
    # subProb = randbin(1,n1*n2,1-prob)
    # # print("mean,var",np.mean(subProb),np.std(subProb)**2,prob*(1-prob))
    # Jbsub= np.reshape(subProb,(n1,n2))
    Jbsub = Jiid.copy()
    JbsubT = Jbsub.copy().T

    idxTsym,idyTsym = np.where(JbsubT>0)
    nnonzero=len(idxTsym)
    # print('nonzero:',nnonzero)
    cutIdx = int(nnonzero*eta)
    # print("fnum:",cutIdx)
    Idxuse=random.sample(range(0,nnonzero),cutIdx)
    idxT,idyT = idxTsym[Idxuse],idyTsym[Idxuse]
    JbsubT_=np.zeros((n2,n1))
    JbsubT_[idxT,idyT]=1
    ## compensate
    idxTinv,idyTinv = np.where(JbsubT<1)
    zeros=len(idxTinv)
    cutIdxinv = (nnonzero-cutIdx)
    print("should / available snum:",cutIdxinv, zeros)
    if zeros<cutIdxinv:
        cutIdxinv = zeros#int(nnonzero*(1-eta))
    # print("snum:",cutIdxinv)
    # print("sumnum:",cutIdx+cutIdxinv,n1*n2*prob)
    Idxinvuse=random.sample(range(0,zeros),cutIdxinv)
    idxinvT,idyinvT = idxTinv[Idxinvuse],idyTinv[Idxinvuse]
    JbsubT_[idxinvT,idyinvT]=1 ## complementary 
    # print("IJ_mean,var",np.mean(np.squeeze(JbsubT_).flatten()),np.std(np.squeeze(JbsubT_).flatten())**2,prob*(1-prob))
    return Jbsub,JbsubT_ 

def generate_SymSpr_MatEI(eta,prob,nsize):
    n1,n2=nsize[0],nsize[1]
    probie,probei=prob[0],prob[1]
    subProb = randbin(1,n1*n2,1-probie)
    Jbsub   = np.reshape(subProb,(n1,n2)) #subIE
    JbsubT  = Jbsub.copy().T #subEI

    idxTsym,idyTsym = np.where(JbsubT>0)
    nnonzero=len(idxTsym)
    cutIdx = int(nnonzero*eta)
    Idxuse=random.sample(range(0,nnonzero),cutIdx)
    idxT,idyT = idxTsym[Idxuse],idyTsym[Idxuse]
    JbsubT_=np.zeros((n2,n1))
    JbsubT_[idxT,idyT]=1
    ## compensate
    idxTinv,idyTinv = np.where(JbsubT<1)
    zeros=len(idxTinv)
    cutIdxinv = (nnonzero-cutIdx)
    if zeros<cutIdxinv:
        cutIdxinv = zeros#int(nnonzero*(1-eta))
    Idxinvuse=random.sample(range(0,zeros),cutIdxinv)
    idxinvT,idyinvT = idxTinv[Idxinvuse],idyTinv[Idxinvuse]
    JbsubT_[idxinvT,idyinvT]=1 ## complementary 
    return Jbsub,JbsubT_  

def generate_SymSpr_MatEI_trial(Jiid,eta,prob,nsize):
    n1,n2=nsize[0],nsize[1]
    probie,probei=prob[0],prob[1]
    # subProb = randbin(1,n1*n2,1-probie)
    # Jbsub   = np.reshape(subProb,(n1,n2)) #subIE
    Jbsub = Jiid.copy()
    JbsubT  = Jbsub.copy().T #subEI

    idxTsym,idyTsym = np.where(JbsubT>0)
    nnonzero=len(idxTsym)
    cutIdx = int(nnonzero*eta)
    Idxuse=random.sample(range(0,nnonzero),cutIdx)
    idxT,idyT = idxTsym[Idxuse],idyTsym[Idxuse]
    JbsubT_=np.zeros((n2,n1))
    JbsubT_[idxT,idyT]=1
    ## compensate
    idxTinv,idyTinv = np.where(JbsubT<1)
    zeros=len(idxTinv)
    cutIdxinv = (nnonzero-cutIdx)
    if zeros<cutIdxinv:
        cutIdxinv = zeros#int(nnonzero*(1-eta))
    Idxinvuse=random.sample(range(0,zeros),cutIdxinv)
    idxinvT,idyinvT = idxTinv[Idxinvuse],idyTinv[Idxinvuse]
    JbsubT_[idxinvT,idyinvT]=1 ## complementary 
    return Jbsub,JbsubT_  

In [ ]:
import math
import cmath
def ThreeSquare(x):
    if x.imag == 0:
        #实数
        m = x.real
        ans = -math.pow(-m,1/3) if m<0 else math.pow(m,1/3)
    else:
        #复数
        ans = x**(1/3)
    return ans

def RoundAns(x,num):
    if x.imag == 0:
        m = x.real
        ans = round(m,num)
    else:
        m = round(x.real,num)
        n = round(x.imag,num)
        ans = complex(m,n)
    return ans
    
def Cubic(args):
    a,b,c,d = args
    p = c/a-b**2/(3*a**2)
    q = d/a+2*b**3/(27*a**3)-b*c/(3*a**2)
    w = complex(-0.5,(3**0.5)/2)
    ww = complex(-0.5,-(3**0.5)/2)
    A = cmath.sqrt((q/2)**2+(p/3)**3)
    B = ThreeSquare(-q/2+A)
    C = ThreeSquare(-q/2-A)
    y1 = B+C
    y2 = w*B+ww*C
    y3 = ww*B+w*C
    D = b/(3*a)
    roots=[RoundAns(y1-D,6),RoundAns(y2-D,6),RoundAns(y3-D,6)]
    return roots

In [ ]:
'''
WITHOUT SYMMETRY, 
* increase g0 from zero to considerable
* constant JE,JI,a,b
'''
from functools import partial
# generate mean matrix
nrank=1
nn = 200
gnn = 4
Nt=np.array([gnn*nn,nn])
NE,NI=Nt[0],Nt[1]
N=NE+NI
## useful for determining g
Nparams=np.array([NE,NI])
Npercent = Nparams/N
nrank,ntrial,neta,nvec,nmaxFP=1,5,1,2,3
iktrial=0
''' Network Setting  '''
## heterogeneous random gain
ngEI=15
gEIseries = np.linspace(1,8.,ngEI)
print(gEIseries)

Cs = 60
kE,kI = gnn,1
## heterogeneous degree of symmetry: amplitudes and signs
coeffetaEsub=np.array([1.0,1.0,1.0])#
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]
## E->total ppercentEsub[0]/2.0,ppercentEsub[1]/2.0, (I) 1/2.0

'''  Recording Variables. '''
## mean connectivity
RAmvecseries,LAmvecseries = np.zeros((ngEI,N)),np.zeros((ngEI,N))
BAmvseries = np.zeros((ngEI,N),dtype=complex)

## perturbed + symmetric eigenvectors
ZSparserandom = np.zeros((ngEI,ntrial,N,N))
## eigenvectors, decomposing Sparse Matrix
Rsprvecseries,Lsprvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RsprvecTseries,LsprvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Bsprvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
## Gaussian iid random connections
gYbasexee=np.zeros((ngEI,N,N)) 
Riidvecseries,Liidvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RiidvecTseries,LiidvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Biidvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
## corresponding statistics \mu and \sigma \cov
armu,sigrcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2)) # 2 for E and I
almu,siglcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
siglr = np.zeros((ngEI,ntrial,2))

''' Nullclines parameters  '''
## compare three models' dynamcis
kappaintersect_Sparse = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_Full   = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_R1     = np.zeros((ngEI,ntrial,nmaxFP*2))

''' simulating parameters '''
tt = np.linspace(0,150,500)
dt = tt[2]-tt[1]
ntt = len(tt)
## compare three neuron population activities
xfpseries_Sparse = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_Full   = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_R1     = np.zeros((ngEI,ntrial,N,ntt))
''' All random matrices for each trials '''
''' ## Three \bar{J} cases '''
EPSP,a,b=0.02,0,0# JE PER EPSP 

''' Iterative Processing '''
for ig, gEI in enumerate (gEIseries):
    print('>>>>>>> simulating neuronal activity ... >>>>>>')
    ## mean matrix, reference eigenvalue and eigenvectors
    Am = np.zeros((N,N))
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    Am[:,:NE],Am[:,NE:] = JE0/NE,-JI0/NI
    # meigvecAm,neigvecAm,eigvAm = decompunperturbedEI(Am)
    # xAm,yAm=np.reshape(meigvecAm[:,0],(N,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(N,1))
    xAm,yAm = np.zeros((N,1)),np.zeros((N,1))
    eigvAm = np.zeros(N)
    eigvAm[0]=(JE0-JI0)
    xAm[:NE,0],xAm[NE:,0] = 1/np.sqrt(N),1/np.sqrt(N)
    yAm[:NE,0],yAm[NE:,0] = N*JE0/NE/np.sqrt(N)/eigvAm[0],-N*JI0/NI/np.sqrt(N)/eigvAm[0]
    RAmvecseries[ig,:],LAmvecseries[ig,:] = xAm[:,0],yAm[:,0]
    BAmvseries[ig,:] = eigvAm[:]

    for iktrial in range(ntrial):
        ## dilute/sparse matrix 
        Jdilute = np.zeros((N,N))
        JE,JI = EPSP,IPSP#JE0/Cs/kE,JI0/Cs/kI
        for inn in range(NE+NI):
            # for NE
            Einput = randbin(1,NE,1-kE*Cs/NE)
            Jdilute[inn,:NE] = Einput*JE
            # for NE
            Iinput = randbin(1,NI,1-kI*Cs/NI)
            Jdilute[inn,NE:] = Iinput*(-JI)

        # overall
        J = Jdilute.copy()
        ## subtract mean value
        # X = Jdilute.copy() - Am.copy() # random matrix, with zero mean 
        X = Jdilute.copy() # random matrix, with zero mean 
        X[:NE,:NE] = X[:NE,:NE] - np.mean(X[:NE,:NE].flatten())
        X[:NE,NE:] = X[:NE,NE:] - np.mean(X[:NE,NE:].flatten())
        X[NE:,:NE] = X[NE:,:NE] - np.mean(X[NE:,:NE].flatten())
        X[NE:,NE:] = X[NE:,NE:] - np.mean(X[NE:,NE:].flatten())
        ''' Original Sparse Network '''
        eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xAm,yAm,xAm,yAm,nparams = Nparams,sort=0)
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0,ynorm0 = xnorm0*np.sqrt(N),ynorm0*np.sqrt(N)
        Rsprvecseries[ig,iktrial,:,0],Lsprvecseries[ig,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
        Bsprvseries[ig,iktrial,:]=eigvJ.copy()

        axrmu,aylmu,sigxr,sigyl,sigcov = numerical_stats(xnorm0,ynorm0,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJ,nrank,2,ppercent=Npercent)
        armu[ig,iktrial,:],almu[ig,iktrial,:] = axrmu[:,0],aylmu[:,0]
        sigrcov[ig,iktrial,:],siglcov[ig,iktrial,:]= sigxr[:,0],sigyl[:,0]
        siglr[ig,iktrial,:] = sigcov[:,0,0]
        
        ## Rank One Approximation, Nullclines of \kappa
        xnorm0, ynorm0 = Rsprvecseries[ig,iktrial,:,0].copy(),Lsprvecseries[ig,iktrial,:,0]
        xnorm0, ynorm0 = np.reshape(xnorm0,(N,1)),np.reshape(ynorm0,(N,1))
        xmu,ymu = armu[ig,iktrial,:].copy(),almu[ig,iktrial,:].copy()
        xsig,ysig = sigrcov[ig,iktrial,:].copy(),siglcov[ig,iktrial,:].copy()
        yxcov = siglr[ig,iktrial,:].copy()

        ''' Sparse Connectivity -- Dynamics '''
        Jpt = J.copy()
        xinit = np.random.normal(0,1E-2,(1,N))
        xinit = np.squeeze(xinit)
        xtemporal = odesimulation(tt,xinit,Jpt,0)
        xfpseries_Sparse[ig,iktrial,:,:] = xtemporal.T.copy()
        ## 2 KAPPA_0
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_Sparse[ig,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        kappaintersect_Sparse[ig,iktrial,:3]=kappanum[:].copy()

        ''' Rank one model -- Dynamics '''
        ''' use perturbation theorem to calculate the perturbed vectors ''' 
        xnormt, ynormt = X.copy()@xAm.copy()/eigvAm[0], X.copy().T@yAm.copy()/eigvAm[0]#yAm.copy().T@X/eigvAm[0]*np.sqrt(N)
        xnormt, ynormt = xnormt+xAm.copy(), ynormt+yAm.copy()
        ## normalize 
        xnormt = xnormt.copy()/np.linalg.norm(xnormt.copy())
        ynormt = ynormt.copy()
        _,_,xnormt,ynormt=Normalization(xnormt.copy(),ynormt.copy(),xAm.copy(),yAm.copy(),nparams=Nparams,sort=0,nrank=1)
        RsprvecTseries[ig,iktrial,:,0],LsprvecTseries[ig,iktrial,:,0]=xnormt[:,0].copy(),ynormt[:,0].copy()
        xnormt,ynormt = xnormt*np.sqrt(N),ynormt*np.sqrt(N)
        # xnormt, ynormt = X.copy()@xAm.copy()/eigvAm[0]*np.sqrt(NE*2), X.copy().T@yAm.copy()/eigvAm[0]*np.sqrt(NE*2)#yAm.copy().T@X/eigvAm[0]*np.sqrt(N)
        # xnormt, ynormt = xnormt+xAm.copy()*np.sqrt(NE*2), ynormt+yAm.copy()*np.sqrt(NE*2)
        # ReigvecTseries[idxg,iktrial,:,0],LeigvecTseries[idxg,iktrial,:,0]=xnormt[:,0].copy(),ynormt[:,0].copy()
        r1Matt = np.real(xnormt@ynormt.T)
        r1Matt = r1Matt*np.real(eigvAm[0])/N
        ### START SIMULATING AND CALCULATE KAPPA AND xfpsereis_R1
        ## use the same initial values
        xtemporal = odesimulation(tt,xinit,r1Matt,0)
        xfpseries_R1[ig,iktrial,:,:] = xtemporal.T.copy()
        ''' kappa dynamics    '''
        ## 2 populations 
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_R1[ig,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            # do not change
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        kappaintersect_R1[ig,iktrial,:3]=kappanum[:].copy()

        ''' Full Gaussian iid model -- Dynamics '''
        ## variance
        gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs*N/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs*N/NI
        alphaNEI = Npercent.copy()
        Xgiid =iidGaussian([0,1.0/np.sqrt(N)],[N,N])
        Xgiid[:NE,:NE]*=np.sqrt(gE2)
        Xgiid[:NE,NE:]*=np.sqrt(gI2)
        Xgiid[NE:,:NE]*=np.sqrt(gE2)
        Xgiid[NE:,NE:]*=np.sqrt(gI2)

        Jgiid = Xgiid.copy()+Am.copy()

        ''' Original Sparse Network '''
        eigvJiid,leigveciid,reigveciid,xnorm0iid,ynorm0iid=decompNormalization(Jgiid,xAm,yAm,xAm,yAm,nparams = Nparams,sort=0)
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0iid,ynorm0iid = xnorm0iid*np.sqrt(N),ynorm0iid*np.sqrt(N)
        Riidvecseries[ig,iktrial,:,0],Liidvecseries[ig,iktrial,:,0]=xnorm0iid[:,0].copy(),ynorm0iid[:,0].copy()
        Biidvseries[ig,iktrial,:]=eigvJiid.copy()
        axrmuiid,aylmuiid,sigxriid,sigyliid,sigcoviid = numerical_stats(xnorm0iid,ynorm0iid,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJiid,nrank,2,ppercent=Npercent)

        xtemporal = odesimulation(tt,xinit,Jgiid,0)
        xfpseries_Full[ig,iktrial,:,:] = xtemporal.T.copy()
        ''' kappa dynamics    '''
        ## 2 populations 
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_Full[ig,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((aylmuiid[0,0]*eigvJiid[0])*(yAm[0,0]*eigvAm[0])>0):
            # do not change
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        kappaintersect_Full[ig,iktrial,:3]=kappanum[:].copy()

## eigenvalue spectrum

In [ ]:
'''
with symmetry, subcircuit
eigvJ0
'''
xticks = np.linspace(-1.0,2.0,1)
xlims = [-1.5,1.5]
yticks = np.linspace(-1,1,2)
ylims = [-1.5,1.5]
theta = np.linspace(0, 2 * np.pi, 200)
idxc=0
cm='rbg'
print(ntrial)
for ig,gEI in enumerate (gEIseries):
    figtspt,axtspt=plt.subplots(figsize=(6,3))
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    ## variance
    gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs/NI
    gE2,gI2 = gE2*N,gI2*N
    alphaNEI = Npercent.copy()
    gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
    eigvgm,eigvecgm=la.eig(gMmat) 
    r_g2=np.max(eigvgm)
    r_g = np.sqrt(r_g2)
    print(r_g)
    for idxtrial in range(ntrial):
        axtspt.scatter(np.real(Bsprvseries[ig,idxtrial,:]),np.imag(Bsprvseries[ig,idxtrial,:]),s=5,c='blue',alpha=0.25) # >>>>>>>>>>>>>>
    axtspt.scatter(np.real(Bsprvseries[ig,:,0]),np.imag(Bsprvseries[ig,:,0])-0.2*idxc,s=50,c='red',marker='^',alpha=0.5) # >>>>>>>>>>>>
    # axtspt.scatter(np.real(Bsprvseries[idxgavg,:,1]),np.imag(Bsprvseries[idxgavg,:,1])-0.2*idxc,s=20,c='darkblue',marker='^',alpha=0.5) # >>>>>>>>>>>>
    axtspt.set_aspect('equal')
    axtspt.plot(np.ones(2)*BAmvseries[ig,0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
    axtspt.spines['right'].set_color('none')
    axtspt.spines['top'].set_color('none')
    axtspt.xaxis.set_ticks_position('bottom')
    axtspt.spines['bottom'].set_position(('data', 0))
    # axtspt.set_xlim(xlims)
    # axtspt.set_xticks(xticks)
    axtspt.set_ylim(ylims)
    axtspt.set_yticks(yticks)
    axtspt.set_aspect('equal')
    eta=0
    longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
    xr = longaxis*np.cos(theta)
    yr = shortaxis*np.sin(theta)
    axtspt.plot(xr, yr, color="red", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>

In [ ]:
'''
with symmetry, subcircuit
eigvJ0
'''
xticks = np.linspace(-1.0,2.0,1)
xlims = [-1.5,1.5]
yticks = np.linspace(-1,1,2)
ylims = [-1.5,1.5]
theta = np.linspace(0, 2 * np.pi, 200)
idxc=0
cm='rbg'
print(ntrial)
for ig,gEI in enumerate (gEIseries):
    figtspt,axtspt=plt.subplots(figsize=(6,3))
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    ## variance
    gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs/NI
    gE2,gI2 = gE2*N,gI2*N
    alphaNEI = Npercent.copy()
    gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
    eigvgm,eigvecgm=la.eig(gMmat) 
    r_g2=np.max(eigvgm)
    r_g = np.sqrt(r_g2)
    print(r_g)
    for idxtrial in range(ntrial):
        axtspt.scatter(np.real(Biidvseries[ig,idxtrial,:]),np.imag(Biidvseries[ig,idxtrial,:]),s=5,c='blue',alpha=0.25) # >>>>>>>>>>>>>>
    axtspt.scatter(np.real(Biidvseries[ig,:,0]),np.imag(Biidvseries[ig,:,0])-0.2*idxc,s=50,c='red',marker='^',alpha=0.5) # >>>>>>>>>>>>
    # axtspt.scatter(np.real(Bsprvseries[idxgavg,:,1]),np.imag(Bsprvseries[idxgavg,:,1])-0.2*idxc,s=20,c='darkblue',marker='^',alpha=0.5) # >>>>>>>>>>>>
    axtspt.set_aspect('equal')
    axtspt.plot(np.ones(2)*BAmvseries[ig,0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
    axtspt.spines['right'].set_color('none')
    axtspt.spines['top'].set_color('none')
    axtspt.xaxis.set_ticks_position('bottom')
    axtspt.spines['bottom'].set_position(('data', 0))
    # axtspt.set_xlim(xlims)
    # axtspt.set_ylim(ylims)
    # axtspt.set_xticks(xticks)
    # axtspt.set_yticks(yticks)
    axtspt.set_aspect('equal')
    eta=0
    longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
    xr = longaxis*np.cos(theta)
    yr = shortaxis*np.sin(theta)
    axtspt.plot(xr, yr, color="red", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>

In [ ]:
print(gEIseries[4],gEIseries[6],gEIseries[14])

## statistics of eigenvectors

In [20]:
'''
loading patterns
for ensemble-averaging results
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
xticks = np.linspace(-80,20,3)
xlims = [-90,40]
yticks = np.linspace(0,2,2)
ylims = [-2,3]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''

iktrial=0
ig=4
alphaval=0.10
edgv='black'
cm='br'
fig,ax=plt.subplots(figsize=(5,2))  
nrank=1
ntrial = np.shape(Rsprvecseries)[1]
NE=int(NE)

'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)

mEvec,mIvec,nEvec,nIvec=np.squeeze(Rsprvecseries[ig,:,:NE,0]),np.squeeze(Rsprvecseries[ig,:,NE:,0]),np.squeeze(Lsprvecseries[ig,:,:NE,0]),np.squeeze(Lsprvecseries[ig,:,NE:,0])
eigVforn = np.mean(Bsprvseries[ig,:,0])
nEvec,nIvec = nEvec*eigVforn,nIvec*eigVforn
mEvec,mIvec,nEvec,nIvec=mEvec.flatten(),mIvec.flatten(),nEvec.flatten(),nIvec.flatten()

scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NI*ntrial,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    print('shape:',np.shape(mEvec),np.shape(meanmE))
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

ax.scatter(nIvec,mIvec,s=5.0,c='b',alpha=0.2)#alphaval)#cm[1],alpha=alphaval)
ax.scatter(nEvec,mEvec,s=5.0,c='r',alpha=0.2)#alphaval)#cm[0],alpha=alphaval)

ax.plot([meannE[0],meannE[0]+np.sqrt(N)*dirvecE[0,0]],[meanmE[0],meanmE[0]+np.sqrt(N)*dirvecE[1,0]],color='gray',linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+np.sqrt(N)*dirvecE[0,1]],[meanmE[0],meanmE[0]+np.sqrt(N)*dirvecE[1,1]],color='gray',linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+np.sqrt(N)*dirvecI[0,0]],[meanmI[0],meanmI[0]+np.sqrt(N)*dirvecI[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+np.sqrt(N)*dirvecI[0,1]],[meanmI[0],meanmI[0]+np.sqrt(N)*dirvecI[1,1]],color=edgv,linestyle='--',linewidth=1.5)
confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=edgv)
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=edgv)


ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
# ax.set_aspect('equal')

shape: (4000,) (1,)
(2, 2)
(2, 2)


<ipython-input-20-ad1bc71d61f3>:52: ComplexWarning: Casting complex values to real discards the imaginary part
  meannE[irank]=np.mean(nEvec)
<ipython-input-20-ad1bc71d61f3>:59: ComplexWarning: Casting complex values to real discards the imaginary part
  meannI[irank]=np.mean(nIvec)
<ipython-input-20-ad1bc71d61f3>:67: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
<ipython-input-20-ad1bc71d61f3>:68: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]


## theoretical $\sigma_{m^2}$ and $\sigma_{n^2}$

In [21]:
'''
loading patterns
for ensemble-averaging results
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
ntrial = np.shape(Rsprvecseries)[1]
mean_theo_mvec,mean_theo_nvec = np.zeros((ngEI,2)),np.zeros((ngEI,2))
mean_num_mvec,mean_num_nvec = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
var_theo_mvec,var_theo_nvec = np.zeros((ngEI,2)),np.zeros((ngEI,2))
var_num_mvec,var_num_nvec = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
for ig, gEI in enumerate (gEIseries): 
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    ## variance
    gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs/NI
    gE2,gI2 = gE2*N,gI2*N
    alphaNEI = Npercent.copy()
    gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
    eigvgm,eigvecgm=la.eig(gMmat) 
    '''loading vector changing'''
    for iktrial in range(ntrial):
        ## numerical variances
        mEvec,mIvec,nEvec,nIvec=np.squeeze(Rsprvecseries[ig,iktrial,:NE,0]),np.squeeze(Rsprvecseries[ig,iktrial,NE:,0]),np.squeeze(Lsprvecseries[ig,iktrial,:NE,0]),np.squeeze(Lsprvecseries[ig,iktrial,NE:,0])
        eigVforn = np.mean(Bsprvseries[ig,iktrial,0])
        # print('eigenvalue:',JE0-JI0,eigVforn)
        nEvec,nIvec = nEvec*eigVforn,nIvec*eigVforn
        mEvec,mIvec,nEvec,nIvec=mEvec.flatten(),mIvec.flatten(),nEvec.flatten(),nIvec.flatten()
        scale_std=3.0
        for irank in range(nrank):
            mean_num_mvec[ig,iktrial,0]=np.mean(mEvec)
            var_num_mvec[ig,iktrial,0] = np.std(mEvec)**2

            mean_num_mvec[ig,iktrial,1]=np.mean(mIvec)
            var_num_mvec[ig,iktrial,1] = np.std(mIvec)**2
            
            mean_num_nvec[ig,iktrial,0]=np.mean(nEvec)
            var_num_nvec[ig,iktrial,0] = np.std(nEvec)**2

            mean_num_nvec[ig,iktrial,1]=np.mean(nIvec)
            var_num_nvec[ig,iktrial,1] = np.std(nIvec)**2

    ## theoretical variances
    ''' calculate directions '''
    mean_theo_mvec[ig,:] = 1
    mean_theo_nvec[ig,0],mean_theo_nvec[ig,1] = 2*JE0,-2*JI0
    
    gQE,gQI = EPSP*np.sqrt(kE*Cs/NE*(1-kE*Cs/NE))*np.sqrt(N),IPSP*np.sqrt(kI*Cs/NI*(1-kI*Cs/NI))*np.sqrt(N)
    # print('gvalue:',gQE,gQI)
    # print('LAMBDA:',JE0-JI0,BAmvseries[ig,0])
    lambda0=(JE0-JI0)
    var_theo_mvec[ig,0],var_theo_mvec[ig,1] = (gQE**2*NE/N+gQI**2*NI/N)/lambda0**2,(gQE**2*NE/N+gQI**2*NI/N)/lambda0**2
    var_theo_nvec[ig,0],var_theo_nvec[ig,1] = ((N*JE0/NE)**2*NE*(gQE**2/N)+(N*JI0/NI)**2*NI*(gQE**2/N))/lambda0**2,((N*JE0/NE)**2*NE*(gQI**2/N)+(N*JI0/NI)**2*NI*(gQI**2/N))/lambda0**2
    # print('varn:',var_theo_nvec[ig,:],gQI/gQE)

fign, axn = plt.subplots(2,1,figsize=(6,3))
xticks = np.linspace(2,8,3)
xlims = [0,9]
yticks0 = np.linspace(0,30,2)
ylims0 = [-5,40]
yticks1 = np.linspace(0,500,2)
ylims1 = [-50,600]
axn[0].plot(gEIseries,var_theo_nvec[:,0],c='red',linestyle='-.')
axn[1].plot(gEIseries,var_theo_nvec[:,1],c='blue',linestyle='-.')
for iktrial in range(ntrial):
    axn[0].plot(gEIseries,var_num_nvec[:,iktrial,0],'^',color='red',markersize=3) 
    axn[1].plot(gEIseries,var_num_nvec[:,iktrial,1],'^',color='blue',markersize=3)
for i in range(2):
    axn[i].set_xlim(xlims)
    axn[i].set_xticks(xticks)
axn[0].set_ylim(ylims0)
axn[0].set_yticks(yticks0)
axn[1].set_ylim(ylims1)
axn[1].set_yticks(yticks1)

fign, axm = plt.subplots(2,1,figsize=(6,3))
xticks = np.linspace(2,8,3)
xlims = [0,9]
yticks = np.linspace(0,1,2)
ylims = [-0.2,1.2]
for iktrial in range(ntrial):
    axm[0].plot(gEIseries,var_num_mvec[:,iktrial,0],'^',color='red',markersize=3) 
    axm[1].plot(gEIseries,var_num_mvec[:,iktrial,1],'^',color='blue',markersize=3)
axm[0].plot(gEIseries,var_theo_mvec[:,0],c='red',linestyle='-.')
axm[1].plot(gEIseries,var_theo_mvec[:,1],c='blue',linestyle='-.')
for i in range(2):
    axm[i].set_xlim(xlims)
    axm[i].set_xticks(xticks)
    axm[i].set_ylim(ylims)
    axm[i].set_yticks(yticks)

<ipython-input-21-24b14acb62c6>:37: ComplexWarning: Casting complex values to real discards the imaginary part
  mean_num_nvec[ig,iktrial,0]=np.mean(nEvec)
<ipython-input-21-24b14acb62c6>:40: ComplexWarning: Casting complex values to real discards the imaginary part
  mean_num_nvec[ig,iktrial,1]=np.mean(nIvec)
<ipython-input-21-24b14acb62c6>:52: RuntimeWarning: divide by zero encountered in double_scalars
  var_theo_mvec[ig,0],var_theo_mvec[ig,1] = (gQE**2*NE/N+gQI**2*NI/N)/lambda0**2,(gQE**2*NE/N+gQI**2*NI/N)/lambda0**2
<ipython-input-21-24b14acb62c6>:53: RuntimeWarning: divide by zero encountered in double_scalars
  var_theo_nvec[ig,0],var_theo_nvec[ig,1] = ((N*JE0/NE)**2*NE*(gQE**2/N)+(N*JI0/NI)**2*NI*(gQE**2/N))/lambda0**2,((N*JE0/NE)**2*NE*(gQI**2/N)+(N*JI0/NI)**2*NI*(gQI**2/N))/lambda0**2


In [ ]:
print('vector:',LAmvecseries[0,::nn]*BAmvseries[0,0]*np.sqrt(N))
print('vector:',LAmvecseries[1,::nn]*BAmvseries[1,0]*np.sqrt(N))
print('theo:',N*EPSP/NE*kE*Cs,N*EPSP/NI*gEIseries*kI*Cs)
print('eigenvalue:',BAmvseries[:,:2])
print('overlap:',np.sum(LAmvecseries[0,:]*BAmvseries[0,0]*RAmvecseries[0,:]),np.sum(LAmvecseries[1,:]*BAmvseries[1,0]*RAmvecseries[1,:]))

print('vector:',RAmvecseries[0,::nn]*np.sqrt(N))
print('vector:',RAmvecseries[1,::nn]*np.sqrt(N))
print('theo:',2*EPSP*kE*Cs,2*EPSP*gEIseries*kI*Cs)

## dynamics $x$
* $\kappa$
* $\mu_x$ and $\Delta_x$

In [15]:
## self-consistency in \kappa, for rank one approximation 
kappa_theo_iid = np.zeros((ngEI,3))
for ig,gEI in enumerate (gEIseries):
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs*N/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs*N/NI
    gE,gI = np.sqrt(gE2),np.sqrt(gI2)
    gmat = np.array([gE,gI,gE,gI])
    init_k=1.5
    kappa_theo_iid[ig,0]= fsolve(iidperturbation,init_k,args=(JE0,JI0,gmat,Nparams))
    init_k=0.0
    kappa_theo_iid[ig,1]= fsolve(iidperturbation,init_k,args=(JE0,JI0,gmat,Nparams))
    init_k=-1.5
    kappa_theo_iid[ig,2]= fsolve(iidperturbation,init_k,args=(JE0,JI0,gmat,Nparams))
## plot kappa change with JE ##
fig,ax = plt.subplots(figsize=(6,2))
xticks = np.linspace(2,8,3)
xlims = [0,9]
yticks = np.linspace(-3,3,2)
ylims = [-4.,4.0]
ax.plot(gEIseries,kappa_theo_iid[:,0],color='black',linewidth=1.5)
ax.plot(gEIseries,kappa_theo_iid[:,1],color='gray',linewidth=1.5)
ax.plot(gEIseries,kappa_theo_iid[:,2],color='black',linewidth=1.5)
for iktrial in range(ntrial):
    ax.plot(gEIseries,kappaintersect_Full[:,iktrial,0],'^',color='blue',markersize=5)
    ax.plot(gEIseries,kappaintersect_R1[:,iktrial,0],'^',color='green',markersize=5)
    ax.plot(gEIseries,kappaintersect_Sparse[:,iktrial,0],'^',color='red',markersize=5)
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)
    

<ipython-input-5-b61ccc80ff8e>:35: RuntimeWarning: divide by zero encountered in true_divide
  muphi,delta0phiE,delta0phiI = x, x**2*(gee**2*NE/N+gei**2*NI/N)/(JE-JI)**2, x**2*(gie**2*NE/N+gii**2*NI/N)/(JE-JI)**2
<ipython-input-5-b61ccc80ff8e>:35: RuntimeWarning: invalid value encountered in true_divide
  muphi,delta0phiE,delta0phiI = x, x**2*(gee**2*NE/N+gei**2*NI/N)/(JE-JI)**2, x**2*(gie**2*NE/N+gii**2*NI/N)/(JE-JI)**2
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [16]:
### test two expressions of variance ##
## numerically calculate the variance and mean, using xfpseries_Full and xfpseries_R1
iktrial=0
dshift = 0.2
variance_spr_num, mu_spr_num =np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
variance_full_num, mu_full_num =np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
variance_R1_num, mu_R1_num = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
variance_spr_theo, mu_spr_theo =np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
variance_full_theo, mu_full_theo =np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
variance_R1_theo, mu_R1_theo = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))


for ig,gEI in enumerate (gEIseries):
    for iktrial in range(ntrial):
        ## numerical results for Full Mat
        variance_full_num[ig,iktrial,0],variance_full_num[ig,iktrial,1]=np.std(xfpseries_Full[ig,iktrial,:NE,-1])**2,np.std(xfpseries_Full[ig,iktrial,NE:N,-1])**2
        mu_full_num[ig,iktrial,0],mu_full_num[ig,iktrial,1]=np.mean(xfpseries_Full[ig,iktrial,:NE,-1]),np.mean(xfpseries_Full[ig,iktrial,NE:N,-1])
        ## numerical results for Sparse Mat
        variance_spr_num[ig,iktrial,0],variance_spr_num[ig,iktrial,1]=np.std(xfpseries_Sparse[ig,iktrial,:NE,-1])**2,np.std(xfpseries_Sparse[ig,iktrial,NE:N,-1])**2
        mu_spr_num[ig,iktrial,0],mu_spr_num[ig,iktrial,1]=np.mean(xfpseries_Sparse[ig,iktrial,:NE,-1]),np.mean(xfpseries_Sparse[ig,iktrial,NE:N,-1])
        ## numerical results for Rank one Appriximation Mat
        variance_R1_num[ig,iktrial,0],variance_R1_num[ig,iktrial,1]=np.std(xfpseries_R1[ig,iktrial,:NE,-1])**2,np.std(xfpseries_R1[ig,iktrial,NE:N,-1])**2
        mu_R1_num[ig,iktrial,0],mu_R1_num[ig,iktrial,1]=np.mean(xfpseries_R1[ig,iktrial,:NE,-1]),np.mean(xfpseries_R1[ig,iktrial,NE:N,-1])

for ig,gEI in enumerate (gEIseries):
    ## keep the mean value to be the same -- JE0,JI0.
    ## the connectivity strength of sparse connections
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs*N/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs*N/NI
    gE,gI = np.sqrt(gE2),np.sqrt(gI2)
    gmat = np.array([gE,gI,gE,gI])
    # ## theoretical    
    deltainit,meaninit = np.mean(variance_full_num[ig,:,0]),kappa_theo_iid[ig,0]#np.mean(kappaintersect[idxg,:,0])
    ## JEI0 are effective mean connectivity weights, the nonzero connectivity strengths are JE0/(Cs/NE),JI0/(Cs/NI)
    statsfull = fsolve(iidfull_mudelta_consistency,[meaninit,meaninit,deltainit,deltainit],args=(JE0,JI0,gmat,Nparams))
    mu_full_theo[ig,:,0],variance_full_theo[ig,:,0]= statsfull[0]*np.ones(ntrial),statsfull[2]*np.ones(ntrial)
    mu_full_theo[ig,:,1],variance_full_theo[ig,:,1]= statsfull[1]*np.ones(ntrial),statsfull[3]*np.ones(ntrial)

    statsR1 = fsolve(iidR1_mudelta_consistency,[meaninit,meaninit,deltainit,deltainit],args=(JE0,JI0,gmat,Nparams))
    mu_R1_theo[ig,:,0],variance_R1_theo[ig,:,0]= statsR1[0]*np.ones(ntrial),statsR1[2]*np.ones(ntrial)
    mu_R1_theo[ig,:,1],variance_R1_theo[ig,:,1]= statsR1[1]*np.ones(ntrial),statsR1[3]*np.ones(ntrial)

    # ## theoretical    
    deltainit,meaninit = np.mean(variance_full_num[ig,:,0]),kappa_theo_iid[ig,2]#np.mean(kappaintersect[idxg,:,0])
    ## JEI0 are effective mean connectivity weights, the nonzero connectivity strengths are JE0/(Cs/NE),JI0/(Cs/NI)
    statsfull = fsolve(iidfull_mudelta_consistency,[meaninit,meaninit,deltainit,deltainit],args=(JE0,JI0,gmat,Nparams))
    mu_full_theo[ig,0,0],variance_full_theo[ig,0,0]= statsfull[0],statsfull[2]
    mu_full_theo[ig,0,1],variance_full_theo[ig,0,1]= statsfull[1],statsfull[3]

    statsR1 = fsolve(iidR1_mudelta_consistency,[meaninit,meaninit,deltainit,deltainit],args=(JE0,JI0,gmat,Nparams))
    mu_R1_theo[ig,0,0],variance_R1_theo[ig,0,0]= statsR1[0],statsR1[2]
    mu_R1_theo[ig,0,1],variance_R1_theo[ig,0,1]= statsR1[1],statsR1[3]

## Figures reflecting how mean and variance change with random gain of iid Gaussian matrix
fig,ax2 = plt.subplots(2,1,figsize=(6,4))
xticks = np.linspace(2,8,3)
xlims = [0,9]
yticks = np.linspace(-3,3,2)
ylims = [-4,4]
for iktrial in range(ntrial):
    # ax2[0].scatter(gEIseries-dshift,np.abs(mu_full_num[:,iktrial,0]),s=8.0,color='red',edgecolors='red',marker='^',alpha=0.5)
    # ax2[1].scatter(gEIseries-dshift,np.abs(mu_full_num[:,iktrial,1]),s=8.0,color='blue',edgecolors='blue',marker='^',alpha=0.5)
    # ax2[0].scatter(gEIseries+dshift,np.abs(mu_R1_num[:,iktrial,0]),s=8.0,color='orange',edgecolors='orange',marker='s',alpha=0.5)
    # ax2[1].scatter(gEIseries+dshift,np.abs(mu_R1_num[:,iktrial,1]),s=8.0,color='green',edgecolors='green',marker='s',alpha=0.5)
    # ax2[0].scatter(gEIseries+dshift,np.abs(mu_spr_num[:,iktrial,0]),s=8.0,color='gray',edgecolors='gray',marker='s',alpha=0.5)
    # ax2[1].scatter(gEIseries+dshift,np.abs(mu_spr_num[:,iktrial,1]),s=8.0,color='gray',edgecolors='gray',marker='s',alpha=0.5)
    ax2[0].scatter(gEIseries-dshift,(mu_full_num[:,iktrial,0]),s=8.0,color='red',edgecolors='red',marker='^',alpha=0.5)
    ax2[1].scatter(gEIseries-dshift,(mu_full_num[:,iktrial,1]),s=8.0,color='blue',edgecolors='blue',marker='^',alpha=0.5)
    ax2[0].scatter(gEIseries+dshift,(mu_R1_num[:,iktrial,0]),s=8.0,color='orange',edgecolors='orange',marker='s',alpha=0.5)
    ax2[1].scatter(gEIseries+dshift,(mu_R1_num[:,iktrial,1]),s=8.0,color='green',edgecolors='green',marker='s',alpha=0.5)
    ax2[0].scatter(gEIseries+dshift,(mu_spr_num[:,iktrial,0]),s=8.0,color='gray',edgecolors='gray',marker='s',alpha=0.5)
    ax2[1].scatter(gEIseries+dshift,(mu_spr_num[:,iktrial,1]),s=8.0,color='gray',edgecolors='gray',marker='s',alpha=0.5)

ax2[0].plot(gEIseries,(mu_full_theo[:,1,0]),color='red',linewidth=1.5,label=r'$\mu_F^E$',alpha=0.5)
ax2[1].plot(gEIseries,(mu_full_theo[:,1,1]),color='blue',linewidth=1.5,label=r'$\mu_F^I$',alpha=0.5)
ax2[0].plot(gEIseries,(mu_R1_theo[:,1,0]),color='orange',linewidth=1.5,label=r'$\mu_{R1}^E$',alpha=0.5)
ax2[1].plot(gEIseries,(mu_R1_theo[:,1,1]),color='green',linewidth=1.5,label=r'$\mu_{R1}^I$',alpha=0.5)

ax2[0].plot(gEIseries,(mu_full_theo[:,0,0]),color='red',linewidth=1.5,alpha=0.5)
ax2[1].plot(gEIseries,(mu_full_theo[:,0,1]),color='blue',linewidth=1.5,alpha=0.5)
ax2[0].plot(gEIseries,(mu_R1_theo[:,0,0]),color='orange',linewidth=1.5,alpha=0.5)
ax2[1].plot(gEIseries,(mu_R1_theo[:,0,1]),color='green',linewidth=1.5,alpha=0.5)


for i in range(2):     
        # ax2[i].set_xlim(xlims)
        # ax2[i].set_ylim(ylims)
        # ax2[i].set_xticks(xticks)
        # ax2[i].set_yticks(yticks)
        ax2[i].legend()


fig,ax2 = plt.subplots(2,1,figsize=(6,4))
xticks = np.linspace(2,8,3)
xlims = [0,9]
yticks = np.linspace(0,0.15,2)
ylims = [-0.01,0.18]
for iktrial in range(ntrial):
    ax2[0].scatter(gEIseries-dshift,variance_full_num[:,iktrial,0],s=8.0,color='red',edgecolors='red',marker='^',alpha=0.5)
    ax2[1].scatter(gEIseries-dshift,variance_full_num[:,iktrial,1],s=8.0,color='blue',edgecolors='blue',marker='^',alpha=0.5)
    ax2[0].scatter(gEIseries+dshift,variance_R1_num[:,iktrial,0],s=8.0,color='orange',edgecolors='orange',marker='s',alpha=0.5)
    ax2[1].scatter(gEIseries+dshift,variance_R1_num[:,iktrial,1],s=8.0,color='green',edgecolors='green',marker='s',alpha=0.5)
    ax2[0].scatter(gEIseries+dshift,variance_spr_num[:,iktrial,0],s=8.0,color='gray',edgecolors='gray',marker='s',alpha=0.5)
    ax2[1].scatter(gEIseries+dshift,variance_spr_num[:,iktrial,1],s=8.0,color='gray',edgecolors='gray',marker='s',alpha=0.5)


ax2[0].plot(gEIseries,np.mean(variance_full_theo[:,:,0],axis=1),color='red',linewidth=1.5,label=r'$\Delta_F^E$',alpha=0.5)
ax2[1].plot(gEIseries,np.mean(variance_full_theo[:,:,1],axis=1),color='blue',linewidth=1.5,label=r'$\Delta_F^I$',alpha=0.5)
ax2[0].plot(gEIseries,np.mean(variance_R1_theo[:,:,0],axis=1),color='orange',linewidth=1.5,label=r'$\Delta_{R1}^E$',alpha=0.5)
ax2[1].plot(gEIseries,np.mean(variance_R1_theo[:,:,1],axis=1),color='green',linewidth=1.5,label=r'$\Delta_{R1}^I$',alpha=0.5)

# for i in range(2):     
#         ax2[i].set_xlim(xlims)
#         ax2[i].set_ylim(ylims)
#         ax2[i].set_xticks(xticks)
#         ax2[i].set_yticks(yticks)
#         ax2[i].legend()


<ipython-input-5-b61ccc80ff8e>:19: RuntimeWarning: invalid value encountered in sqrt
  integrand = np.tanh(mu+np.sqrt(delta0)*gauss_points)
<ipython-input-5-b61ccc80ff8e>:8: RuntimeWarning: invalid value encountered in sqrt
  integrand = np.tanh(mu+np.sqrt(delta0)*gauss_points)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
<ipython-input-5-b61ccc80ff8e>:58: RuntimeWarning: divide by zero encountered in double_scalars
  consistency = [x[2]-(gee**2*NE/N+gei**2*NI/N)/(JE-JI)**2*(JE*PhimeanE-JI*PhimeanI)**2,x[0]-(JE*PhimeanE-JI*PhimeanI),x[3]-(gie**2*NE/N+gii**2*NI/N)/(JE-JI)**2*(JE*PhimeanE-JI*PhimeanI)**2,x[1]-(JE*PhimeanE-JI*PhimeanI)]
<ipython-input-5-b61ccc80ff8e>:58: RuntimeWarning: invalid value encountered in double_scalars
  consistency = [x[2]-(gee**2*NE/N+gei**2*NI/N)/(JE-JI)*

In [ ]:
fig,ax2 = plt.subplots(3,2,figsize=(6,4))
for iz in range(1,NE,80):
    ax2[0][0].plot(tt,xfpseries_Sparse[4,2,iz,:])
    ax2[1][0].plot(tt,xfpseries_Sparse[5,2,iz,:])
    ax2[2][0].plot(tt,xfpseries_Sparse[14,2,iz,:])

    ax2[0][1].plot(tt,xfpseries_R1[4,2,iz,:])
    ax2[1][1].plot(tt,xfpseries_R1[5,2,iz,:])
    ax2[2][1].plot(tt,xfpseries_R1[14,2,iz,:])
for i in range(2):
    ax2[i][0].set_xticks([])
    ax2[i][1].set_xticks([])
    ax2[i][1].set_yticks([])
yticks = np.linspace(-0.5,0.5,2)
ylims = [-0.6,0.6]
ax2[2][0].set_ylim(ylims)
ax2[2][0].set_yticks(yticks)
ax2[2][1].set_ylim(ylims)
ax2[2][1].set_yticks([])

yticks = np.linspace(-0.02,0.02,2)
ylims = [-0.025,0.025]
ax2[1][0].set_ylim(ylims)
ax2[1][0].set_yticks(yticks)
ax2[1][1].set_ylim(ylims)
ax2[1][1].set_yticks([])

yticks = np.linspace(0,1.0,2)
ylims = [-0.25,1.25]
ax2[0][0].set_ylim(ylims)
ax2[0][0].set_yticks(yticks)
ax2[0][1].set_ylim(ylims)
ax2[0][1].set_yticks([])

### Compare Critical Points

In [ ]:
'''
WITHOUT SYMMETRY, 
* increase g0 from zero to considerable
* constant JE,JI,a,b
'''
from functools import partial
# generate mean matrix
nrank=1
nn = 200
gnn = 4
Nt=np.array([gnn*nn,nn])
NE,NI=Nt[0],Nt[1]
N=NE+NI
## useful for determining g
Nparams=np.array([NE,NI])
Npercent = Nparams/N
nrank,ntrial,neta,nvec,nmaxFP=1,6,1,2,3
iktrial=0
''' Network Setting  '''
## heterogeneous random gain
ngEI=10
gEIseries = np.linspace(3.5,4,ngEI)
print(gEIseries)

Cs = 80
kE,kI = gnn,1
## heterogeneous degree of symmetry: amplitudes and signs
coeffetaEsub=np.array([1.0,1.0,1.0])#
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]
## E->total ppercentEsub[0]/2.0,ppercentEsub[1]/2.0, (I) 1/2.0

'''  Recording Variables. '''
## mean connectivity
RAmvecseries,LAmvecseries = np.zeros((ngEI,N)),np.zeros((ngEI,N))
BAmvseries = np.zeros((ngEI,N),dtype=complex)

## perturbed + symmetric eigenvectors
ZSparserandom = np.zeros((ngEI,ntrial,N,N))
## eigenvectors, decomposing Sparse Matrix
Rsprvecseries,Lsprvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RsprvecTseries,LsprvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Bsprvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
Radiusseries = np.zeros(ngEI)
## Gaussian iid random connections
gYbasexee=np.zeros((ngEI,N,N)) 
Riidvecseries,Liidvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RiidvecTseries,LiidvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Biidvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
## corresponding statistics \mu and \sigma \cov
armu,sigrcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2)) # 2 for E and I
almu,siglcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
siglr = np.zeros((ngEI,ntrial,2))

''' Nullclines parameters  '''
## compare three models' dynamcis
kappaintersect_Sparse = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_Full   = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_R1     = np.zeros((ngEI,ntrial,nmaxFP*2))

''' simulating parameters '''
tt = np.linspace(0,150,500)
dt = tt[2]-tt[1]
ntt = len(tt)
## compare three neuron population activities
xfpseries_Sparse = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_Full   = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_R1     = np.zeros((ngEI,ntrial,N,ntt))
''' All random matrices for each trials '''
''' ## Three \bar{J} cases '''
EPSP,a,b=0.09,0,0# JE PER EPSP 

''' Iterative Processing '''
for ig, gEI in enumerate (gEIseries):
    print('>>>>>>> simulating neuronal activity ... >>>>>>')
    ## mean matrix, reference eigenvalue and eigenvectors
    Am = np.zeros((N,N))
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    Am[:,:NE],Am[:,NE:] = JE0/NE,-JI0/NI
    # meigvecAm,neigvecAm,eigvAm = decompunperturbedEI(Am)
    # xAm,yAm=np.reshape(meigvecAm[:,0],(N,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(N,1))
    xAm,yAm = np.zeros((N,1)),np.zeros((N,1))
    eigvAm = np.zeros(N)
    eigvAm[0]=(JE0-JI0)
    xAm[:NE,0],xAm[NE:,0] = 1/np.sqrt(N),1/np.sqrt(N)
    yAm[:NE,0],yAm[NE:,0] = N*JE0/NE/np.sqrt(N)/eigvAm[0],-N*JI0/NI/np.sqrt(N)/eigvAm[0]
    RAmvecseries[ig,:],LAmvecseries[ig,:] = xAm[:,0],yAm[:,0]
    BAmvseries[ig,:] = eigvAm[:]

    for iktrial in range(ntrial):
        ## dilute/sparse matrix 
        Jdilute = np.zeros((N,N))
        JE,JI = EPSP,IPSP#JE0/Cs/kE,JI0/Cs/kI
        for inn in range(NE+NI):
            # for NE
            Einput = randbin(1,NE,1-kE*Cs/NE)
            Jdilute[inn,:NE] = Einput*JE
            # for NE
            Iinput = randbin(1,NI,1-kI*Cs/NI)
            Jdilute[inn,NE:] = Iinput*(-JI)

        # overall
        J = Jdilute.copy()
        ## subtract mean value
        # X = Jdilute.copy() - Am.copy() # random matrix, with zero mean 
        X = Jdilute.copy() # random matrix, with zero mean 
        X[:NE,:NE] = X[:NE,:NE] - np.mean(X[:NE,:NE].flatten())
        X[:NE,NE:] = X[:NE,NE:] - np.mean(X[:NE,NE:].flatten())
        X[NE:,:NE] = X[NE:,:NE] - np.mean(X[NE:,:NE].flatten())
        X[NE:,NE:] = X[NE:,NE:] - np.mean(X[NE:,NE:].flatten())
        ''' Original Sparse Network '''
        eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xAm,yAm,xAm,yAm,nparams = Nparams,sort=0)
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0,ynorm0 = xnorm0*np.sqrt(N),ynorm0*np.sqrt(N)
        Rsprvecseries[ig,iktrial,:,0],Lsprvecseries[ig,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
        Bsprvseries[ig,iktrial,:]=eigvJ.copy()

        axrmu,aylmu,sigxr,sigyl,sigcov = numerical_stats(xnorm0,ynorm0,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJ,nrank,2,ppercent=Npercent)
        armu[ig,iktrial,:],almu[ig,iktrial,:] = axrmu[:,0],aylmu[:,0]
        sigrcov[ig,iktrial,:],siglcov[ig,iktrial,:]= sigxr[:,0],sigyl[:,0]
        siglr[ig,iktrial,:] = sigcov[:,0,0]
        
        ## Rank One Approximation, Nullclines of \kappa
        xnorm0, ynorm0 = Rsprvecseries[ig,iktrial,:,0].copy(),Lsprvecseries[ig,iktrial,:,0]
        xnorm0, ynorm0 = np.reshape(xnorm0,(N,1)),np.reshape(ynorm0,(N,1))
        xmu,ymu = armu[ig,iktrial,:].copy(),almu[ig,iktrial,:].copy()
        xsig,ysig = sigrcov[ig,iktrial,:].copy(),siglcov[ig,iktrial,:].copy()
        yxcov = siglr[ig,iktrial,:].copy()

        ''' Sparse Connectivity -- Dynamics '''
        Jpt = J.copy()
        xinit = np.random.normal(0,1E-2,(1,N))
        xinit = np.squeeze(xinit)
        xtemporal = odesimulation(tt,xinit,Jpt,0)
        xfpseries_Sparse[ig,iktrial,:,:] = xtemporal.T.copy()
        ## 2 KAPPA_0
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_Sparse[ig,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        kappaintersect_Sparse[ig,iktrial,:3]=kappanum[:].copy()

        ''' Rank one model -- Dynamics '''
        ''' use perturbation theorem to calculate the perturbed vectors ''' 
        xnormt, ynormt = X.copy()@xAm.copy()/eigvAm[0], X.copy().T@yAm.copy()/eigvAm[0]#yAm.copy().T@X/eigvAm[0]*np.sqrt(N)
        xnormt, ynormt = xnormt+xAm.copy(), ynormt+yAm.copy()
        ## normalize 
        xnormt = xnormt.copy()/np.linalg.norm(xnormt.copy())
        ynormt = ynormt.copy()
        _,_,xnormt,ynormt=Normalization(xnormt.copy(),ynormt.copy(),xAm.copy(),yAm.copy(),nparams=Nparams,sort=0,nrank=1)
        RsprvecTseries[ig,iktrial,:,0],LsprvecTseries[ig,iktrial,:,0]=xnormt[:,0].copy(),ynormt[:,0].copy()
        xnormt,ynormt = xnormt*np.sqrt(N),ynormt*np.sqrt(N)
        # xnormt, ynormt = X.copy()@xAm.copy()/eigvAm[0]*np.sqrt(NE*2), X.copy().T@yAm.copy()/eigvAm[0]*np.sqrt(NE*2)#yAm.copy().T@X/eigvAm[0]*np.sqrt(N)
        # xnormt, ynormt = xnormt+xAm.copy()*np.sqrt(NE*2), ynormt+yAm.copy()*np.sqrt(NE*2)
        # ReigvecTseries[idxg,iktrial,:,0],LeigvecTseries[idxg,iktrial,:,0]=xnormt[:,0].copy(),ynormt[:,0].copy()
        r1Matt = np.real(xnormt@ynormt.T)
        r1Matt = r1Matt*np.real(eigvAm[0])/N
        ### START SIMULATING AND CALCULATE KAPPA AND xfpsereis_R1
        ## use the same initial values
        xtemporal = odesimulation(tt,xinit,r1Matt,0)
        xfpseries_R1[ig,iktrial,:,:] = xtemporal.T.copy()
        ''' kappa dynamics    '''
        ## 2 populations 
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_R1[ig,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            # do not change
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        kappaintersect_R1[ig,iktrial,:3]=kappanum[:].copy()

        ''' Full Gaussian iid model -- Dynamics '''
        ## variance
        gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs*N/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs*N/NI
        print("ratio gE/gI ",np.sqrt(gE2/gI2),', ',1/gEI)
        alphaNEI = Npercent.copy()
        gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
        eigvgm,eigvecgm=la.eig(gMmat) 
        r_g2=np.max(eigvgm)
        r_g = np.sqrt(r_g2)
        Radiusseries[ig] = r_g
        Xgiid =iidGaussian([0,1.0/np.sqrt(N)],[N,N])
        Xgiid[:NE,:NE]*=np.sqrt(gE2)
        Xgiid[:NE,NE:]*=np.sqrt(gI2)
        Xgiid[NE:,:NE]*=np.sqrt(gE2)
        Xgiid[NE:,NE:]*=np.sqrt(gI2)

        Jgiid = Xgiid.copy()+Am.copy()

        ''' Original Sparse Network '''
        eigvJiid,leigveciid,reigveciid,xnorm0iid,ynorm0iid=decompNormalization(Jgiid,xAm,yAm,xAm,yAm,nparams = Nparams,sort=0)
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0iid,ynorm0iid = xnorm0iid*np.sqrt(N),ynorm0iid*np.sqrt(N)
        Riidvecseries[ig,iktrial,:,0],Liidvecseries[ig,iktrial,:,0]=xnorm0iid[:,0].copy(),ynorm0iid[:,0].copy()
        Biidvseries[ig,iktrial,:]=eigvJiid.copy()
        axrmuiid,aylmuiid,sigxriid,sigyliid,sigcoviid = numerical_stats(xnorm0iid,ynorm0iid,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJiid,nrank,2,ppercent=Npercent)

        xtemporal = odesimulation(tt,xinit,Jgiid,0)
        xfpseries_Full[ig,iktrial,:,:] = xtemporal.T.copy()
        ''' kappa dynamics    '''
        ## 2 populations 
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_Full[ig,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((aylmuiid[0,0]*eigvJiid[0])*(yAm[0,0]*eigvAm[0])>0):
            # do not change
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        kappaintersect_Full[ig,iktrial,:3]=kappanum[:].copy()

In [ ]:
def CrossRadiusLambda(x,EPSP,kparams,nparams,Npercent):
    kE,kI,Cs=kparams[0],kparams[1],kparams[2]
    
    NE,NI=nparams[0],nparams[1]
    N = NE+NI
    IPSP = EPSP*x
    ## Radius
    gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs*N/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs*N/NI
    alphaNEI = Npercent.copy()
    gMmat = np.zeros((2,2))
    gMmat[:,0],gMmat[:,1]=gE2*alphaNEI[0],gI2*alphaNEI[1]
    # gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
    eigvgm,eigvecgm=la.eig(gMmat) 
    r_g2=np.max(eigvgm)
    r_g = np.sqrt(r_g2)
    ## Lambda 
    lambda_Gm = np.abs((EPSP*kE*Cs)*(1-x*kI/kE))

    return (r_g-lambda_Gm)

def theo_radius(epsp,kparams,nparams,gei):
    cgei = gei.copy()
    rho = kparams[0]*kparams[2]/nparams[0]
    kE,kI = kparams[0]*kparams[2],kparams[1]*kparams[2]
    NE,NI = nparams[0],nparams[1]
    N = NE+NI
    Radius2 = EPSP**2*kI*(1-rho)*(cgei**2+4)

    return np.sqrt(Radius2)



In [ ]:
### plot the eigenvalue and radius
fig,ax=plt.subplots(figsize=(6,3))
ax.plot(gEIseries,Radiusseries,'r',linewidth=1.5)
ax.plot(gEIseries,-Radiusseries,'r',linewidth=1.5)
ax.plot(gEIseries,BAmvseries[:,0],'b',linewidth=1.5)

In [ ]:
Kparams = np.array([kE,kI,Cs])
CrossRgOut = fsolve(CrossRadiusLambda,[1],args=(EPSP,Kparams,Nparams,Npercent))
print('cross of r_g and lambda:',CrossRgOut)
print(CrossRadiusLambda(1.0,EPSP,Kparams,Nparams,Npercent))
Radius = theo_radius(EPSP,Kparams,Nparams,gEIseries)

### plot the eigenvalue and radius
fig,ax=plt.subplots(figsize=(6,3))
ax.plot(gEIseries,Radiusseries,'r',linewidth=1.5)
ax.plot(gEIseries,-Radiusseries,'r',linewidth=1.5)
ax.plot(gEIseries,BAmvseries[:,0],'b',linewidth=1.5)
ax.plot(gEIseries,Radius[:],'g',linewidth=1.5)
### c is around the solution given above
ncadjacent = 5
cadjacent = np.linspace(CrossRgOut-0.5,CrossRgOut+0.5,ncadjacent)
for idx, gEI in enumerate(cadjacent):
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs*N/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs*N/NI
    gE,gI = np.sqrt(gE2),np.sqrt(gI2)
    gmat = np.zeros(4)
    gmat[0],gmat[2]=gE,gE
    gmat[1],gmat[3]=gI,gI
    statsfull = fsolve(iidfull_mudelta_consistency,[1.1,1.1,1.1,1.1],args=(JE0,JI0,gmat,Nparams))
    print('statistics, mu',statsfull[0])   
    print('statistics, sigma',statsfull[2])   


In [ ]:
'''
with symmetry, subcircuit
eigvJ0
'''
xticks = np.linspace(-1.0,2.0,1)
xlims = [-1.5,1.5]
yticks = np.linspace(-1,1,2)
ylims = [-1.5,1.5]
theta = np.linspace(0, 2 * np.pi, 200)
idxc=0
cm='rbg'
print(ntrial)
for ig,gEI in enumerate (gEIseries):
    figtspt,axtspt=plt.subplots(figsize=(6,3))
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    ## variance
    gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs/NI
    gE2,gI2 = gE2*N,gI2*N
    alphaNEI = Npercent.copy()
    gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
    eigvgm,eigvecgm=la.eig(gMmat) 
    r_g2=np.max(eigvgm)
    r_g = np.sqrt(r_g2)
    print(r_g)
    for idxtrial in range(ntrial):
        axtspt.scatter(np.real(Biidvseries[ig,idxtrial,:]),np.imag(Biidvseries[ig,idxtrial,:]),s=5,c='blue',alpha=0.25) # >>>>>>>>>>>>>>
    axtspt.scatter(np.real(Biidvseries[ig,:,0]),np.imag(Biidvseries[ig,:,0])-0.2*idxc,s=50,c='red',marker='^',alpha=0.5) # >>>>>>>>>>>>
    # axtspt.scatter(np.real(Bsprvseries[idxgavg,:,1]),np.imag(Bsprvseries[idxgavg,:,1])-0.2*idxc,s=20,c='darkblue',marker='^',alpha=0.5) # >>>>>>>>>>>>
    axtspt.set_aspect('equal')
    axtspt.plot(np.ones(2)*BAmvseries[ig,0],np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='grey')
    axtspt.spines['right'].set_color('none')
    axtspt.spines['top'].set_color('none')
    axtspt.xaxis.set_ticks_position('bottom')
    axtspt.spines['bottom'].set_position(('data', 0))
    # axtspt.set_xlim(xlims)
    # axtspt.set_ylim(ylims)
    # axtspt.set_xticks(xticks)
    # axtspt.set_yticks(yticks)
    axtspt.set_aspect('equal')
    eta=0
    longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
    xr = longaxis*np.cos(theta)
    yr = shortaxis*np.sin(theta)
    axtspt.plot(xr, yr, color="red", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>

In [ ]:
## self-consistency in \kappa, for rank one approximation 
kappa_theo_iid = np.zeros((ngEI,3))
for ig,gEI in enumerate (gEIseries):
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs*N/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs*N/NI
    gE,gI = np.sqrt(gE2),np.sqrt(gI2)
    gmat = np.array([gE,gI,gE,gI])
    init_k=1.5
    kappa_theo_iid[ig,0]= fsolve(iidperturbation,init_k,args=(JE0,JI0,gmat,Nparams))
    init_k=0.0
    kappa_theo_iid[ig,1]= fsolve(iidperturbation,init_k,args=(JE0,JI0,gmat,Nparams))
    init_k=-1.5
    kappa_theo_iid[ig,2]= fsolve(iidperturbation,init_k,args=(JE0,JI0,gmat,Nparams))
## plot kappa change with JE ##
fig,ax = plt.subplots(figsize=(6,2))
xticks = np.linspace(2,8,3)
xlims = [0,9]
yticks = np.linspace(-3,3,2)
ylims = [-4.,4.0]
ax.plot(gEIseries,kappa_theo_iid[:,0],color='black',linewidth=1.5)
ax.plot(gEIseries,kappa_theo_iid[:,1],color='gray',linewidth=1.5)
ax.plot(gEIseries,kappa_theo_iid[:,2],color='black',linewidth=1.5)
for iktrial in range(ntrial):
    ax.plot(gEIseries,kappaintersect_Full[:,iktrial,0],'^',color='blue',markersize=5)
    ax.plot(gEIseries,kappaintersect_R1[:,iktrial,0],'^',color='green',markersize=5)
    ax.plot(gEIseries,kappaintersect_Sparse[:,iktrial,0],'^',color='red',markersize=5)
    # ax.set_xlim(xlims)
    # ax.set_ylim(ylims)
    # ax.set_xticks(xticks)
    # ax.set_yticks(yticks)
    

In [ ]:
### test two expressions of variance ##
## numerically calculate the variance and mean, using xfpseries_Full and xfpseries_R1
iktrial=0
dshift = 0.02
variance_spr_num, mu_spr_num =np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
variance_full_num, mu_full_num =np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
variance_R1_num, mu_R1_num = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
variance_spr_theo, mu_spr_theo =np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
variance_full_theo, mu_full_theo =np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
variance_R1_theo, mu_R1_theo = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))


for ig,gEI in enumerate (gEIseries):
    for iktrial in range(ntrial):
        ## numerical results for Full Mat
        variance_full_num[ig,iktrial,0],variance_full_num[ig,iktrial,1]=np.std(xfpseries_Full[ig,iktrial,:NE,-1])**2,np.std(xfpseries_Full[ig,iktrial,NE:N,-1])**2
        mu_full_num[ig,iktrial,0],mu_full_num[ig,iktrial,1]=np.mean(xfpseries_Full[ig,iktrial,:NE,-1]),np.mean(xfpseries_Full[ig,iktrial,NE:N,-1])
        ## numerical results for Sparse Mat
        variance_spr_num[ig,iktrial,0],variance_spr_num[ig,iktrial,1]=np.std(xfpseries_Sparse[ig,iktrial,:NE,-1])**2,np.std(xfpseries_Sparse[ig,iktrial,NE:N,-1])**2
        mu_spr_num[ig,iktrial,0],mu_spr_num[ig,iktrial,1]=np.mean(xfpseries_Sparse[ig,iktrial,:NE,-1]),np.mean(xfpseries_Sparse[ig,iktrial,NE:N,-1])
        ## numerical results for Rank one Appriximation Mat
        variance_R1_num[ig,iktrial,0],variance_R1_num[ig,iktrial,1]=np.std(xfpseries_R1[ig,iktrial,:NE,-1])**2,np.std(xfpseries_R1[ig,iktrial,NE:N,-1])**2
        mu_R1_num[ig,iktrial,0],mu_R1_num[ig,iktrial,1]=np.mean(xfpseries_R1[ig,iktrial,:NE,-1]),np.mean(xfpseries_R1[ig,iktrial,NE:N,-1])

for ig,gEI in enumerate (gEIseries):
    ## keep the mean value to be the same -- JE0,JI0.
    ## the connectivity strength of sparse connections
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs*N/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs*N/NI
    gE,gI = np.sqrt(gE2),np.sqrt(gI2)
    gmat = np.array([gE,gI,gE,gI])
    # ## theoretical    
    deltainit,meaninit = np.mean(variance_full_num[ig,:,0]),np.mean(np.abs(mu_full_num[ig,:,0]))#15#np.mean(kappaintersect[idxg,:,0])
    ## JEI0 are effective mean connectivity weights, the nonzero connectivity strengths are JE0/(Cs/NE),JI0/(Cs/NI)
    statsfull = fsolve(iidfull_mudelta_consistency,[meaninit,meaninit,deltainit,deltainit],args=(JE0,JI0,gmat,Nparams))
    mu_full_theo[ig,:,0],variance_full_theo[ig,:,0]= statsfull[0]*np.ones(ntrial),statsfull[2]*np.ones(ntrial)
    mu_full_theo[ig,:,1],variance_full_theo[ig,:,1]= statsfull[1]*np.ones(ntrial),statsfull[3]*np.ones(ntrial)

    deltainit,meaninit = np.mean(variance_R1_num[ig,:,0]),kappa_theo_iid[ig,0]
    statsR1 = fsolve(iidR1_mudelta_consistency,[meaninit,meaninit,deltainit,deltainit],args=(JE0,JI0,gmat,Nparams))
    mu_R1_theo[ig,:,0],variance_R1_theo[ig,:,0]= statsR1[0]*np.ones(ntrial),statsR1[2]*np.ones(ntrial)
    mu_R1_theo[ig,:,1],variance_R1_theo[ig,:,1]= statsR1[1]*np.ones(ntrial),statsR1[3]*np.ones(ntrial)

    # ## theoretical    
    deltainit,meaninit = np.mean(variance_full_num[ig,:,0]),np.mean(np.abs(mu_full_num[ig,:,0]))#np.mean(kappaintersect[idxg,:,0])
    ## JEI0 are effective mean connectivity weights, the nonzero connectivity strengths are JE0/(Cs/NE),JI0/(Cs/NI)
    statsfull = fsolve(iidfull_mudelta_consistency,[-meaninit,-meaninit,deltainit,deltainit],args=(JE0,JI0,gmat,Nparams))
    mu_full_theo[ig,0,0],variance_full_theo[ig,0,0]= statsfull[0],statsfull[2]
    mu_full_theo[ig,0,1],variance_full_theo[ig,0,1]= statsfull[1],statsfull[3]

    deltainit,meaninit = np.mean(variance_R1_num[ig,:,0]),kappa_theo_iid[ig,2]
    statsR1 = fsolve(iidR1_mudelta_consistency,[meaninit,meaninit,deltainit,deltainit],args=(JE0,JI0,gmat,Nparams))
    mu_R1_theo[ig,0,0],variance_R1_theo[ig,0,0]= statsR1[0],statsR1[2]
    mu_R1_theo[ig,0,1],variance_R1_theo[ig,0,1]= statsR1[1],statsR1[3]

## Figures reflecting how mean and variance change with random gain of iid Gaussian matrix
fig,ax2 = plt.subplots(2,1,figsize=(6,4))
xticks = np.linspace(2,8,3)
xlims = [0,9]
yticks = np.linspace(-3,3,2)
ylims = [-4,4]
for iktrial in range(ntrial):
    # ax2[0].scatter(gEIseries-dshift,np.abs(mu_full_num[:,iktrial,0]),s=8.0,color='red',edgecolors='red',marker='^',alpha=0.5)
    # ax2[1].scatter(gEIseries-dshift,np.abs(mu_full_num[:,iktrial,1]),s=8.0,color='blue',edgecolors='blue',marker='^',alpha=0.5)
    # ax2[0].scatter(gEIseries+dshift,np.abs(mu_R1_num[:,iktrial,0]),s=8.0,color='orange',edgecolors='orange',marker='s',alpha=0.5)
    # ax2[1].scatter(gEIseries+dshift,np.abs(mu_R1_num[:,iktrial,1]),s=8.0,color='green',edgecolors='green',marker='s',alpha=0.5)
    # ax2[0].scatter(gEIseries+dshift,np.abs(mu_spr_num[:,iktrial,0]),s=8.0,color='gray',edgecolors='gray',marker='s',alpha=0.5)
    # ax2[1].scatter(gEIseries+dshift,np.abs(mu_spr_num[:,iktrial,1]),s=8.0,color='gray',edgecolors='gray',marker='s',alpha=0.5)
    ax2[0].scatter(gEIseries-dshift,(mu_full_num[:,iktrial,0]),s=8.0,color='red',edgecolors='red',marker='^',alpha=0.5)
    ax2[1].scatter(gEIseries-dshift,(mu_full_num[:,iktrial,1]),s=8.0,color='blue',edgecolors='blue',marker='^',alpha=0.5)
    ax2[0].scatter(gEIseries+dshift,(mu_R1_num[:,iktrial,0]),s=8.0,color='orange',edgecolors='orange',marker='s',alpha=0.5)
    ax2[1].scatter(gEIseries+dshift,(mu_R1_num[:,iktrial,1]),s=8.0,color='green',edgecolors='green',marker='s',alpha=0.5)
    ax2[0].scatter(gEIseries,(mu_spr_num[:,iktrial,0]),s=8.0,color='gray',edgecolors='gray',marker='s',alpha=0.5)
    ax2[1].scatter(gEIseries,(mu_spr_num[:,iktrial,1]),s=8.0,color='gray',edgecolors='gray',marker='s',alpha=0.5)

ax2[0].plot(gEIseries,(mu_full_theo[:,1,0]),color='red',linewidth=1.5,label=r'$\mu_F^E$',alpha=0.5)
ax2[1].plot(gEIseries,(mu_full_theo[:,1,1]),color='blue',linewidth=1.5,label=r'$\mu_F^I$',alpha=0.5)
ax2[0].plot(gEIseries,(mu_R1_theo[:,1,0]),color='orange',linewidth=1.5,label=r'$\mu_{R1}^E$',alpha=0.5)
ax2[1].plot(gEIseries,(mu_R1_theo[:,1,1]),color='green',linewidth=1.5,label=r'$\mu_{R1}^I$',alpha=0.5)

ax2[0].plot(gEIseries,(mu_full_theo[:,0,0]),color='red',linewidth=1.5,alpha=0.5)
ax2[1].plot(gEIseries,(mu_full_theo[:,0,1]),color='blue',linewidth=1.5,alpha=0.5)
ax2[0].plot(gEIseries,(mu_R1_theo[:,0,0]),color='orange',linewidth=1.5,alpha=0.5)
ax2[1].plot(gEIseries,(mu_R1_theo[:,0,1]),color='green',linewidth=1.5,alpha=0.5)


for i in range(2):     
        # ax2[i].set_xlim(xlims)
        # ax2[i].set_ylim(ylims)
        # ax2[i].set_xticks(xticks)
        # ax2[i].set_yticks(yticks)
        ax2[i].legend()


fig,ax2 = plt.subplots(2,1,figsize=(6,4))
xticks = np.linspace(2,8,3)
xlims = [0,9]
yticks = np.linspace(0,0.15,2)
ylims = [-0.01,0.18]
for iktrial in range(ntrial):
    ax2[0].scatter(gEIseries-dshift,variance_full_num[:,iktrial,0],s=8.0,color='red',edgecolors='red',marker='^',alpha=0.5)
    ax2[1].scatter(gEIseries-dshift,variance_full_num[:,iktrial,1],s=8.0,color='blue',edgecolors='blue',marker='^',alpha=0.5)
    ax2[0].scatter(gEIseries+dshift,variance_R1_num[:,iktrial,0],s=8.0,color='orange',edgecolors='orange',marker='s',alpha=0.5)
    ax2[1].scatter(gEIseries+dshift,variance_R1_num[:,iktrial,1],s=8.0,color='green',edgecolors='green',marker='s',alpha=0.5)
    ax2[0].scatter(gEIseries,variance_spr_num[:,iktrial,0],s=8.0,color='gray',edgecolors='gray',marker='s',alpha=0.5)
    ax2[1].scatter(gEIseries,variance_spr_num[:,iktrial,1],s=8.0,color='gray',edgecolors='gray',marker='s',alpha=0.5)


ax2[0].plot(gEIseries,np.mean(variance_full_theo[:,:,0],axis=1),color='red',linewidth=1.5,label=r'$\Delta_F^E$',alpha=0.5)
ax2[1].plot(gEIseries,np.mean(variance_full_theo[:,:,1],axis=1),color='blue',linewidth=1.5,label=r'$\Delta_F^I$',alpha=0.5)
ax2[0].plot(gEIseries,np.mean(variance_R1_theo[:,:,0],axis=1),color='orange',linewidth=1.5,label=r'$\Delta_{R1}^E$',alpha=0.5)
ax2[1].plot(gEIseries,np.mean(variance_R1_theo[:,:,1],axis=1),color='green',linewidth=1.5,label=r'$\Delta_{R1}^I$',alpha=0.5)

# for i in range(2):     
#         ax2[i].set_xlim(xlims)
#         ax2[i].set_ylim(ylims)
#         ax2[i].set_xticks(xticks)
#         ax2[i].set_yticks(yticks)
#         ax2[i].legend()


### Reconstruct RANK ONE matrix from iid approximation

In [ ]:
'''
WITHOUT SYMMETRY, 
* increase g0 from zero to considerable
* constant JE,JI,a,b
'''
from functools import partial
# generate mean matrix
nrank=1
nn = 200
gnn = 4
Nt=np.array([gnn*nn,nn])
NE,NI=Nt[0],Nt[1]
N=NE+NI
## useful for determining g
Nparams=np.array([NE,NI])
Npercent = Nparams/N
nrank,ntrial,neta,nvec,nmaxFP=1,3,1,2,3
iktrial=0
''' Network Setting  '''
## heterogeneous random gain
ngEI=5
gEIseries = np.linspace(3.0,4.5,ngEI)
print(gEIseries)

Cs = 90
kE,kI = gnn,1
## heterogeneous degree of symmetry: amplitudes and signs
coeffetaEsub=np.array([1.0,1.0,1.0])#
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]
## E->total ppercentEsub[0]/2.0,ppercentEsub[1]/2.0, (I) 1/2.0

'''  Recording Variables. '''
## mean connectivity
RAmvecseries,LAmvecseries = np.zeros((ngEI,N)),np.zeros((ngEI,N))
BAmvseries = np.zeros((ngEI,N),dtype=complex)

## perturbed + symmetric eigenvectors
ZSparserandom = np.zeros((ngEI,ntrial,N,N))
## eigenvectors, decomposing Sparse Matrix
Rsprvecseries,Lsprvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RsprvecTseries,LsprvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Bsprvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
Radiusseries = np.zeros(ngEI)
## Gaussian iid random connections
gYbasexee=np.zeros((ngEI,N,N)) 
Riidvecseries,Liidvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RiidvecTseries,LiidvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Biidvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
## corresponding statistics \mu and \sigma \cov
armu,sigrcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2)) # 2 for E and I
almu,siglcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
siglr = np.zeros((ngEI,ntrial,2))

''' Nullclines parameters  '''
## compare three models' dynamcis
kappaintersect_Sparse = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_Full   = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_R1     = np.zeros((ngEI,ntrial,nmaxFP*2))

''' simulating parameters '''
tt = np.linspace(0,150,500)
dt = tt[2]-tt[1]
ntt = len(tt)
## compare three neuron population activities
xfpseries_Sparse = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_Full   = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_R1     = np.zeros((ngEI,ntrial,N,ntt))
''' All random matrices for each trials '''
''' ## Three \bar{J} cases '''
EPSP,a,b=0.06,0,0# JE PER EPSP 

''' Iterative Processing '''
for ig, gEI in enumerate (gEIseries):
    print('>>>>>>> simulating neuronal activity ... >>>>>>')
    ## mean matrix, reference eigenvalue and eigenvectors
    Am = np.zeros((N,N))
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    Am[:,:NE],Am[:,NE:] = JE0/NE,-JI0/NI
    # meigvecAm,neigvecAm,eigvAm = decompunperturbedEI(Am)
    # xAm,yAm=np.reshape(meigvecAm[:,0],(N,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(N,1))
    xAm,yAm = np.zeros((N,1)),np.zeros((N,1))
    eigvAm = np.zeros(N)
    eigvAm[0]=(JE0-JI0)
    xAm[:NE,0],xAm[NE:,0] = 1/np.sqrt(N),1/np.sqrt(N)
    yAm[:NE,0],yAm[NE:,0] = N*JE0/NE/np.sqrt(N)/eigvAm[0],-N*JI0/NI/np.sqrt(N)/eigvAm[0]
    RAmvecseries[ig,:],LAmvecseries[ig,:] = xAm[:,0],yAm[:,0]
    BAmvseries[ig,:] = eigvAm[:]
    print('eigvAm:',eigvAm[0])

    for iktrial in range(ntrial):
        ## dilute/sparse matrix 
        Jdilute = np.zeros((N,N))
        JE,JI = EPSP,IPSP#JE0/Cs/kE,JI0/Cs/kI
        for inn in range(NE+NI):
            # for NE
            Einput = randbin(1,NE,1-kE*Cs/NE)
            Jdilute[inn,:NE] = Einput*JE
            # for NE
            Iinput = randbin(1,NI,1-kI*Cs/NI)
            Jdilute[inn,NE:] = Iinput*(-JI)

        # overall
        J = Jdilute.copy()
        ## subtract mean value
        # X = Jdilute.copy() - Am.copy() # random matrix, with zero mean 
        X = Jdilute.copy() # random matrix, with zero mean 
        X[:NE,:NE] = X[:NE,:NE] - np.mean(X[:NE,:NE].flatten())
        X[:NE,NE:] = X[:NE,NE:] - np.mean(X[:NE,NE:].flatten())
        X[NE:,:NE] = X[NE:,:NE] - np.mean(X[NE:,:NE].flatten())
        X[NE:,NE:] = X[NE:,NE:] - np.mean(X[NE:,NE:].flatten())
        ''' Original Sparse Network '''
        eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xAm,yAm,xAm,yAm,nparams = Nparams,sort=0)
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0,ynorm0 = xnorm0*np.sqrt(N),ynorm0*np.sqrt(N)
        Rsprvecseries[ig,iktrial,:,0],Lsprvecseries[ig,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
        Bsprvseries[ig,iktrial,:]=eigvJ.copy()

        axrmu,aylmu,sigxr,sigyl,sigcov = numerical_stats(xnorm0,ynorm0,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJ,nrank,2,ppercent=Npercent)
        armu[ig,iktrial,:],almu[ig,iktrial,:] = axrmu[:,0],aylmu[:,0]
        sigrcov[ig,iktrial,:],siglcov[ig,iktrial,:]= sigxr[:,0],sigyl[:,0]
        siglr[ig,iktrial,:] = sigcov[:,0,0]
        
        ## Rank One Approximation, Nullclines of \kappa
        xnorm0, ynorm0 = Rsprvecseries[ig,iktrial,:,0].copy(),Lsprvecseries[ig,iktrial,:,0]
        xnorm0, ynorm0 = np.reshape(xnorm0,(N,1)),np.reshape(ynorm0,(N,1))
        xmu,ymu = armu[ig,iktrial,:].copy(),almu[ig,iktrial,:].copy()
        xsig,ysig = sigrcov[ig,iktrial,:].copy(),siglcov[ig,iktrial,:].copy()
        yxcov = siglr[ig,iktrial,:].copy()

        ''' Sparse Connectivity -- Dynamics '''
        Jpt = J.copy()
        xinit = np.random.normal(0,1E-2,(1,N))
        xinit = np.squeeze(xinit)
        xtemporal = odesimulation(tt,xinit,Jpt,0)
        xfpseries_Sparse[ig,iktrial,:,:] = xtemporal.T.copy()
        ## 2 KAPPA_0
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_Sparse[ig,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        kappaintersect_Sparse[ig,iktrial,:3]=kappanum[:].copy()

        ''' Full Gaussian iid model -- Dynamics '''
        ## variance
        gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs*N/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs*N/NI
        # print("ratio gE/gI ",np.sqrt(gE2/gI2),', ',1/gEI)
        alphaNEI = Npercent.copy()
        gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
        eigvgm,eigvecgm=la.eig(gMmat) 
        r_g2=np.max(eigvgm)
        r_g = np.sqrt(r_g2)
        Radiusseries[ig] = r_g
        print("RG:",r_g)
        Xgiid =iidGaussian([0,1.0/np.sqrt(N)],[N,N])
        Xgiid[:NE,:NE]*=np.sqrt(gE2)
        Xgiid[:NE,NE:]*=np.sqrt(gI2)
        Xgiid[NE:,:NE]*=np.sqrt(gE2)
        Xgiid[NE:,NE:]*=np.sqrt(gI2)

        Jgiid = Xgiid.copy()+Am.copy()

        ''' Original Sparse Network '''
        eigvJiid,leigveciid,reigveciid,xnorm0iid,ynorm0iid=decompNormalization(Jgiid,xAm,yAm,xAm,yAm,nparams = Nparams,sort=0)
        print('Full:',eigvJiid[0])
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0iid,ynorm0iid = xnorm0iid*np.sqrt(N),ynorm0iid*np.sqrt(N)
        Riidvecseries[ig,iktrial,:,0],Liidvecseries[ig,iktrial,:,0]=xnorm0iid[:,0].copy(),ynorm0iid[:,0].copy()
        Biidvseries[ig,iktrial,:]=eigvJiid.copy()
        axrmuiid,aylmuiid,sigxriid,sigyliid,sigcoviid = numerical_stats(xnorm0iid,ynorm0iid,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJiid,nrank,2,ppercent=Npercent)

        xtemporal = odesimulation(tt,xinit,Jgiid,0)
        xfpseries_Full[ig,iktrial,:,:] = xtemporal.T.copy()
        ''' kappa dynamics    '''
        ## 2 populations 
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_Full[ig,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((aylmuiid[0,0]*eigvJiid[0])*(yAm[0,0]*eigvAm[0])>0):
            # do not change
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        kappaintersect_Full[ig,iktrial,:3]=kappanum[:].copy()


        ''' Rank one model -- Dynamics '''
        ''' use perturbation theorem to calculate the perturbed vectors ''' 
        xnormt, ynormt = Xgiid.copy()@xAm.copy()/eigvAm[0], Xgiid.copy().T@yAm.copy()/eigvAm[0] # use Gaussian random connectivity
        xnormt, ynormt = xnormt+xAm.copy(), ynormt+yAm.copy()
        xnormt, ynormt = X.copy()@xAm.copy()/eigvAm[0], X.copy().T@yAm.copy()/eigvAm[0] # use sparse random connectivity
        xnormt, ynormt = xnormt+xAm.copy(), ynormt+yAm.copy()
        ## normalize 
        xnormt = xnormt.copy()/np.linalg.norm(xnormt.copy())
        ynormt = ynormt.copy()
        _,_,xnormt,ynormt=Normalization(xnormt.copy(),ynormt.copy(),xAm.copy(),yAm.copy(),nparams=Nparams,sort=0,nrank=1)
        RsprvecTseries[ig,iktrial,:,0],LsprvecTseries[ig,iktrial,:,0]=xnormt[:,0].copy(),ynormt[:,0].copy()
        xnormt,ynormt = xnormt*np.sqrt(N),ynormt*np.sqrt(N)
        # xnormt, ynormt = X.copy()@xAm.copy()/eigvAm[0]*np.sqrt(NE*2), X.copy().T@yAm.copy()/eigvAm[0]*np.sqrt(NE*2)#yAm.copy().T@X/eigvAm[0]*np.sqrt(N)
        # xnormt, ynormt = xnormt+xAm.copy()*np.sqrt(NE*2), ynormt+yAm.copy()*np.sqrt(NE*2)
        # ReigvecTseries[idxg,iktrial,:,0],LeigvecTseries[idxg,iktrial,:,0]=xnormt[:,0].copy(),ynormt[:,0].copy()
        r1Matt = np.real(xnormt@ynormt.T)
        r1Matt = r1Matt*np.real(eigvAm[0])/N        
        ### START SIMULATING AND CALCULATE KAPPA AND xfpsereis_R1
        ## use the same initial values
        xtemporal = odesimulation(tt,xinit,r1Matt,0)
        xfpseries_R1[ig,iktrial,:,:] = xtemporal.T.copy()
        ''' kappa dynamics    '''
        ## 2 populations 
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_R1[ig,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            # do not change
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        kappaintersect_R1[ig,iktrial,:3]=kappanum[:].copy()

## Compare $c_{rad}$ (for connectivity) and $c_{x}$ (for dynamics)

In [ ]:
nrank=1
nn = 200
gnn = 4
Nt=np.array([gnn*nn,nn])
NE,NI=Nt[0],Nt[1]
N=NE+NI
## useful for determining g
Nparams=np.array([NE,NI])
Npercent = Nparams/N
''' Network Setting  '''
## heterogeneous random gain
ngEI=20
gEIseries = np.linspace(3.0,4.5,ngEI)

Cs = 90
kE,kI = gnn,1
a = kI/NI # sparsity
nEPSP = 20
EPSPseries = np.linspace(2*1e-2,8*1e-1,nEPSP)
def criticcRad(x,EPSP,KE,KI,A,):
    result = EPSP**2*KI*(1-A)*(x**2+4)-EPSP**2*KE**2*(1-x/(KE/KI))**2
    return result
def criticcX(x,EPSP,KE,KI,A,):
    result = EPSP*KE*(1-x/4.0)-1.0
    return result

cinit = 1
KE,KI = kE*Cs,kI*Cs
a = KE/NE
CriticCrad,CriticCx = np.zeros(nEPSP),np.zeros(nEPSP)
for iepsp,EPSP in enumerate(EPSPseries):
    CriticCrad[iepsp] = fsolve(criticcRad,[cinit],args=(EPSP,KE,KI,a))
    CriticCx[iepsp] = fsolve(criticcX,[cinit],args=(EPSP,KE,KI,a))
fig,ax2 = plt.subplots(figsize=(6,4))
xticks = np.linspace(2,8,3)
xlims = [0,9]
yticks = np.linspace(0,0.15,2)
ylims = [-0.01,0.18]

ax2.plot(EPSPseries,CriticCrad,'r',linewidth =1.5,label=r'$c_{rad}$')
ax2.plot(EPSPseries,CriticCx,'orange',linewidth =1.5,label=r'$c_{x}$')

In [ ]:
nrank=1
nn = 200
gnn = 4
Nt=np.array([gnn*nn,nn])
NE,NI=Nt[0],Nt[1]
N=NE+NI
## useful for determining g
Nparams=np.array([NE,NI])
Npercent = Nparams/N
''' Network Setting  '''
## heterogeneous random gain
ngEI=20
gEIseries = np.linspace(3.0,4.5,ngEI)

kE,kI = gnn,1
a = kI/NI # sparsity
EPSP = 0.04
def criticcRad(x,EPSP,KE,KI,A,):
    result = EPSP**2*KI*(1-A)*(x**2+4)-EPSP**2*KE**2*(1-x/(KE/KI))**2
    return result
def criticcX(x,EPSP,KE,KI,A,):
    result = EPSP*KE*(1-x/4.0)-1.0
    return result

Cseries = np.arange(20,120,5)
nC = len(Cseries)
cinit = 1
CriticCrad,CriticCx = np.zeros(nC),np.zeros(nC)
for iC,Cs in enumerate(Cseries):
    KE,KI = kE*Cs,kI*Cs
    a = KE/NE
    CriticCrad[iC] = fsolve(criticcRad,[cinit],args=(EPSP,KE,KI,a))
    CriticCx[iC] = fsolve(criticcX,[cinit],args=(EPSP,KE,KI,a))
fig,ax2 = plt.subplots(figsize=(6,4))
xticks = np.linspace(2,8,3)
xlims = [0,9]
yticks = np.linspace(0,0.15,2)
ylims = [-0.01,0.18]

ax2.plot(Cseries,CriticCrad,'r',linewidth =1.5,label=r'$c_{rad}$')
ax2.plot(Cseries,CriticCx,'orange',linewidth =1.5,label=r'$c_{x}$')

In [ ]:
#%%
def blocklikeEtasymmetry(Jmat,nparams,EPSP,IPSP):
    J,JT = Jmat.copy(),Jmat.copy().T
    NE,NI = nparams[0],nparams[1]
    K = int(NE/NI)
    N = NE+NI
    ## ee
    AEE,AEET = np.squeeze(J[:NE,:NE])/EPSP,np.squeeze(JT[:NE,:NE])/EPSP
    etaEE = np.sum((AEE*AEET).flatten())/NE
    etaEE = etaEE/(np.sum((AEE*AEE).flatten())/NE)
    ## ei set
    etaEIset = np.zeros(K)
    for i in range(K):
        subJIE,subJEI = J[NE:,i*NI:(i+1)*NI]/EPSP,JT[NE:,i*NI:(i+1)*NI]/IPSP
        etaEIset[i]   = np.sum((subJIE*subJEI).flatten())/NI
        etaEIset[i]   = etaEIset[i]/np.sqrt(np.sum((subJIE*subJIE).flatten())/NI)
        etaEIset[i]   = etaEIset[i]/np.sqrt(np.sum((subJEI*subJEI).flatten())/NI)
    ## ii
    AII,AIIT = np.squeeze(J[NE:,NE:])/IPSP,np.squeeze(JT[NE:,NE:])/IPSP
    etaII = np.sum((AII*AIIT).flatten())/NI
    etaII = etaII/(np.sum((AII*AII).flatten())/NI)
    
    return etaEE,etaEIset,etaII
def blocklikeEtasymmetry_binary(Jmat,nparams):
    J,JT = Jmat.copy(),Jmat.copy().T
    NE,NI = nparams[0],nparams[1]
    K = int(NE/NI)
    N = NE+NI
    ## ee
    AEE,AEET = np.squeeze(J[:NE,:NE]),np.squeeze(JT[:NE,:NE])
    etaEE = np.sum((AEE*AEET).flatten())/NE
    etaEE = etaEE/(np.sum((AEE*AEE).flatten())/NE)
    ## ei set
    etaEIset = np.zeros(K)
    for i in range(K):
        subJIE,subJEI = J[NE:,i*NI:(i+1)*NI],JT[NE:,i*NI:(i+1)*NI]
        etaEIset[i]   = np.sum((subJIE*subJEI).flatten())/NI

        etaEIset[i]   = etaEIset[i]/(np.sum((subJIE*subJIE).flatten())/NI)
    ## ii
    AII,AIIT = np.squeeze(J[NE:,NE:]),np.squeeze(JT[NE:,NE:])
    etaII = np.sum((AII*AIIT).flatten())/NI
    etaII = etaII/(np.sum((AII*AII).flatten())/NI)
    
    return etaEE,etaEIset,etaII

nrank=1
nn = 200
gnn = 4
Nt=np.array([gnn*nn,nn])
NE,NI=Nt[0],Nt[1]
N=NE+NI
## useful for determining g
Nparams=np.array([NE,NI])
Npercent = Nparams/N
nrank,ntrial,neta,nvec,nmaxFP=1,3,1,2,3
iktrial=0
''' Network Setting  '''
## heterogeneous random gain
ngEI=15
gEIseries = np.linspace(3.0,4.5,ngEI)
gEI = 2.0
Cs = 150
kE,kI = gnn,1
EPSP=0.02
## mean matrix, reference eigenvalue and eigenvectors
IPSP = gEI*EPSP
JE0=EPSP*kE*Cs
JI0=IPSP*kI*Cs

## dilute/sparse matrix 
Jdilute = np.zeros((N,N))
JE,JI = EPSP,IPSP#JE0/Cs/kE,JI0/Cs/kI
for inn in range(NE+NI):
    # for NE
    Einput = randbin(1,NE,1-kE*Cs/NE)
    Jdilute[inn,:NE] = Einput*JE
    # for NE
    Iinput = randbin(1,NI,1-kI*Cs/NI)
    Jdilute[inn,NE:] = Iinput*(-JI)
# overall
J = Jdilute.copy()

etaEE,etaEIset,etaII = blocklikeEtasymmetry(J,Nparams,EPSP,IPSP)
print('blocklike eta:',etaEE,etaEIset,etaII)


# analogous to reciprocal connections and anti-symmetric connections

In [ ]:
'''
SYMMETRY, 
* increase g0 from zero to considerable
* constant JE,JI,a,b
'''
# generate mean matrix
nrank=1
nn = 200
gnn = 4
Nt=np.array([gnn*nn,nn])
NE,NI=Nt[0],Nt[1]
N=NE+NI
## useful for determining g
Nparams=np.array([NE,NI])
Npercent = Nparams/N
nrank,ntrial,neta,nvec,nmaxFP=1,5,1,2,3
iktrial=0
''' Network Setting  '''
## heterogeneous random gain
ngEI=11
gEIseries = np.linspace(3.0,3.5,ngEI)
print(gEIseries)

Cs = 160
kE,kI = gnn,1
## heterogeneous degree of symmetry: amplitudes and signs
coeffetaEsub=np.array([1.0,1.0,1.0])#
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]
## E->total ppercentEsub[0]/2.0,ppercentEsub[1]/2.0, (I) 1/2.0

'''  Recording Variables. '''
## mean connectivity
RAmvecseries,LAmvecseries = np.zeros((ngEI,N)),np.zeros((ngEI,N))
BAmvseries = np.zeros((ngEI,N),dtype=complex)

## perturbed + symmetric eigenvectors
ZSparserandom = np.zeros((ngEI,ntrial,N,N))
## eigenvectors, decomposing Sparse Matrix
Rsprvecseries,Lsprvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RsprvecTseries,LsprvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Bsprvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
Radiusseries = np.zeros(ngEI)
## Gaussian iid random connections
gYbasexee=np.zeros((ngEI,N,N)) 
Riidvecseries,Liidvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RiidvecTseries,LiidvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Biidvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
## corresponding statistics \mu and \sigma \cov
armu,sigrcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2)) # 2 for E and I
almu,siglcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
siglr = np.zeros((ngEI,ntrial,2))
Sigmn_num = np.zeros((ngEI,ntrial,3))
Sigmn_theo= np.zeros((ngEI,ntrial,3))

## block-like eta
etaMat =np.zeros((ngEI,ntrial,2,2))
gRandom = np.zeros((ngEI,ntrial,2))

''' Nullclines parameters  '''
## compare three models' dynamcis
kappaintersect_Sparse = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_Full   = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_R1     = np.zeros((ngEI,ntrial,nmaxFP*2))

''' simulating parameters '''
tt = np.linspace(0,150,500)
dt = tt[2]-tt[1]
ntt = len(tt)
## compare three neuron population activities
xfpseries_Sparse = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_Full   = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_R1     = np.zeros((ngEI,ntrial,N,ntt))
''' All random matrices for each trials '''
''' ## Three \bar{J} cases '''
EPSP,a,b=0.01,0,0# JE PER EPSP 

''' Iterative Processing '''
for ig, gEI in enumerate (gEIseries):
    print('>>>>>>> simulating neuronal activity ... >>>>>>')
    ## mean matrix, reference eigenvalue and eigenvectors
    Am = np.zeros((N,N))
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    Am[:,:NE],Am[:,NE:] = JE0/NE,-JI0/NI
    # meigvecAm,neigvecAm,eigvAm = decompunperturbedEI(Am)
    # xAm,yAm=np.reshape(meigvecAm[:,0],(N,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(N,1))
    xAm,yAm = np.zeros((N,1)),np.zeros((N,1))
    eigvAm = np.zeros(N)
    eigvAm[0]=(JE0-JI0)
    xAm[:NE,0],xAm[NE:,0] = 1/np.sqrt(N),1/np.sqrt(N)
    yAm[:NE,0],yAm[NE:,0] = N*JE0/NE/np.sqrt(N)/eigvAm[0],-N*JI0/NI/np.sqrt(N)/eigvAm[0]
    RAmvecseries[ig,:],LAmvecseries[ig,:] = xAm[:,0],yAm[:,0]
    BAmvseries[ig,:] = eigvAm[:]
    print('eigvAm:',eigvAm[0])

    for iktrial in range(ntrial):
        ## dilute/sparse matrix 
        Jdilute = np.zeros((N,N))
        JE,JI = EPSP,IPSP#JE0/Cs/kE,JI0/Cs/kI
        for inn in range(NE+NI):
            # for NE
            Einput = randbin(1,NE,1-kE*Cs/NE)
            Jdilute[inn,:NE] = Einput*JE
            # for NE
            Iinput = randbin(1,NI,1-kI*Cs/NI)
            Jdilute[inn,NE:] = Iinput*(-JI)
        # overall
        J = Jdilute.copy()
        JT = Jdilute.copy().T
        ## calculate the block-like degree of symmetry
        # EE 
        etaEE,etaEIset,etaII = blocklikeEtasymmetry(J,Nparams,EPSP,IPSP)
        etaMat[ig,iktrial,0,0],etaMat[ig,iktrial,1,0],etaMat[ig,iktrial,1,1]=etaEE,np.mean(etaEIset),etaII

        ## subtract mean value
        # X = Jdilute.copy() - Am.copy() # random matrix, with zero mean 
        X = Jdilute.copy() # random matrix, with zero mean 
        X[:NE,:NE] = X[:NE,:NE] - np.mean(X[:NE,:NE].flatten())
        X[:NE,NE:] = X[:NE,NE:] - np.mean(X[:NE,NE:].flatten())
        X[NE:,:NE] = X[NE:,:NE] - np.mean(X[NE:,:NE].flatten())
        X[NE:,NE:] = X[NE:,NE:] - np.mean(X[NE:,NE:].flatten())
        ''' Original Sparse Network '''
        eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xAm,yAm,nparams = Nparams,sort=0)
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0,ynorm0 = xnorm0*np.sqrt(N),ynorm0*np.sqrt(N)
        Rsprvecseries[ig,iktrial,:,0],Lsprvecseries[ig,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
        Bsprvseries[ig,iktrial,:]=eigvJ.copy()

        axrmu,aylmu,sigxr,sigyl,sigcov = numerical_stats(xnorm0,ynorm0,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJ,nrank,2,ppercent=Npercent)
        armu[ig,iktrial,:],almu[ig,iktrial,:] = axrmu[:,0],aylmu[:,0]
        sigrcov[ig,iktrial,:],siglcov[ig,iktrial,:]= sigxr[:,0],sigyl[:,0]
        siglr[ig,iktrial,:] = sigcov[:,0,0]
        Sigmn_num[ig,iktrial,:2] = siglr[ig,iktrial,:]*eigvAm[0]
        Sigmn_num[ig,iktrial,2] = Sigmn_num[ig,iktrial,0]*Npercent[0]+Sigmn_num[ig,iktrial,1]*Npercent[1]
           
        
        ## Rank One Approximation, Nullclines of \kappa
        xnorm0, ynorm0 = Rsprvecseries[ig,iktrial,:,0].copy(),Lsprvecseries[ig,iktrial,:,0]
        xnorm0, ynorm0 = np.reshape(xnorm0,(N,1)),np.reshape(ynorm0,(N,1))
        xmu,ymu = armu[ig,iktrial,:].copy(),almu[ig,iktrial,:].copy()
        xsig,ysig = sigrcov[ig,iktrial,:].copy(),siglcov[ig,iktrial,:].copy()
        yxcov = siglr[ig,iktrial,:].copy()

        ''' Sparse Connectivity -- Dynamics '''
        Jpt = J.copy()
        xinit = np.random.normal(0,1E-2,(1,N))
        xinit = np.squeeze(xinit)
        xtemporal = odesimulation(tt,xinit,Jpt,0)
        xfpseries_Sparse[ig,iktrial,:,:] = xtemporal.T.copy()
        ## 2 KAPPA_0
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_Sparse[ig,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        kappaintersect_Sparse[ig,iktrial,:3]=kappanum[:].copy()

        ''' Full EI model -- Dynamics '''
        ## variance
        gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs*N/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs*N/NI
        # print("ratio gE/gI ",np.sqrt(gE2/gI2),', ',1/gEI)
        alphaNEI = Npercent.copy()
        gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
        eigvgm,eigvecgm=la.eig(gMmat) 
        r_g2=np.max(eigvgm)
        r_g = np.sqrt(r_g2)
        Radiusseries[ig] = r_g
        print("RG:",r_g)
        gRandom[ig,iktrial,0],gRandom[ig,iktrial,1]=np.sqrt(gE2),np.sqrt(gI2)
        ## theoret0ical Value
        #e
        forE = JE0*etaEE/2.0-gEI*JE0*np.mean(etaEIset)*kI/kE/2.0
        forI = JE0*np.mean(etaEIset)/2.0-gEI*JE0*etaII*kI/kE/2.0
        forE,forI = forE*gE2*NE/eigvAm[0]**2,forI*gI2*NI/eigvAm[0]**2
        Sigmn_theo[ig,iktrial,0],Sigmn_theo[ig,iktrial,1]=forE/NE, forI/NI
        Sigmn_theo[ig,iktrial,2] = Sigmn_theo[ig,iktrial,0]*Npercent[0]+Sigmn_theo[ig,iktrial,1]*Npercent[1]
        # Xgiid =iidGaussian([0,1.0/np.sqrt(N)],[N,N])
        # Xgiid[:NE,:NE]*=np.sqrt(gE2)
        # Xgiid[:NE,NE:]*=np.sqrt(gI2)
        # Xgiid[NE:,:NE]*=np.sqrt(gE2)
        # Xgiid[NE:,NE:]*=np.sqrt(gI2)

        # Jgiid = Xgiid.copy()+Am.copy()

        # ''' Original Sparse Network '''
        # eigvJiid,leigveciid,reigveciid,xnorm0iid,ynorm0iid=decompNormalization(Jgiid,xAm,yAm,nparams = Nparams,sort=0)
        # print('Full:',eigvJiid[0])
        # ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        # xnorm0iid,ynorm0iid = xnorm0iid*np.sqrt(N),ynorm0iid*np.sqrt(N)
        # Riidvecseries[ig,iktrial,:,0],Liidvecseries[ig,iktrial,:,0]=xnorm0iid[:,0].copy(),ynorm0iid[:,0].copy()
        # Biidvseries[ig,iktrial,:]=eigvJiid.copy()
        # axrmuiid,aylmuiid,sigxriid,sigyliid,sigcoviid = numerical_stats(xnorm0iid,ynorm0iid,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJiid,nrank,2,ppercent=Npercent)

        # xtemporal = odesimulation(tt,xinit,Jgiid,0)
        # xfpseries_Full[ig,iktrial,:,:] = xtemporal.T.copy()
        # ''' kappa dynamics    '''
        # ## 2 populations 
        # kappanum = np.zeros(3)
        # xact = np.squeeze(xfpseries_Full[ig,iktrial,:,-1])
        # ## use yAm -- unperturbed centre.
        # if((aylmuiid[0,0]*eigvJiid[0])*(yAm[0,0]*eigvAm[0])>0):
        #     # do not change
        #     kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        # else:
        #     kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        # kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        # kappaintersect_Full[ig,iktrial,:3]=kappanum[:].copy()


        # ''' Rank one model -- Dynamics '''
        # ''' use perturbation theorem to calculate the perturbed vectors ''' 
        # xnormt, ynormt = Xgiid.copy()@xAm.copy()/eigvAm[0], Xgiid.copy().T@yAm.copy()/eigvAm[0] # use Gaussian random connectivity
        # xnormt, ynormt = xnormt+xAm.copy(), ynormt+yAm.copy()
        # xnormt, ynormt = X.copy()@xAm.copy()/eigvAm[0], X.copy().T@yAm.copy()/eigvAm[0] # use sparse random connectivity
        # xnormt, ynormt = xnormt+xAm.copy(), ynormt+yAm.copy()
        # ## normalize 
        # xnormt = xnormt.copy()/np.linalg.norm(xnormt.copy())
        # ynormt = ynormt.copy()
        # _,_,xnormt,ynormt=Normalization(xnormt.copy(),ynormt.copy(),xAm.copy(),yAm.copy(),nparams=Nparams,sort=0,nrank=1)
        # RsprvecTseries[ig,iktrial,:,0],LsprvecTseries[ig,iktrial,:,0]=xnormt[:,0].copy(),ynormt[:,0].copy()
        # xnormt,ynormt = xnormt*np.sqrt(N),ynormt*np.sqrt(N)
        # # xnormt, ynormt = X.copy()@xAm.copy()/eigvAm[0]*np.sqrt(NE*2), X.copy().T@yAm.copy()/eigvAm[0]*np.sqrt(NE*2)#yAm.copy().T@X/eigvAm[0]*np.sqrt(N)
        # # xnormt, ynormt = xnormt+xAm.copy()*np.sqrt(NE*2), ynormt+yAm.copy()*np.sqrt(NE*2)
        # # ReigvecTseries[idxg,iktrial,:,0],LeigvecTseries[idxg,iktrial,:,0]=xnormt[:,0].copy(),ynormt[:,0].copy()
        # r1Matt = np.real(xnormt@ynormt.T)
        # r1Matt = r1Matt*np.real(eigvAm[0])/N        
        # ### START SIMULATING AND CALCULATE KAPPA AND xfpsereis_R1
        # ## use the same initial values
        # xtemporal = odesimulation(tt,xinit,r1Matt,0)
        # xfpseries_R1[ig,iktrial,:,:] = xtemporal.T.copy()
        # ''' kappa dynamics    '''
        # ## 2 populations 
        # kappanum = np.zeros(3)
        # xact = np.squeeze(xfpseries_R1[ig,iktrial,:,-1])
        # ## use yAm -- unperturbed centre.
        # if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
        #     # do not change
        #     kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        # else:
        #     kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        # kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        # kappaintersect_R1[ig,iktrial,:3]=kappanum[:].copy()

In [ ]:
'''
SYMMETRY, 
* increase g0 from zero to considerable
* constant JE,JI,a,b
'''
# generate mean matrix
print('sigmm')
nrank=1
nn = 200
gnn = 4

Nt=np.array([gnn*nn,nn])

NE,NI=Nt[0],Nt[1]
N=NE+NI
## useful for determining g
Nparams=np.array([NE,NI])
Npercent = Nparams/N
nrank,ntrial,neta,nvec,nmaxFP=1,10,1,2,3
''' Network Setting  '''
## heterogeneous random gain
ngEIH=10
ngEI = ngEIH*2
gEIseries1 = np.linspace(2.5,4.0,ngEIH)
gEIseries2 = np.linspace(5.,6.5,ngEIH)
gEIseries = np.zeros(ngEI)
gEIseries[:ngEIH] = gEIseries1
gEIseries[ngEIH:] = gEIseries2
print(gEIseries)

Cs = 90
kE,kI = gnn,1
## heterogeneous degree of symmetry: amplitudes and signs
coeffetaEsub=np.array([1.0,1.0,1.0])#
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]
## E->total ppercentEsub[0]/2.0,ppercentEsub[1]/2.0, (I) 1/2.0

'''  Recording Variables. '''
## mean connectivity
RAmvecseries,LAmvecseries = np.zeros((ngEI,N)),np.zeros((ngEI,N))
BAmvseries = np.zeros((ngEI,N),dtype=complex)

## perturbed + symmetric eigenvectors
ZSparserandom = np.zeros((ngEI,ntrial,N,N))
## eigenvectors, decomposing Sparse Matrix
Rsprvecseries,Lsprvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RsprvecTseries,LsprvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Bsprvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
Radiusseries = np.zeros(ngEI)
## Gaussian iid random connections
gYbasexee=np.zeros((ngEI,N,N)) 
Riidvecseries,Liidvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RiidvecTseries,LiidvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Biidvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
## corresponding statistics \mu and \sigma \cov
armu,sigrcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2)) # 2 for E and I
almu,siglcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
siglr = np.zeros((ngEI,ntrial,2))
Sigmn_num = np.zeros((ngEI,ntrial,3))
Sigmn_theo= np.zeros((ngEI,ntrial,3))


## block-like eta
etaMat =np.zeros((ngEI,ntrial,2,2))
gRandom = np.zeros((ngEI,ntrial,2))

''' Nullclines parameters  '''
## compare three models' dynamcis
kappaintersect_Sparse = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_Full   = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_R1     = np.zeros((ngEI,ntrial,nmaxFP*2))

''' simulating parameters '''
tt = np.linspace(0,150,500)
dt = tt[2]-tt[1]
ntt = len(tt)
## compare three neuron population activities
xfpseries_Sparse = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_Full   = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_R1     = np.zeros((ngEI,ntrial,N,ntt))
''' All random matrices for each trials '''
''' ## Three \bar{J} cases '''
EPSP,a,b=0.025,0,0# JE PER EPSP 

## basic preparation
for ig, gEI in enumerate (gEIseries):
    ## mean matrix, reference eigenvalue and eigenvectors
    Am = np.zeros((N,N))
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    Am[:,:NE],Am[:,NE:] = JE0/NE,-JI0/NI
    # meigvecAm,neigvecAm,eigvAm = decompunperturbedEI(Am)
    # xAm,yAm=np.reshape(meigvecAm[:,0],(N,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(N,1))
    xAm,yAm = np.zeros((N,1)),np.zeros((N,1))
    eigvAm = np.zeros(N)
    eigvAm[0]=(JE0-JI0)
    xAm[:NE,0],xAm[NE:,0] = 1/np.sqrt(N),1/np.sqrt(N)
    yAm[:NE,0],yAm[NE:,0] = N*JE0/NE/np.sqrt(N)/eigvAm[0],-N*JI0/NI/np.sqrt(N)/eigvAm[0]
    RAmvecseries[ig,:],LAmvecseries[ig,:] = xAm[:,0],yAm[:,0]
    BAmvseries[ig,:] = eigvAm[:]
    print('eigvAm:',eigvAm[0])

    # Jbinary = np.zeros((N,N))
    # for inn in range(NE+NI):
    #     # for NE
    #     Einput = randbin(1,NE,1-kE*Cs/NE)
    #     Jbinary[inn,:NE] = Einput
    #     # for NE
    #     Iinput = randbin(1,NI,1-kI*Cs/NI)
    #     Jbinary[inn,NE:] = -Iinput
    # etaEE,etaEIset,etaII = blocklikeEtasymmetry_binary(Jbinary,Nparams)
    # etaMat[ig,iktrial,0,0],etaMat[ig,iktrial,1,0],etaMat[ig,iktrial,1,1]=etaEE,np.mean(etaEIset),etaII
    # print("eta:",etaEE,np.mean(etaEIset),etaII)

''' Iterative Processing '''
for iktrial in range(ntrial):
    ## dilute/sparse matrix /attic
    for ig, gEI in enumerate (gEIseries):
        print('>>>>>>> simulating neuronal activity ... >>>>>>')
        ## mean matrix, reference eigenvalue and eigenvectors
        Am = np.zeros((N,N))
        IPSP = gEI*EPSP
        JE0=EPSP*kE*Cs
        JI0=IPSP*kI*Cs
        Am[:,:NE],Am[:,NE:] = JE0/NE,-JI0/NI
        # meigvecAm,neigvecAm,eigvAm = decompunperturbedEI(Am)
        # xAm,yAm=np.reshape(meigvecAm[:,0],(N,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(N,1))
        xAm,yAm = np.zeros((N,1)),np.zeros((N,1))
        eigvAm = np.zeros(N)
        eigvAm[0]=(JE0-JI0)
        xAm[:NE,0],xAm[NE:,0] = 1/np.sqrt(N),1/np.sqrt(N)
        yAm[:NE,0],yAm[NE:,0] = N*JE0/NE/np.sqrt(N)/eigvAm[0],-N*JI0/NI/np.sqrt(N)/eigvAm[0]
        

        ''' Full Gaussian iid model -- Dynamics '''
        ## variance
        gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs*N/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs*N/NI
        # print("ratio gE/gI ",np.sqrt(gE2/gI2),', ',1/gEI)
        alphaNEI = Npercent.copy()
        gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
        eigvgm,eigvecgm=la.eig(gMmat) 
        r_g2=np.max(eigvgm)
        r_g = np.sqrt(r_g2)
        Radiusseries[ig] = r_g
        print("RG:",r_g)
        Xgiid =iidGaussian([0,1.0/np.sqrt(N)],[N,N])
        Xgiid[:NE,:NE]*=np.sqrt(gE2)
        Xgiid[:NE,NE:]*=np.sqrt(gI2)
        Xgiid[NE:,:NE]*=np.sqrt(gE2)
        Xgiid[NE:,NE:]*=np.sqrt(gI2)
        Jgiid = Xgiid.copy()+Am.copy()
        
        gRandom[ig,iktrial,0],gRandom[ig,iktrial,1]=np.sqrt(gE2),np.sqrt(gI2)


        ''' Original Sparse Network '''
        eigvJiid,leigveciid,reigveciid,xnorm0iid,ynorm0iid=decompNormalization(Jgiid,xAm,yAm,xAm,yAm,nparams = Nparams,sort=0)
        print('Full:',eigvJiid[0])
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0iid,ynorm0iid = xnorm0iid*np.sqrt(N),ynorm0iid*np.sqrt(N)
        Riidvecseries[ig,iktrial,:,0],Liidvecseries[ig,iktrial,:,0]=xnorm0iid[:,0].copy(),ynorm0iid[:,0].copy()
        Biidvseries[ig,iktrial,:]=eigvJiid.copy()
        axrmuiid,aylmuiid,sigxriid,sigyliid,sigcoviid = numerical_stats(xnorm0iid,ynorm0iid,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJiid,nrank,2,ppercent=Npercent)
        

        
        Jbinary = np.zeros((N,N))
        for inn in range(NE+NI):
            # for NE
            Einput = randbin(1,NE,1-kE*Cs/NE)
            Jbinary[inn,:NE] = Einput
            # for NE
            Iinput = randbin(1,NI,1-kI*Cs/NI)
            Jbinary[inn,NE:] = -Iinput
        print('varEE:',np.std(np.squeeze(Jdilute[:,:NE]).flatten())**2,kE*Cs/NE*(1-kE*Cs/NE)*EPSP**2)
        etaEE,etaEIset,etaII = blocklikeEtasymmetry_binary(Jbinary,Nparams)
        etaMat[ig,iktrial,0,0],etaMat[ig,iktrial,1,0],etaMat[ig,iktrial,1,1]=etaEE,np.mean(etaEIset),etaII
        print("eta:",etaEE,np.mean(etaEIset),etaII)
        ## theoret0ical Value
        #e
        forE = JE0*etaEE/2.0-gEI*JE0*np.mean(etaEIset)*kI/kE/2.0
        forI = JE0*np.mean(etaEIset)/2.0-gEI*JE0*etaII*kI/kE/2.0
        forE,forI = forE*gE2*NE/eigvAm[0]**2,forI*gI2*NI/eigvAm[0]**2
        Sigmn_theo[ig,iktrial,0],Sigmn_theo[ig,iktrial,1]=forE/NE, forI/NI
        Sigmn_theo[ig,iktrial,2] = Sigmn_theo[ig,iktrial,0]*Npercent[0]+Sigmn_theo[ig,iktrial,1]*Npercent[1]

        ## from attic to sparse connectivity
        JE,JI = EPSP,IPSP
        Jdilute = Jbinary.copy()
        Jdilute[:,:NE] *= JE
        Jdilute[:,NE:] *= JI
        # etaEE,etaEIset,etaII = blocklikeEtasymmetry(Jdilute,Nparams,EPSP,IPSP)
        # etaMat[ig,iktrial,0,0],etaMat[ig,iktrial,1,0],etaMat[ig,iktrial,1,1]=etaEE,np.mean(etaEIset),etaII
        # print("eta:",etaEE,np.mean(etaEIset),etaII)

        # overall
        J = Jdilute.copy()
        JT = Jdilute.copy().T
        ## calculate the block-like degree of symmetry
        # EE 

        ## subtract mean value
        # X = Jdilute.copy() - Am.copy() # random matrix, with zero mean 
        X = Jdilute.copy() # random matrix, with zero mean 
        X[:NE,:NE] = X[:NE,:NE] - np.mean(X[:NE,:NE].flatten())
        X[:NE,NE:] = X[:NE,NE:] - np.mean(X[:NE,NE:].flatten())
        X[NE:,:NE] = X[NE:,:NE] - np.mean(X[NE:,:NE].flatten())
        X[NE:,NE:] = X[NE:,NE:] - np.mean(X[NE:,NE:].flatten())
        ''' Original Sparse Network '''
        eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xnorm0iid/np.sqrt(N),ynorm0iid/np.sqrt(N),xAm,yAm,nparams = Nparams,sort=1)
        # eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xAm,yAm,xAm,yAm,nparams = Nparams,sort=1)
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0,ynorm0 = xnorm0*np.sqrt(N),ynorm0*np.sqrt(N)
        Rsprvecseries[ig,iktrial,:,0],Lsprvecseries[ig,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
        Bsprvseries[ig,iktrial,:]=eigvJ.copy()

        axrmu,aylmu,sigxr,sigyl,sigcov = numerical_stats(xnorm0,ynorm0,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJ,nrank,2,ppercent=Npercent)
        armu[ig,iktrial,:],almu[ig,iktrial,:] = axrmu[:,0],aylmu[:,0]
        sigrcov[ig,iktrial,:],siglcov[ig,iktrial,:]= sigxr[:,0],sigyl[:,0]
        siglr[ig,iktrial,:] = sigcov[:,0,0]
        Sigmn_num[ig,iktrial,:2] = siglr[ig,iktrial,:]*eigvAm[0]
        Sigmn_num[ig,iktrial,2] = Sigmn_num[ig,iktrial,0]*Npercent[0]+Sigmn_num[ig,iktrial,1]*Npercent[1]
           
        
        ## Rank One Approximation, Nullclines of \kappa
        xnorm0, ynorm0 = Rsprvecseries[ig,iktrial,:,0].copy(),Lsprvecseries[ig,iktrial,:,0]
        xnorm0, ynorm0 = np.reshape(xnorm0,(N,1)),np.reshape(ynorm0,(N,1))
        xmu,ymu = armu[ig,iktrial,:].copy(),almu[ig,iktrial,:].copy()
        xsig,ysig = sigrcov[ig,iktrial,:].copy(),siglcov[ig,iktrial,:].copy()
        yxcov = siglr[ig,iktrial,:].copy()

        ''' Sparse Connectivity -- Dynamics '''
        Jpt = J.copy()
        xinit = np.random.normal(0,1E-2,(1,N))
        xinit = np.squeeze(xinit)
        xtemporal = odesimulation(tt,xinit,Jpt,0)
        xfpseries_Sparse[ig,iktrial,:,:] = xtemporal.T.copy()
        ## 2 KAPPA_0
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_Sparse[ig,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        kappaintersect_Sparse[ig,iktrial,:3]=kappanum[:].copy()

   

## Statistic properties of Connectivity

In [ ]:
'''
with symmetry, subcircuit
eigvJ0
'''
xticks = np.linspace(-1.0,2.0,1)
xlims = [-1.5,1.5]
yticks = np.linspace(-1,1,2)
ylims = [-1.5,1.5]
theta = np.linspace(0, 2 * np.pi, 200)
idxc=0
cm='rbg'
print(ntrial)
for ig,gEI in enumerate (gEIseries):
    figtspt,axtspt=plt.subplots(figsize=(6,3))
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    ## variance
    gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs/NI
    gE2,gI2 = gE2*N,gI2*N
    alphaNEI = Npercent.copy()
    gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
    eigvgm,eigvecgm=la.eig(gMmat) 
    r_g2=np.max(eigvgm)
    r_g = np.sqrt(r_g2)
    print(r_g)
    for idxtrial in range(ntrial):
        axtspt.scatter(np.real(Bsprvseries[ig,idxtrial,:]),np.imag(Bsprvseries[ig,idxtrial,:]),s=5,c='blue',alpha=0.25) # >>>>>>>>>>>>>>
    axtspt.scatter(np.real(Bsprvseries[ig,:,0]),np.imag(Bsprvseries[ig,:,0])-0.2*idxc,s=50,c='red',marker='^',alpha=0.5) # >>>>>>>>>>>>
    # axtspt.scatter(np.real(Bsprvseries[idxgavg,:,1]),np.imag(Bsprvseries[idxgavg,:,1])-0.2*idxc,s=20,c='darkblue',marker='^',alpha=0.5) # >>>>>>>>>>>>
    axtspt.set_aspect('equal')
    axtspt.plot(np.ones(2)*BAmvseries[ig,0],np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='grey')
    axtspt.spines['right'].set_color('none')
    axtspt.spines['top'].set_color('none')
    axtspt.xaxis.set_ticks_position('bottom')
    axtspt.spines['bottom'].set_position(('data', 0))
    # axtspt.set_xlim(xlims)
    # axtspt.set_ylim(ylims)
    # axtspt.set_xticks(xticks)
    # axtspt.set_yticks(yticks)
    axtspt.set_aspect('equal')
    eta=0
    longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
    xr = longaxis*np.cos(theta)
    yr = shortaxis*np.sin(theta)
    axtspt.plot(xr, yr, color="red", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>

In [ ]:
fig,ax2 = plt.subplots(figsize=(5,2))
for iktrial in range(ntrial):
    ax2.scatter(gEIseries,Sigmn_num[:,iktrial,2],c='r',s=5,alpha=0.5)
ax2.plot(gEIseries,np.mean(Sigmn_num[:,:,2],axis=1),'red',linewidth =1.5,label=r'numerical')
ax2.plot(gEIseries,np.mean(Sigmn_theo[:,:,2],axis=1),'orange',linewidth =1.5,label=r'theoretical')

xticks = np.linspace(3,6,2)
xlims = [2.3,6.8]
yticks = np.linspace(-1,1,3)
ylims = [-1.2,1.2]
ax2.set_xlim(xlims)
ax2.set_ylim(ylims)
ax2.set_xticks(xticks)
ax2.set_yticks(yticks)
ax2.axhline(c='grey', lw=1)

In [ ]:
#%%
'''
loading patterns
for ensemble-averaging results
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
xticks = np.linspace(-80,20,3)
xlims = [-90,40]
yticks = np.linspace(0,2,2)
ylims = [-2,3]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''

iktrial=0
ig=0
alphaval=0.10
edgv='black'
cm='br'
fig,ax=plt.subplots(figsize=(5,2))  
nrank=1
ntrial = np.shape(Rsprvecseries)[1]
NE=int(NE)

'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)

mEvec,mIvec,nEvec,nIvec=np.squeeze(Rsprvecseries[ig,:,:NE,0]),np.squeeze(Rsprvecseries[ig,:,NE:,0]),np.squeeze(Lsprvecseries[ig,:,:NE,0]),np.squeeze(Lsprvecseries[ig,:,NE:,0])
eigVforn = np.mean(Bsprvseries[ig,:,0])
nEvec,nIvec = nEvec*eigVforn,nIvec*eigVforn
mEvec,mIvec,nEvec,nIvec=mEvec.flatten(),mIvec.flatten(),nEvec.flatten(),nIvec.flatten()

scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NI*ntrial,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    print('shape:',np.shape(mEvec),np.shape(meanmE))
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

ax.scatter(nIvec,mIvec,s=5.0,c='b',alpha=0.2)#alphaval)#cm[1],alpha=alphaval)
ax.scatter(nEvec,mEvec,s=5.0,c='r',alpha=0.2)#alphaval)#cm[0],alpha=alphaval)

# ax.plot([meannE[0],meannE[0]+np.sqrt(N)*dirvecE[0,0]],[meanmE[0],meanmE[0]+np.sqrt(N)*dirvecE[1,0]],color='gray',linestyle='--',linewidth=1.5)
# ax.plot([meannE[0],meannE[0]+np.sqrt(N)*dirvecE[0,1]],[meanmE[0],meanmE[0]+np.sqrt(N)*dirvecE[1,1]],color='gray',linestyle='--',linewidth=1.5)
# ax.plot([meannI[0],meannI[0]+np.sqrt(N)*dirvecI[0,0]],[meanmI[0],meanmI[0]+np.sqrt(N)*dirvecI[1,0]],color=edgv,linestyle='--',linewidth=1.5)
# ax.plot([meannI[0],meannI[0]+np.sqrt(N)*dirvecI[0,1]],[meanmI[0],meanmI[0]+np.sqrt(N)*dirvecI[1,1]],color=edgv,linestyle='--',linewidth=1.5)
confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=edgv)
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=edgv)


ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')

In [ ]:
'''
SYMMETRY, 
* increase g0 from zero to considerable
* constant JE,JI,a,b
'''
# generate mean matrix
print('sigmm')
nrank=1
nn = 200
gnn = 4

Nt=np.array([gnn*nn,nn])

NE,NI=Nt[0],Nt[1]
N=NE+NI
## useful for determining g
Nparams=np.array([NE,NI])
Npercent = Nparams/N
nrank,ntrial,neta,nvec,nmaxFP=1,10,1,2,3
''' Network Setting  '''
## heterogeneous random gain
ngEIH=10
ngEI = ngEIH*2
gEIseries1 = np.linspace(2.5,4.0,ngEIH)
gEIseries2 = np.linspace(5.,6.5,ngEIH)
gEIseries = np.zeros(ngEI)
gEIseries[:ngEIH] = gEIseries1
gEIseries[ngEIH:] = gEIseries2
print(gEIseries)

Cs = 160
kE,kI = gnn,1
## heterogeneous degree of symmetry: amplitudes and signs
coeffetaEsub=np.array([1.0,1.0,1.0])#
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]
## E->total ppercentEsub[0]/2.0,ppercentEsub[1]/2.0, (I) 1/2.0

'''  Recording Variables. '''
## mean connectivity
RAmvecseries,LAmvecseries = np.zeros((ngEI,N)),np.zeros((ngEI,N))
BAmvseries = np.zeros((ngEI,N),dtype=complex)

## perturbed + symmetric eigenvectors
ZSparserandom = np.zeros((ngEI,ntrial,N,N))
## eigenvectors, decomposing Sparse Matrix
Rsprvecseries,Lsprvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RsprvecTseries,LsprvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Bsprvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
Radiusseries = np.zeros(ngEI)
## Gaussian iid random connections
gYbasexee=np.zeros((ngEI,N,N)) 
Riidvecseries,Liidvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RiidvecTseries,LiidvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Biidvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
## corresponding statistics \mu and \sigma \cov
armu,sigrcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2)) # 2 for E and I
almu,siglcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
siglr = np.zeros((ngEI,ntrial,2))
Sigmn_num = np.zeros((ngEI,ntrial,3))
Sigmn_theo= np.zeros((ngEI,ntrial,3))


## block-like eta
etaMat =np.zeros((ngEI,ntrial,2,2))
gRandom = np.zeros((ngEI,ntrial,2))

''' Nullclines parameters  '''
## compare three models' dynamcis
kappaintersect_Sparse = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_Full   = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_R1     = np.zeros((ngEI,ntrial,nmaxFP*2))

''' simulating parameters '''
tt = np.linspace(0,150,500)
dt = tt[2]-tt[1]
ntt = len(tt)
## compare three neuron population activities
xfpseries_Sparse = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_Full   = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_R1     = np.zeros((ngEI,ntrial,N,ntt))
''' All random matrices for each trials '''
''' ## Three \bar{J} cases '''
EPSP,a,b=0.025,0,0# JE PER EPSP 

## basic preparation
for ig, gEI in enumerate (gEIseries):
    ## mean matrix, reference eigenvalue and eigenvectors
    Am = np.zeros((N,N))
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    Am[:,:NE],Am[:,NE:] = JE0/NE,-JI0/NI
    # meigvecAm,neigvecAm,eigvAm = decompunperturbedEI(Am)
    # xAm,yAm=np.reshape(meigvecAm[:,0],(N,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(N,1))
    xAm,yAm = np.zeros((N,1)),np.zeros((N,1))
    eigvAm = np.zeros(N)
    eigvAm[0]=(JE0-JI0)
    xAm[:NE,0],xAm[NE:,0] = 1/np.sqrt(N),1/np.sqrt(N)
    yAm[:NE,0],yAm[NE:,0] = N*JE0/NE/np.sqrt(N)/eigvAm[0],-N*JI0/NI/np.sqrt(N)/eigvAm[0]
    RAmvecseries[ig,:],LAmvecseries[ig,:] = xAm[:,0],yAm[:,0]
    BAmvseries[ig,:] = eigvAm[:]
    print('eigvAm:',eigvAm[0])

    # Jbinary = np.zeros((N,N))
    # for inn in range(NE+NI):
    #     # for NE
    #     Einput = randbin(1,NE,1-kE*Cs/NE)
    #     Jbinary[inn,:NE] = Einput
    #     # for NE
    #     Iinput = randbin(1,NI,1-kI*Cs/NI)
    #     Jbinary[inn,NE:] = -Iinput
    # etaEE,etaEIset,etaII = blocklikeEtasymmetry_binary(Jbinary,Nparams)
    # etaMat[ig,iktrial,0,0],etaMat[ig,iktrial,1,0],etaMat[ig,iktrial,1,1]=etaEE,np.mean(etaEIset),etaII
    # print("eta:",etaEE,np.mean(etaEIset),etaII)

''' Iterative Processing '''
for iktrial in range(ntrial):
    ## dilute/sparse matrix /attic
    for ig, gEI in enumerate (gEIseries):
        print('>>>>>>> simulating neuronal activity ... >>>>>>')
        ## mean matrix, reference eigenvalue and eigenvectors
        Am = np.zeros((N,N))
        IPSP = gEI*EPSP
        JE0=EPSP*kE*Cs
        JI0=IPSP*kI*Cs
        Am[:,:NE],Am[:,NE:] = JE0/NE,-JI0/NI
        # meigvecAm,neigvecAm,eigvAm = decompunperturbedEI(Am)
        # xAm,yAm=np.reshape(meigvecAm[:,0],(N,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(N,1))
        xAm,yAm = np.zeros((N,1)),np.zeros((N,1))
        eigvAm = np.zeros(N)
        eigvAm[0]=(JE0-JI0)
        xAm[:NE,0],xAm[NE:,0] = 1/np.sqrt(N),1/np.sqrt(N)
        yAm[:NE,0],yAm[NE:,0] = N*JE0/NE/np.sqrt(N)/eigvAm[0],-N*JI0/NI/np.sqrt(N)/eigvAm[0]
        

        ''' Full Gaussian iid model -- Dynamics '''
        ## variance
        gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs*N/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs*N/NI
        # print("ratio gE/gI ",np.sqrt(gE2/gI2),', ',1/gEI)
        alphaNEI = Npercent.copy()
        gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
        eigvgm,eigvecgm=la.eig(gMmat) 
        r_g2=np.max(eigvgm)
        r_g = np.sqrt(r_g2)
        Radiusseries[ig] = r_g
        print("RG:",r_g)
        Xgiid =iidGaussian([0,1.0/np.sqrt(N)],[N,N])
        Xgiid[:NE,:NE]*=np.sqrt(gE2)
        Xgiid[:NE,NE:]*=np.sqrt(gI2)
        Xgiid[NE:,:NE]*=np.sqrt(gE2)
        Xgiid[NE:,NE:]*=np.sqrt(gI2)
        Jgiid = Xgiid.copy()+Am.copy()
        
        gRandom[ig,iktrial,0],gRandom[ig,iktrial,1]=np.sqrt(gE2),np.sqrt(gI2)


        ''' Original Sparse Network '''
        eigvJiid,leigveciid,reigveciid,xnorm0iid,ynorm0iid=decompNormalization(Jgiid,xAm,yAm,xAm,yAm,nparams = Nparams,sort=0)
        print('Full:',eigvJiid[0])
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0iid,ynorm0iid = xnorm0iid*np.sqrt(N),ynorm0iid*np.sqrt(N)
        Riidvecseries[ig,iktrial,:,0],Liidvecseries[ig,iktrial,:,0]=xnorm0iid[:,0].copy(),ynorm0iid[:,0].copy()
        Biidvseries[ig,iktrial,:]=eigvJiid.copy()
        axrmuiid,aylmuiid,sigxriid,sigyliid,sigcoviid = numerical_stats(xnorm0iid,ynorm0iid,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJiid,nrank,2,ppercent=Npercent)
        

        
        Jbinary = np.zeros((N,N))
        for inn in range(NE+NI):
            # for NE
            Einput = randbin(1,NE*NE,1-kE*Cs/NE)
            Jbinary[inn,:NE] = Einput
            # for NE
            Iinput = randbin(1,NI,1-kI*Cs/NI)
            Jbinary[inn,NE:] = -Iinput
        etaEE,etaEIset,etaII = blocklikeEtasymmetry_binary(Jbinary,Nparams)
        etaMat[ig,iktrial,0,0],etaMat[ig,iktrial,1,0],etaMat[ig,iktrial,1,1]=etaEE,np.mean(etaEIset),etaII
        print("eta:",etaEE,np.mean(etaEIset),etaII)
        ## theoret0ical Value
        #e
        forE = JE0*etaEE/2.0-gEI*JE0*np.mean(etaEIset)*kI/kE/2.0
        forI = JE0*np.mean(etaEIset)/2.0-gEI*JE0*etaII*kI/kE/2.0
        forE,forI = forE*gE2*NE/eigvAm[0]**2,forI*gI2*NI/eigvAm[0]**2
        Sigmn_theo[ig,iktrial,0],Sigmn_theo[ig,iktrial,1]=forE/NE, forI/NI
        Sigmn_theo[ig,iktrial,2] = Sigmn_theo[ig,iktrial,0]*Npercent[0]+Sigmn_theo[ig,iktrial,1]*Npercent[1]

        ## from attic to sparse connectivity
        JE,JI = EPSP,IPSP
        Jdilute = Jbinary.copy()
        Jdilute[:,:NE] *= JE
        Jdilute[:,NE:] *= JI
        # etaEE,etaEIset,etaII = blocklikeEtasymmetry(Jdilute,Nparams,EPSP,IPSP)
        # etaMat[ig,iktrial,0,0],etaMat[ig,iktrial,1,0],etaMat[ig,iktrial,1,1]=etaEE,np.mean(etaEIset),etaII
        # print("eta:",etaEE,np.mean(etaEIset),etaII)

        # overall
        J = Jdilute.copy()
        JT = Jdilute.copy().T
        ## calculate the block-like degree of symmetry
        # EE 

        ## subtract mean value
        # X = Jdilute.copy() - Am.copy() # random matrix, with zero mean 
        X = Jdilute.copy() # random matrix, with zero mean 
        X[:NE,:NE] = X[:NE,:NE] - np.mean(X[:NE,:NE].flatten())
        X[:NE,NE:] = X[:NE,NE:] - np.mean(X[:NE,NE:].flatten())
        X[NE:,:NE] = X[NE:,:NE] - np.mean(X[NE:,:NE].flatten())
        X[NE:,NE:] = X[NE:,NE:] - np.mean(X[NE:,NE:].flatten())
        ''' Original Sparse Network '''
        eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xnorm0iid/np.sqrt(N),ynorm0iid/np.sqrt(N),xAm,yAm,nparams = Nparams,sort=1)
        # eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xAm,yAm,xAm,yAm,nparams = Nparams,sort=1)
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0,ynorm0 = xnorm0*np.sqrt(N),ynorm0*np.sqrt(N)
        Rsprvecseries[ig,iktrial,:,0],Lsprvecseries[ig,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
        Bsprvseries[ig,iktrial,:]=eigvJ.copy()

        axrmu,aylmu,sigxr,sigyl,sigcov = numerical_stats(xnorm0,ynorm0,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJ,nrank,2,ppercent=Npercent)
        armu[ig,iktrial,:],almu[ig,iktrial,:] = axrmu[:,0],aylmu[:,0]
        sigrcov[ig,iktrial,:],siglcov[ig,iktrial,:]= sigxr[:,0],sigyl[:,0]
        siglr[ig,iktrial,:] = sigcov[:,0,0]
        Sigmn_num[ig,iktrial,:2] = siglr[ig,iktrial,:]*eigvAm[0]
        Sigmn_num[ig,iktrial,2] = Sigmn_num[ig,iktrial,0]*Npercent[0]+Sigmn_num[ig,iktrial,1]*Npercent[1]
           
        
        ## Rank One Approximation, Nullclines of \kappa
        xnorm0, ynorm0 = Rsprvecseries[ig,iktrial,:,0].copy(),Lsprvecseries[ig,iktrial,:,0]
        xnorm0, ynorm0 = np.reshape(xnorm0,(N,1)),np.reshape(ynorm0,(N,1))
        xmu,ymu = armu[ig,iktrial,:].copy(),almu[ig,iktrial,:].copy()
        xsig,ysig = sigrcov[ig,iktrial,:].copy(),siglcov[ig,iktrial,:].copy()
        yxcov = siglr[ig,iktrial,:].copy()

        ''' Sparse Connectivity -- Dynamics '''
        Jpt = J.copy()
        xinit = np.random.normal(0,1E-2,(1,N))
        xinit = np.squeeze(xinit)
        xtemporal = odesimulation(tt,xinit,Jpt,0)
        xfpseries_Sparse[ig,iktrial,:,:] = xtemporal.T.copy()
        ## 2 KAPPA_0
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_Sparse[ig,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        kappaintersect_Sparse[ig,iktrial,:3]=kappanum[:].copy()

   

In [ ]:
def generate_SymSpr_Mat(eta,prob,nsize):
    n1,n2=nsize[0],nsize[1]
    subProb = randbin(1,n1*n2,1-prob)
    # print("mean,var",np.mean(subProb),np.std(subProb)**2,prob*(1-prob))
    Jbsub= np.reshape(subProb,(n1,n2))
    JbsubT = Jbsub.copy().T

    idxTsym,idyTsym = np.where(JbsubT>0)
    nnonzero=len(idxTsym)
    # print('nonzero:',nnonzero)
    cutIdx = int(nnonzero*eta)
    # print("fnum:",cutIdx)
    Idxuse=random.sample(range(0,nnonzero),cutIdx)
    idxT,idyT = idxTsym[Idxuse],idyTsym[Idxuse]
    JbsubT_=np.zeros((n2,n1))
    JbsubT_[idxT,idyT]=1
    ## compensate
    idxTinv,idyTinv = np.where(JbsubT<1)
    zeros=len(idxTinv)
    cutIdxinv = (nnonzero-cutIdx)
    print("should / available snum:",cutIdxinv, zeros)
    if zeros<cutIdxinv:
        cutIdxinv = zeros#int(nnonzero*(1-eta))
    # print("snum:",cutIdxinv)
    # print("sumnum:",cutIdx+cutIdxinv,n1*n2*prob)
    Idxinvuse=random.sample(range(0,zeros),cutIdxinv)
    idxinvT,idyinvT = idxTinv[Idxinvuse],idyTinv[Idxinvuse]
    JbsubT_[idxinvT,idyinvT]=1 ## complementary 
    # print("IJ_mean,var",np.mean(np.squeeze(JbsubT_).flatten()),np.std(np.squeeze(JbsubT_).flatten())**2,prob*(1-prob))
    return Jbsub,JbsubT_ 

def generate_SymSpr_Mat_trial(Jiid,eta,prob,nsize):
    n1,n2=nsize[0],nsize[1]
    # subProb = randbin(1,n1*n2,1-prob)
    # # print("mean,var",np.mean(subProb),np.std(subProb)**2,prob*(1-prob))
    # Jbsub= np.reshape(subProb,(n1,n2))
    Jbsub = Jiid.copy()
    JbsubT = Jbsub.copy().T

    idxTsym,idyTsym = np.where(JbsubT>0)
    nnonzero=len(idxTsym)
    # print('nonzero:',nnonzero)
    cutIdx = int(nnonzero*eta)
    # print("fnum:",cutIdx)
    Idxuse=random.sample(range(0,nnonzero),cutIdx)
    idxT,idyT = idxTsym[Idxuse],idyTsym[Idxuse]
    JbsubT_=np.zeros((n2,n1))
    JbsubT_[idxT,idyT]=1
    ## compensate
    idxTinv,idyTinv = np.where(JbsubT<1)
    zeros=len(idxTinv)
    cutIdxinv = (nnonzero-cutIdx)
    print("should / available snum:",cutIdxinv, zeros)
    if zeros<cutIdxinv:
        cutIdxinv = zeros#int(nnonzero*(1-eta))
    # print("snum:",cutIdxinv)
    # print("sumnum:",cutIdx+cutIdxinv,n1*n2*prob)
    Idxinvuse=random.sample(range(0,zeros),cutIdxinv)
    idxinvT,idyinvT = idxTinv[Idxinvuse],idyTinv[Idxinvuse]
    JbsubT_[idxinvT,idyinvT]=1 ## complementary 
    # print("IJ_mean,var",np.mean(np.squeeze(JbsubT_).flatten()),np.std(np.squeeze(JbsubT_).flatten())**2,prob*(1-prob))
    return Jbsub,JbsubT_ 

def generate_SymSpr_MatEI(eta,prob,nsize):
    n1,n2=nsize[0],nsize[1]
    probie,probei=prob[0],prob[1]
    subProb = randbin(1,n1*n2,1-probie)
    Jbsub   = np.reshape(subProb,(n1,n2)) #subIE
    JbsubT  = Jbsub.copy().T #subEI

    idxTsym,idyTsym = np.where(JbsubT>0)
    nnonzero=len(idxTsym)
    cutIdx = int(nnonzero*eta)
    Idxuse=random.sample(range(0,nnonzero),cutIdx)
    idxT,idyT = idxTsym[Idxuse],idyTsym[Idxuse]
    JbsubT_=np.zeros((n2,n1))
    JbsubT_[idxT,idyT]=1
    ## compensate
    idxTinv,idyTinv = np.where(JbsubT<1)
    zeros=len(idxTinv)
    cutIdxinv = (nnonzero-cutIdx)
    if zeros<cutIdxinv:
        cutIdxinv = zeros#int(nnonzero*(1-eta))
    Idxinvuse=random.sample(range(0,zeros),cutIdxinv)
    idxinvT,idyinvT = idxTinv[Idxinvuse],idyTinv[Idxinvuse]
    JbsubT_[idxinvT,idyinvT]=1 ## complementary 
    return Jbsub,JbsubT_  

def generate_SymSpr_MatEI_trial(Jiid,eta,prob,nsize):
    n1,n2=nsize[0],nsize[1]
    probie,probei=prob[0],prob[1]
    # subProb = randbin(1,n1*n2,1-probie)
    # Jbsub   = np.reshape(subProb,(n1,n2)) #subIE
    Jbsub = Jiid.copy()
    JbsubT  = Jbsub.copy().T #subEI

    idxTsym,idyTsym = np.where(JbsubT>0)
    nnonzero=len(idxTsym)
    cutIdx = int(nnonzero*eta)
    Idxuse=random.sample(range(0,nnonzero),cutIdx)
    idxT,idyT = idxTsym[Idxuse],idyTsym[Idxuse]
    JbsubT_=np.zeros((n2,n1))
    JbsubT_[idxT,idyT]=1
    ## compensate
    idxTinv,idyTinv = np.where(JbsubT<1)
    zeros=len(idxTinv)
    cutIdxinv = (nnonzero-cutIdx)
    if zeros<cutIdxinv:
        cutIdxinv = zeros#int(nnonzero*(1-eta))
    Idxinvuse=random.sample(range(0,zeros),cutIdxinv)
    idxinvT,idyinvT = idxTinv[Idxinvuse],idyTinv[Idxinvuse]
    JbsubT_[idxinvT,idyinvT]=1 ## complementary 
    return Jbsub,JbsubT_  

In [ ]:
# generate mean matrix
import random
nrank=1
nn = 200
gnn = 4

Nt=np.array([gnn*nn,nn])

NE,NI=Nt[0],Nt[1]
N=NE+NI
## useful for determining g
Nparams=np.array([NE,NI])
Npercent = Nparams/N
nrank,ntrial,neta,nvec,nmaxFP=1,10,1,2,3

Cs = 180
kE,kI = gnn,1

## E->total ppercentEsub[0]/2.0,ppercentEsub[1]/2.0, (I) 1/2.0
''' All random matrices for each trials '''
''' ## Three \bar{J} cases '''
EPSP,a,b=0.025,0,0# JE PER EPSP 

Jbinary = np.zeros((N,N))
### GENERATE SYMMETRIC, SPARSE SUBMATRIX.
# for NE
etaEE  = 1.0
probEE = kE*Cs/NE
JbEE,JbEET_=generate_SymSpr_Mat(etaEE,kE*Cs/NE,np.array([NE,NE]))
JbEE_=JbEET_.copy().T
## combine two to get EE sub-matrix
JbEEt = np.zeros((NE,NE))
for i in range(NE):
    JbEEt[i,i]=JbEE[i,i]
idxEEd, idyEEd = np.where(JbEE>0)
iddown = np.where(idxEEd>idyEEd)
idxEEd,idyEEd=idxEEd[iddown],idyEEd[iddown]
JbEEt[idxEEd,idyEEd]=JbEE[idxEEd,idyEEd]
idxEEu, idyEEu = np.where(JbEET_>0)
idup = np.where(idxEEu<idyEEu)
idxEEu,idyEEu=idxEEu[idup],idyEEu[idup]
JbEEt[idxEEu,idyEEu]=JbEET_[idxEEu,idyEEu]

# for NI
etaII  = 1.0
probII = kE*Cs/NE
JbII,JbIIT_=generate_SymSpr_Mat(etaII,kI*Cs/NI,np.array([NI,NI]))
JbII_=JbIIT_.copy().T
## combine two to get EE sub-matrix
JbIIt = np.zeros((NI,NI))
for i in range(NI):
    JbIIt[i,i]=JbII[i,i]
idxIId, idyIId = np.where(JbII>0)
iddown = np.where(idxIId>idyIId)
idxIId,idyIId=idxIId[iddown],idyIId[iddown]
JbIIt[idxIId,idyIId]=JbII[idxIId,idyIId]
idxIIu, idyIIu = np.where(JbIIT_>0)
idup = np.where(idxIIu<idyIIu)##
idxIIu,idyIIu=idxIIu[idup],idyIIu[idup]
JbIIt[idxIIu,idyIIu]=JbIIT_[idxIIu,idyIIu]

# for EI
etaEI  = 1.0
JbIE,JbIET_=generate_SymSpr_MatEI(etaEI,np.array([kE*Cs/NE,kI*Cs/NI]),np.array([NE,NI]))
## combine two to get EE sub-matrix
JbIEt,JbEIt = np.zeros((NI,NE)),np.zeros((NE,NI))
JbIEt,JbEIt = JbIE.copy(),JbIET_.copy()


ETAII = np.sum(np.sum(JbIEt*JbEIt.T))/(NE*NI)
print(ETAII)
## TEST THE SPARSITY


## symmetric condition

In [ ]:
'''
SYMMETRY, 
* increase g0 from zero to considerable
* constant JE,JI,a,b
'''
# generate mean matrix
nrank=1
nn = 200
gnn = 4

Nt=np.array([gnn*nn,nn])
NE,NI=Nt[0],Nt[1]
N=NE+NI
## useful for determining g
Nparams=np.array([NE,NI])
Npercent = Nparams/N
nrank,ntrial,neta,nvec,nmaxFP=1,10,1,2,3
''' Network Setting  '''
# ## heterogeneous random gain
# ngEI=6
# gEIseries = np.linspace(2.5,3.5,ngEI)
## heterogeneous random gain
ngEIH=10
ngEI = ngEIH*2
gEIseries1 = np.linspace(2.5,4.0,ngEIH)
gEIseries2 = np.linspace(5.,6.5,ngEIH)
gEIseries = np.zeros(ngEI)
gEIseries[:ngEIH] = gEIseries1
gEIseries[ngEIH:] = gEIseries2
print(gEIseries)


Cs = 90
kE,kI = gnn,1
## heterogeneous degree of symmetry: amplitudes and signs
coeffetaEsub=np.array([1.0,1.0,1.0])#
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]
## E->total ppercentEsub[0]/2.0,ppercentEsub[1]/2.0, (I) 1/2.0

'''  Recording Variables. '''
## mean connectivity
RAmvecseries,LAmvecseries = np.zeros((ngEI,N)),np.zeros((ngEI,N))
BAmvseries = np.zeros((ngEI,N),dtype=complex)

## perturbed + symmetric eigenvectors
ZSparserandom = np.zeros((ngEI,ntrial,N,N))
## eigenvectors, decomposing Sparse Matrix
Rsprvecseries,Lsprvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RsprvecTseries,LsprvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Bsprvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
Radiusseries = np.zeros(ngEI)
## Gaussian iid random connections
gYbasexee=np.zeros((ngEI,N,N)) 
Riidvecseries,Liidvecseries=np.zeros((ngEI,ntrial,N,nvec*2),dtype=complex),np.zeros((ngEI,ntrial,N,nvec*2),dtype=complex)
RiidvecTseries,LiidvecTseries=np.zeros((ngEI,ntrial,N,nvec*2),dtype=complex),np.zeros((ngEI,ntrial,N,nvec*2),dtype=complex)
Biidvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
## corresponding statistics \mu and \sigma \cov
armu,sigrcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2)) # 2 for E and I
almu,siglcov = np.zeros((ngEI,ntrial,2),dtype=complex),np.zeros((ngEI,ntrial,2),dtype=complex)
siglr = np.zeros((ngEI,ntrial,2))
Sigmn_num = np.zeros((ngEI,ntrial,3))
Sigmn_theo= np.zeros((ngEI,ntrial,3))


## block-like eta
etaMat =np.zeros((ngEI,ntrial,2,2))
gRandom = np.zeros((ngEI,ntrial,2))

''' Nullclines parameters  '''
## compare three models' dynamcis
kappaintersect_Sparse = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_Full   = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_R1     = np.zeros((ngEI,ntrial,nmaxFP*2))

''' simulating parameters '''
tt = np.linspace(0,150,500)
dt = tt[2]-tt[1]
ntt = len(tt)
## compare three neuron population activities
xfpseries_Sparse = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_Full   = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_R1     = np.zeros((ngEI,ntrial,N,ntt))
''' All random matrices for each trials '''
''' ## Three \bar{J} cases '''
EPSP,a,b=0.025,0,0# JE PER EPSP 
eta=0.8

## basic preparation
for ig, gEI in enumerate (gEIseries):
    ## mean matrix, reference eigenvalue and eigenvectors
    Am = np.zeros((N,N))
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    Am[:,:NE],Am[:,NE:] = JE0/NE,-JI0/NI
    # meigvecAm,neigvecAm,eigvAm = decompunperturbedEI(Am)
    # xAm,yAm=np.reshape(meigvecAm[:,0],(N,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(N,1))
    xAm,yAm = np.zeros((N,1)),np.zeros((N,1))
    eigvAm = np.zeros(N)
    eigvAm[0]=(JE0-JI0)
    xAm[:NE,0],xAm[NE:,0] = 1/np.sqrt(N),1/np.sqrt(N)
    yAm[:NE,0],yAm[NE:,0] = N*JE0/NE/np.sqrt(N)/eigvAm[0],-N*JI0/NI/np.sqrt(N)/eigvAm[0]
    RAmvecseries[ig,:],LAmvecseries[ig,:] = xAm[:,0],yAm[:,0]
    BAmvseries[ig,:] = eigvAm[:]
    # print('eigvAm:',eigvAm[0])

''' Iterative Processing '''
for iktrial in range(ntrial):
    print('---- simulating neuronal activity #trial-',iktrial,'------')
    ## dilute/sparse matrix /attic
    for ig, gEI in enumerate (gEIseries):
        ## mean matrix, reference eigenvalue and eigenvectors
        Am = np.zeros((N,N))
        IPSP = gEI*EPSP
        JE0=EPSP*kE*Cs
        JI0=IPSP*kI*Cs
        Am[:,:NE],Am[:,NE:] = JE0/NE,-JI0/NI
        # meigvecAm,neigvecAm,eigvAm = decompunperturbedEI(Am)
        # xAm,yAm=np.reshape(meigvecAm[:,0],(N,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(N,1))
        xAm,yAm = np.zeros((N,1)),np.zeros((N,1))
        eigvAm = np.zeros(N)
        eigvAm[0]=(JE0-JI0)
        xAm[:NE,0],xAm[NE:,0] = 1/np.sqrt(N),1/np.sqrt(N)
        yAm[:NE,0],yAm[NE:,0] = N*JE0/NE/np.sqrt(N)/eigvAm[0],-N*JI0/NI/np.sqrt(N)/eigvAm[0]
        

        ''' Full Gaussian iid model -- Dynamics '''
        ## variance
        gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs*N/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs*N/NI
        # print("ratio gE/gI ",np.sqrt(gE2/gI2),', ',1/gEI)
        alphaNEI = Npercent.copy()
        gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
        eigvgm,eigvecgm=la.eig(gMmat) 
        r_g2=np.max(eigvgm)
        r_g = np.sqrt(r_g2)
        Radiusseries[ig] = r_g
        # print("RG:",r_g)
        Xgiid =iidGaussian([0,1.0/np.sqrt(N)],[N,N])
        Xgiid[:NE,:NE]*=np.sqrt(gE2)
        Xgiid[:NE,NE:]*=np.sqrt(gI2)
        Xgiid[NE:,:NE]*=np.sqrt(gE2)
        Xgiid[NE:,NE:]*=np.sqrt(gI2)
        Jgiid = Xgiid.copy()+Am.copy()
        
        gRandom[ig,iktrial,0],gRandom[ig,iktrial,1]=np.sqrt(gE2),np.sqrt(gI2)


        ''' Original Sparse Network '''
        eigvJiid,leigveciid,reigveciid,xnorm0iid,ynorm0iid=decompNormalization(Jgiid,xAm,yAm,xAm,yAm,nparams = Nparams,sort=0)
        # print('Full:',eigvJiid[0])
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0iid,ynorm0iid = xnorm0iid*np.sqrt(N),ynorm0iid*np.sqrt(N)
        Riidvecseries[ig,iktrial,:,0],Liidvecseries[ig,iktrial,:,0]=xnorm0iid[:,0].copy(),ynorm0iid[:,0].copy()
        Biidvseries[ig,iktrial,:]=eigvJiid.copy()
        axrmuiid,aylmuiid,sigxriid,sigyliid,sigcoviid = numerical_stats(xnorm0iid,ynorm0iid,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJiid,nrank,2,ppercent=Npercent)
        

        
        Jbinary = np.zeros((N,N))
        Jbinary = np.zeros((N,N))
        ### GENERATE SYMMETRIC, SPARSE SUBMATRIX.
        # for NE
        etaEE  = eta#1.0
        probEE = kE*Cs/NE
        JbEE,JbEET_=generate_SymSpr_Mat(etaEE,kE*Cs/NE,np.array([NE,NE]))
        JbEE_=JbEET_.copy().T
        ## combine two to get EE sub-matrix
        JbEEt = np.zeros((NE,NE))
        for i in range(NE):
            JbEEt[i,i]=JbEE[i,i]
        idxEEd, idyEEd = np.where(JbEE>0)
        iddown = np.where(idxEEd>idyEEd)
        idxEEd,idyEEd=idxEEd[iddown],idyEEd[iddown]
        JbEEt[idxEEd,idyEEd]=JbEE[idxEEd,idyEEd]
        idxEEu, idyEEu = np.where(JbEET_>0)
        idup = np.where(idxEEu<idyEEu)
        idxEEu,idyEEu=idxEEu[idup],idyEEu[idup]
        JbEEt[idxEEu,idyEEu]=JbEET_[idxEEu,idyEEu]
        print('symmetryEE:',np.sum(np.sum(JbEEt*JbEEt.T))/kE/Cs/NE,etaEE)
        # print('probEE:',np.sum(np.sum(JbEEt))/NE**2,kE*Cs/NE)

        # for NI
        etaII  = eta#1.0
        probII = kE*Cs/NE
        JbII,JbIIT_=generate_SymSpr_Mat(etaII,kI*Cs/NI,np.array([NI,NI]))
        JbII_=JbIIT_.copy().T
        ## combine two to get EE sub-matrix
        JbIIt = np.zeros((NI,NI))
        for i in range(NI):
            JbIIt[i,i]=JbII[i,i]
        idxIId, idyIId = np.where(JbII>0)
        iddown = np.where(idxIId>idyIId)
        idxIId,idyIId=idxIId[iddown],idyIId[iddown]
        JbIIt[idxIId,idyIId]=JbII[idxIId,idyIId]
        idxIIu, idyIIu = np.where(JbIIT_>0)
        idup = np.where(idxIIu<idyIIu)##
        idxIIu,idyIIu=idxIIu[idup],idyIIu[idup]
        JbIIt[idxIIu,idyIIu]=JbIIT_[idxIIu,idyIIu]

        # for EI
        etaEI  = eta#1.0
        JbIE,JbIET_=generate_SymSpr_MatEI(etaEI,np.array([kE*Cs/NE,kI*Cs/NI]),np.array([NI,NE]))
        ## combine two to get EE sub-matrix
        JbIEt,JbEIt = np.zeros((NI,NE)),np.zeros((NE,NI))
        JbIEt,JbEIt = JbIE.copy(),JbIET_.copy()

        Jbinary[:NE,:NE],Jbinary[NE:,NE:]=JbEEt,-JbIIt
        Jbinary[NE:,:NE],Jbinary[:NE,NE:]=JbIEt,-JbEIt

        ## from attic to sparse connectivity
        JE,JI = EPSP,IPSP
        Jdilute = Jbinary.copy()
        Jdilute[:,:NE] *= JE
        Jdilute[:,NE:] *= JI
        print('varEE:',np.std(np.squeeze(Jdilute[:,:NE]).flatten())**2,kE*Cs/NE*(1-kE*Cs/NE)*EPSP**2)
        ## lambda_change
        forE = JE0*eta*gE2-gEI*JE0*eta*kI/kE*np.sqrt(gE2*gI2)
        forI = JE0*eta*np.sqrt(gE2*gI2)-gEI*JE0*eta*kI/kE*gI2
        forE,forI = forE*NE/eigvAm[0]**2,forI*NI/eigvAm[0]**2
        Sigmn_theo[ig,iktrial,0],Sigmn_theo[ig,iktrial,1]=forE/NE, forI/NI
        Sigmn_theo[ig,iktrial,2] = Sigmn_theo[ig,iktrial,0]*Npercent[0]+Sigmn_theo[ig,iktrial,1]*Npercent[1]
        # etaEE,etaEIset,etaII = blocklikeEtasymmetry(Jdilute,Nparams,EPSP,IPSP)
        # etaMat[ig,iktrial,0,0],etaMat[ig,iktrial,1,0],etaMat[ig,iktrial,1,1]=etaEE,np.mean(etaEIset),etaII
        # print("eta:",etaEE,np.mean(etaEIset),etaII)

        # overall
        J = Jdilute.copy()
        JT = Jdilute.copy().T
        ## calculate the block-like degree of symmetry
        # EE 

        ## subtract mean value
        # X = Jdilute.copy() - Am.copy() # random matrix, with zero mean 
        X = Jdilute.copy() # random matrix, with zero mean 
        X[:NE,:NE] = X[:NE,:NE] - np.mean(X[:NE,:NE].flatten())
        X[:NE,NE:] = X[:NE,NE:] - np.mean(X[:NE,NE:].flatten())
        X[NE:,:NE] = X[NE:,:NE] - np.mean(X[NE:,:NE].flatten())
        X[NE:,NE:] = X[NE:,NE:] - np.mean(X[NE:,NE:].flatten())
        ''' Original Sparse Network '''
        eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xnorm0iid/np.sqrt(N),ynorm0iid/np.sqrt(N),xAm,yAm,nparams = Nparams,sort=1)
        # eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xAm,yAm,xAm,yAm,nparams = Nparams,sort=1)
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0,ynorm0 = xnorm0*np.sqrt(N),ynorm0*np.sqrt(N)
        Rsprvecseries[ig,iktrial,:,0],Lsprvecseries[ig,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
        Bsprvseries[ig,iktrial,:]=eigvJ.copy()

        axrmu,aylmu,sigxr,sigyl,sigcov = numerical_stats(xnorm0,ynorm0,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJ,nrank,2,ppercent=Npercent)
        armu[ig,iktrial,:],almu[ig,iktrial,:] = axrmu[:,0],aylmu[:,0]
        sigrcov[ig,iktrial,:],siglcov[ig,iktrial,:]= sigxr[:,0],sigyl[:,0]
        siglr[ig,iktrial,:] = sigcov[:,0,0]
        Sigmn_num[ig,iktrial,:2] = siglr[ig,iktrial,:]*eigvAm[0]
        Sigmn_num[ig,iktrial,2] = Sigmn_num[ig,iktrial,0]*Npercent[0]+Sigmn_num[ig,iktrial,1]*Npercent[1]
           
        
        ## Rank One Approximation, Nullclines of \kappa
        xnorm0, ynorm0 = Rsprvecseries[ig,iktrial,:,0].copy(),Lsprvecseries[ig,iktrial,:,0]
        xnorm0, ynorm0 = np.reshape(xnorm0,(N,1)),np.reshape(ynorm0,(N,1))
        xmu,ymu = armu[ig,iktrial,:].copy(),almu[ig,iktrial,:].copy()
        xsig,ysig = sigrcov[ig,iktrial,:].copy(),siglcov[ig,iktrial,:].copy()
        yxcov = siglr[ig,iktrial,:].copy()

        ''' Sparse Connectivity -- Dynamics '''
        Jpt = J.copy()
        xinit = np.random.normal(0,1E-2,(1,N))
        xinit = np.squeeze(xinit)
        xtemporal = odesimulation(tt,xinit,Jpt,0)
        xfpseries_Sparse[ig,iktrial,:,:] = xtemporal.T.copy()
        ## 2 KAPPA_0
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_Sparse[ig,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        kappaintersect_Sparse[ig,iktrial,:3]=kappanum[:].copy()

   

In [ ]:
fig,ax2 = plt.subplots(figsize=(5,2))
for iktrial in range(ntrial):
    ax2.scatter(gEIseries,Sigmn_num[:,iktrial,2],c='r',s=5,alpha=0.5)
ax2.plot(gEIseries,np.mean(Sigmn_num[:,:,2],axis=1),'red',linewidth =1.5,label=r'numerical')
ax2.plot(gEIseries,np.mean(Sigmn_theo[:,:,2],axis=1),'orange',linewidth =1.5,label=r'theoretical')

fig,ax2 = plt.subplots(figsize=(5,2))
for iktrial in range(ntrial):
    ax2.scatter(gEIseries,Sigmn_num[:,iktrial,0],c='r',s=5,alpha=0.5)
ax2.plot(gEIseries,np.mean(Sigmn_num[:,:,0],axis=1),'red',linewidth =1.5,label=r'numerical')
ax2.plot(gEIseries,np.mean(Sigmn_theo[:,:,0],axis=1),'orange',linewidth =1.5,label=r'theoretical')

fig,ax2 = plt.subplots(figsize=(5,2))
for iktrial in range(ntrial):
    ax2.scatter(gEIseries,Sigmn_num[:,iktrial,1],c='r',s=5,alpha=0.5)
ax2.plot(gEIseries,np.mean(Sigmn_num[:,:,1],axis=1),'red',linewidth =1.5,label=r'numerical')
ax2.plot(gEIseries,np.mean(Sigmn_theo[:,:,1],axis=1),'orange',linewidth =1.5,label=r'theoretical')

xticks = np.linspace(3,6,2)
xlims = [2.3,6.8]
yticks = np.linspace(-1,1,3)
ylims = [-1.2,1.2]
# ax2.set_xlim(xlims)
# ax2.set_ylim(ylims)
# ax2.set_xticks(xticks)
# ax2.set_yticks(yticks)
ax2.axhline(c='grey', lw=1)
print(Sigmn_theo,Sigmn_num)

In [ ]:
print(np.std(np.squeeze(Jdilute[:NE,:NE]).flatten())**2)
print(kE*Cs/NE*(1-kE*Cs/NE)*EPSP**2*2)
print(np.std(JbEE.flatten())**2)
print(np.std(JbEEt.flatten())**2)

print(np.std(JbII.flatten())**2)
print(np.std(JbIIt.flatten())**2)

print(np.mean(np.squeeze(Jdilute[:NE,:NE]).flatten()))
print(kE*Cs*EPSP/NE)

print(np.mean(JbEE.flatten()))
print(np.mean(JbEEt.flatten()))

print(np.mean(JbII.flatten()))
print(np.mean(JbIIt.flatten()))

In [ ]:
import math
import cmath
def ThreeSquare(x):
    if x.imag == 0:
        #实数
        m = x.real
        ans = -math.pow(-m,1/3) if m<0 else math.pow(m,1/3)
    else:
        #复数
        ans = x**(1/3)
    return ans

def RoundAns(x,num):
    if x.imag == 0:
        m = x.real
        ans = round(m,num)
    else:
        m = round(x.real,num)
        n = round(x.imag,num)
        ans = complex(m,n)
    return ans
    
def Cubic(args):
    a,b,c,d = args
    p = c/a-b**2/(3*a**2)
    q = d/a+2*b**3/(27*a**3)-b*c/(3*a**2)
    w = complex(-0.5,(3**0.5)/2)
    ww = complex(-0.5,-(3**0.5)/2)
    A = cmath.sqrt((q/2)**2+(p/3)**3)
    B = ThreeSquare(-q/2+A)
    C = ThreeSquare(-q/2-A)
    y1 = B+C
    y2 = w*B+ww*C
    y3 = ww*B+w*C
    D = b/(3*a)
    roots=[RoundAns(y1-D,6),RoundAns(y2-D,6),RoundAns(y3-D,6)]
    return roots

In [ ]:
EPSP**2*KE*eta*(1-p)**2-2*EPSP**2*KE*(1-eta)*(1-p)*p+EPSP**2*p**2*(NE-KE*2+KE*eta)
-EPSP*IPSP*KI*eta*(1-p)**2+2*EPSP*IPSP*KI*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NI-KI*2+KI*eta)

-EPSP*IPSP*KE*eta*(1-p)**2+2*EPSP*IPSP*KE*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NE-KE*2+KE*eta)
IPSP**2*KI*eta*(1-p)**2-2*IPSP**2*KI*(1-eta)*(1-p)*p+IPSP**2*p**2*(NI-KI*2+KI*eta)

In [ ]:
'''
with symmetry, subcircuit
eigvJ0
'''
xticks = np.linspace(-1.0,2.0,1)
xlims = [-1.5,1.5]
yticks = np.linspace(-1,1,2)
ylims = [-1.5,1.5]
theta = np.linspace(0, 2 * np.pi, 200)
idxc=0
cm='rbg'
print(ntrial)
for ig,gEI in enumerate (gEIseries):
    
    figtspt,axtspt=plt.subplots(figsize=(6,3))
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    KE,KI=kE*Cs,kI*Cs
    p=1-KE/NE
    forJEE=EPSP**2*KE*eta*(1-p)**2-2*EPSP**2*KE*(1-eta)*(1-p)*p+EPSP**2*p**2*(NE-KE*2+KE*eta)
    forJEI=-EPSP*IPSP*KI*eta*(1-p)**2+2*EPSP*IPSP*KI*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NI-KI*2+KI*eta)

    forJIE=-EPSP*IPSP*KE*eta*(1-p)**2+2*EPSP*IPSP*KE*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NE-KE*2+KE*eta)
    forJII=IPSP**2*KI*eta*(1-p)**2-2*IPSP**2*KI*(1-eta)*(1-p)*p+IPSP**2*p**2*(NI-KI*2+KI*eta)
    totallambda=(forJEE+forJEI)*N*JE0/NE*NE/N-(forJIE+forJII)*JI0
    B2=totallambda
    roots = Cubic([1,-BAmvseries[ig,0],0,-B2])
    # roots = np.ones(2)*totallambda/BAmvseries[ig,0]**2+BAmvseries[ig,0]
    ## variance
    gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs/NI
    gE2,gI2 = gE2*N,gI2*N
    alphaNEI = Npercent.copy()
    gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
    eigvgm,eigvecgm=la.eig(gMmat) 
    r_g2=np.max(eigvgm)
    r_g = np.sqrt(r_g2)
    print(r_g)
    for idxtrial in range(ntrial):
        axtspt.scatter(np.real(Bsprvseries[ig,idxtrial,:]),np.imag(Bsprvseries[ig,idxtrial,:]),s=5,c='blue',alpha=0.25) # >>>>>>>>>>>>>>
    axtspt.scatter(np.real(Bsprvseries[ig,:,0]),np.imag(Bsprvseries[ig,:,0])-0.2*idxc,s=50,c='red',marker='^',alpha=0.5) # >>>>>>>>>>>>
    # axtspt.scatter(np.real(Bsprvseries[idxgavg,:,1]),np.imag(Bsprvseries[idxgavg,:,1])-0.2*idxc,s=20,c='darkblue',marker='^',alpha=0.5) # >>>>>>>>>>>>
    axtspt.set_aspect('equal')
    axtspt.plot(np.ones(2)*BAmvseries[ig,0],np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='grey')
    axtspt.plot(np.ones(2)*(roots[0]),np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='green')
    axtspt.plot(np.ones(2)*np.mean(np.real(Bsprvseries[ig,:,0])),np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='blue')
    print('eigv:',BAmvseries[ig,0],roots)
    axtspt.spines['right'].set_color('none')
    axtspt.spines['top'].set_color('none')
    axtspt.xaxis.set_ticks_position('bottom')
    axtspt.spines['bottom'].set_position(('data', 0))
    # axtspt.set_xlim(xlims)
    # axtspt.set_ylim(ylims)
    # axtspt.set_xticks(xticks)
    # axtspt.set_yticks(yticks)
    axtspt.set_aspect('equal')
    longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
    xr = longaxis*np.cos(theta)
    yr = shortaxis*np.sin(theta)
    axtspt.plot(xr, yr, color="red", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>

In [ ]:
#%%
'''
loading patterns
for ensemble-averaging results
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
xticks = np.linspace(-80,20,3)
xlims = [-90,40]
yticks = np.linspace(0,2,2)
ylims = [-2,3]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''

iktrial=0
ig=5
alphaval=0.10
edgv='black'
cm='br'
fig,ax=plt.subplots(figsize=(5,2))  
nrank=1
ntrial = np.shape(Rsprvecseries)[1]
NE=int(NE)

'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)

mEvec,mIvec,nEvec,nIvec=np.squeeze(Rsprvecseries[ig,:,:NE,0]),np.squeeze(Rsprvecseries[ig,:,NE:,0]),np.squeeze(Lsprvecseries[ig,:,:NE,0]),np.squeeze(Lsprvecseries[ig,:,NE:,0])
eigVforn = np.mean(Bsprvseries[ig,:,0])
nEvec,nIvec = nEvec*eigVforn,nIvec*eigVforn
mEvec,mIvec,nEvec,nIvec=mEvec.flatten(),mIvec.flatten(),nEvec.flatten(),nIvec.flatten()

scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NI*ntrial,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    print('shape:',np.shape(mEvec),np.shape(meanmE))
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

ax.scatter(nIvec,mIvec,s=5.0,c='b',alpha=0.2)#alphaval)#cm[1],alpha=alphaval)
ax.scatter(nEvec,mEvec,s=5.0,c='r',alpha=0.2)#alphaval)#cm[0],alpha=alphaval)

# ax.plot([meannE[0],meannE[0]+np.sqrt(N)*dirvecE[0,0]],[meanmE[0],meanmE[0]+np.sqrt(N)*dirvecE[1,0]],color='gray',linestyle='--',linewidth=1.5)
# ax.plot([meannE[0],meannE[0]+np.sqrt(N)*dirvecE[0,1]],[meanmE[0],meanmE[0]+np.sqrt(N)*dirvecE[1,1]],color='gray',linestyle='--',linewidth=1.5)
# ax.plot([meannI[0],meannI[0]+np.sqrt(N)*dirvecI[0,0]],[meanmI[0],meanmI[0]+np.sqrt(N)*dirvecI[1,0]],color=edgv,linestyle='--',linewidth=1.5)
# ax.plot([meannI[0],meannI[0]+np.sqrt(N)*dirvecI[0,1]],[meanmI[0],meanmI[0]+np.sqrt(N)*dirvecI[1,1]],color=edgv,linestyle='--',linewidth=1.5)
confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=edgv)
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=edgv)


ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')

## Given a series of $\eta$ s, fixed c 1) excitatory dominant v.s. 2) inhibitory dominant 

In [ ]:
'''
SYMMETRY, 
* increase g0 from zero to considerable
* constant JE,JI,a,b
'''
# generate mean matrix
print('sigmm')
nrank=1
nn = 200
gnn = 4

Nt=np.array([gnn*nn,nn])

NE,NI=Nt[0],Nt[1]
N=NE+NI
## useful for determining g
Nparams=np.array([NE,NI])
Npercent = Nparams/N
nrank,ntrial,neta,nvec,nmaxFP=1,20,1,2,3
''' Network Setting  '''
## heterogeneous random gain
ngEIH=5
ngEI = ngEIH*2
gEIseries1 = np.linspace(2.5,4.0,ngEIH)
gEIseries2 = np.linspace(5.,6.5,ngEIH)
gEIseries = np.zeros(ngEI)
gEIseries[:ngEIH] = gEIseries1
gEIseries[ngEIH:] = gEIseries2
print(gEIseries)

Cs = 80
kE,kI = gnn,1
## heterogeneous degree of symmetry: amplitudes and signs
coeffetaEsub=np.array([1.0,1.0,1.0])#
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]
## E->total ppercentEsub[0]/2.0,ppercentEsub[1]/2.0, (I) 1/2.0

'''  Recording Variables. '''
## mean connectivity
RAmvecseries,LAmvecseries = np.zeros((ngEI,N)),np.zeros((ngEI,N))
BAmvseries = np.zeros((ngEI,N),dtype=complex)

## perturbed + symmetric eigenvectors
ZSparserandom = np.zeros((ngEI,ntrial,N,N))
## eigenvectors, decomposing Sparse Matrix
Rsprvecseries,Lsprvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RsprvecTseries,LsprvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Bsprvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
Radiusseries = np.zeros(ngEI)
## Gaussian iid random connections
gYbasexee=np.zeros((ngEI,N,N)) 
Riidvecseries,Liidvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RiidvecTseries,LiidvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Biidvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
## corresponding statistics \mu and \sigma \cov
armu,sigrcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2)) # 2 for E and I
almu,siglcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
siglr = np.zeros((ngEI,ntrial,2))
Sigmn_num = np.zeros((ngEI,ntrial,3))
Sigmn_theo= np.zeros((ngEI,ntrial,3))


## block-like eta
etaMat =np.zeros((ngEI,ntrial,2,2))
gRandom = np.zeros((ngEI,ntrial,2))

''' Nullclines parameters  '''
## compare three models' dynamcis
kappaintersect_Sparse = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_Full   = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_R1     = np.zeros((ngEI,ntrial,nmaxFP*2))

''' simulating parameters '''
tt = np.linspace(0,150,500)
dt = tt[2]-tt[1]
ntt = len(tt)
## compare three neuron population activities
xfpseries_Sparse = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_Full   = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_R1     = np.zeros((ngEI,ntrial,N,ntt))
''' All random matrices for each trials '''
''' ## Three \bar{J} cases '''
EPSP,a,b=0.025,0,0# JE PER EPSP 


gEI = 5.0
etaseries = np.linspace(0.0,1.0,ngEI)
## Am is fixed 
## mean matrix, reference eigenvalue and eigenvectors
Am = np.zeros((N,N))
IPSP = gEI*EPSP
JE0=EPSP*kE*Cs
JI0=IPSP*kI*Cs
Am[:,:NE],Am[:,NE:] = JE0/NE,-JI0/NI
# meigvecAm,neigvecAm,eigvAm = decompunperturbedEI(Am)
# xAm,yAm=np.reshape(meigvecAm[:,0],(N,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(N,1))
xAm,yAm = np.zeros((N,1)),np.zeros((N,1))
eigvAm = np.zeros(N)
eigvAm[0]=(JE0-JI0)
xAm[:NE,0],xAm[NE:,0] = 1/np.sqrt(N),1/np.sqrt(N)
yAm[:NE,0],yAm[NE:,0] = N*JE0/NE/np.sqrt(N)/eigvAm[0],-N*JI0/NI/np.sqrt(N)/eigvAm[0]
RAmvecseries[0,:],LAmvecseries[0,:] = xAm[:,0],yAm[:,0]
BAmvseries[0,:] = eigvAm[:]
print('eigvAm:',eigvAm[0])
## basic preparation

''' Full Gaussian iid model -- Dynamics '''
## variance
gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs*N/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs*N/NI
# print("ratio gE/gI ",np.sqrt(gE2/gI2),', ',1/gEI)
alphaNEI = Npercent.copy()
gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
eigvgm,eigvecgm=la.eig(gMmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
# print("RG:",r_g)
Xgiid =iidGaussian([0,1.0/np.sqrt(N)],[N,N])
Xgiid[:NE,:NE]*=np.sqrt(gE2)
Xgiid[:NE,NE:]*=np.sqrt(gI2)
Xgiid[NE:,:NE]*=np.sqrt(gE2)
Xgiid[NE:,NE:]*=np.sqrt(gI2)
Jgiid = Xgiid.copy()+Am.copy()
''' Original Sparse Network '''
eigvJiid,leigveciid,reigveciid,xnorm0iid,ynorm0iid=decompNormalization(Jgiid,xAm,yAm,xAm,yAm,nparams = Nparams,sort=0)
# print('Full:',eigvJiid[0])
''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
xnorm0iid,ynorm0iid = xnorm0iid*np.sqrt(N),ynorm0iid*np.sqrt(N)

''' Iterative Processing '''
for iktrial in range(ntrial):
    print('---- simulating neuronal activity #trial-',iktrial,'------')
    ## EE-trial
    n1,n2,prob=NE,NE,kE*Cs/NE
    subProb = randbin(1,n1*n2,1-prob)
    JEEsub= np.reshape(subProb,(n1,n2))

    ## EI-trial
    n1,n2,prob=NI,NE,kE*Cs/NE
    subProb = randbin(1,n1*n2,1-prob)
    JEIsub= np.reshape(subProb,(n1,n2))

    ## II-trial
    n1,n2,prob=NI,NI,kI*Cs/NI
    subProb = randbin(1,n1*n2,1-prob)
    JIIsub= np.reshape(subProb,(n1,n2))
    ## dilute/sparse matrix /attic
    for ig, eta in enumerate (etaseries):                
        Jbinary = np.zeros((N,N))
        ### GENERATE SYMMETRIC, SPARSE SUBMATRIX.
        # for NE
        etaEE  = eta#1.0
        probEE = kE*Cs/NE
        JbEE,JbEET_=generate_SymSpr_Mat_trial(JEEsub,etaEE,kE*Cs/NE,np.array([NE,NE]))
        JbEE_=JbEET_.copy().T
        ## combine two to get EE sub-matrix
        JbEEt = np.zeros((NE,NE))
        for i in range(NE):
            JbEEt[i,i]=JbEE[i,i]
        idxEEd, idyEEd = np.where(JbEE>0)
        iddown = np.where(idxEEd>idyEEd)
        idxEEd,idyEEd=idxEEd[iddown],idyEEd[iddown]
        JbEEt[idxEEd,idyEEd]=JbEE[idxEEd,idyEEd]
        idxEEu, idyEEu = np.where(JbEET_>0)
        idup = np.where(idxEEu<idyEEu)
        idxEEu,idyEEu=idxEEu[idup],idyEEu[idup]
        JbEEt[idxEEu,idyEEu]=JbEET_[idxEEu,idyEEu]
        print('symmetryEE:',np.sum(np.sum(JbEEt*JbEEt.T))/kE/Cs/NE,etaEE)
        # print('probEE:',np.sum(np.sum(JbEEt))/NE**2,kE*Cs/NE)

        # for NI
        etaII  = eta#1.0
        probII = kE*Cs/NE
        JbII,JbIIT_=generate_SymSpr_Mat_trial(JIIsub,etaII,kI*Cs/NI,np.array([NI,NI]))
        JbII_=JbIIT_.copy().T
        ## combine two to get EE sub-matrix
        JbIIt = np.zeros((NI,NI))
        for i in range(NI):
            JbIIt[i,i]=JbII[i,i]
        idxIId, idyIId = np.where(JbII>0)
        iddown = np.where(idxIId>idyIId)
        idxIId,idyIId=idxIId[iddown],idyIId[iddown]
        JbIIt[idxIId,idyIId]=JbII[idxIId,idyIId]
        idxIIu, idyIIu = np.where(JbIIT_>0)
        idup = np.where(idxIIu<idyIIu)##
        idxIIu,idyIIu=idxIIu[idup],idyIIu[idup]
        JbIIt[idxIIu,idyIIu]=JbIIT_[idxIIu,idyIIu]

        # for EI
        etaEI  = eta#1.0
        JbIE,JbIET_=generate_SymSpr_MatEI_trial(JEIsub,etaEI,np.array([kE*Cs/NE,kI*Cs/NI]),np.array([NI,NE]))
        ## combine two to get EE sub-matrix
        JbIEt,JbEIt = np.zeros((NI,NE)),np.zeros((NE,NI))
        JbIEt,JbEIt = JbIE.copy(),JbIET_.copy()

        Jbinary[:NE,:NE],Jbinary[NE:,NE:]=JbEEt,-JbIIt
        Jbinary[NE:,:NE],Jbinary[:NE,NE:]=JbIEt,-JbEIt

        ## from attic to sparse connectivity
        JE,JI = EPSP,IPSP
        Jdilute = Jbinary.copy()
        Jdilute[:,:NE] *= JE
        Jdilute[:,NE:] *= JI
        print('varEE:',np.std(np.squeeze(Jdilute[:,:NE]).flatten())**2,kE*Cs/NE*(1-kE*Cs/NE)*EPSP**2)
        ## lambda_change
        KE,KI=kE*Cs,kI*Cs
        p=KE/NE
        # p=1-KE/NE
        forJEE=EPSP**2*KE*eta*(1-p)**2-2*EPSP**2*KE*(1-eta)*(1-p)*p+EPSP**2*p**2*(NE-KE*2+KE*eta)## sigma^E
        forJEI=-EPSP*IPSP*KI*eta*(1-p)**2+2*EPSP*IPSP*KI*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NI-KI*2+KI*eta)## sigma^I

        # forJIE=-EPSP*IPSP*KE*eta*(1-p)**2+2*EPSP*IPSP*KE*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NE-KE*2+KE*eta)## ERROR!!!
        forJIE=-EPSP*IPSP*KE*eta*(1-p)**2+EPSP*IPSP*KE*(1-eta)*(1-p)*p+EPSP*IPSP*KE*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NE-KE*2+KE*eta)## signa^E
        forJII=IPSP**2*KI*eta*(1-p)**2-2*IPSP**2*KI*(1-eta)*(1-p)*p+IPSP**2*p**2*(NI-KI*2+KI*eta)
        sigmaE,sigmaI = forJEE*N*JE0/NE*NE+forJIE*(-N*JI0/NI*NI),forJEI*N*JE0/NE*NE+forJII*(-N*JI0/NI*NI)
        Sigmn_theo[ig,iktrial,0],Sigmn_theo[ig,iktrial,1]=sigmaE/NE/eigvAm[0]**2, sigmaI/NI/eigvAm[0]**2
        Sigmn_theo[ig,iktrial,2] = Sigmn_theo[ig,iktrial,0]*Npercent[0]+Sigmn_theo[ig,iktrial,1]*Npercent[1]
        # etaEE,etaEIset,etaII = blocklikeEtasymmetry(Jdilute,Nparams,EPSP,IPSP)
        # etaMat[ig,iktrial,0,0],etaMat[ig,iktrial,1,0],etaMat[ig,iktrial,1,1]=etaEE,np.mean(etaEIset),etaII
        # print("eta:",etaEE,np.mean(etaEIset),etaII)

        # overall
        J = Jdilute.copy()
        JT = Jdilute.copy().T
        ## calculate the block-like degree of symmetry
        # EE 

        ## subtract mean value
        # X = Jdilute.copy() - Am.copy() # random matrix, with zero mean 
        X = Jdilute.copy() # random matrix, with zero mean 
        X[:NE,:NE] = X[:NE,:NE] - np.mean(X[:NE,:NE].flatten())
        X[:NE,NE:] = X[:NE,NE:] - np.mean(X[:NE,NE:].flatten())
        X[NE:,:NE] = X[NE:,:NE] - np.mean(X[NE:,:NE].flatten())
        X[NE:,NE:] = X[NE:,NE:] - np.mean(X[NE:,NE:].flatten())
        ''' Original Sparse Network '''
        eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xnorm0iid/np.sqrt(N),ynorm0iid/np.sqrt(N),xAm,yAm,nparams = Nparams,sort=1)
        # eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xAm,yAm,xAm,yAm,nparams = Nparams,sort=1)
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0,ynorm0 = xnorm0*np.sqrt(N),ynorm0*np.sqrt(N)
        Rsprvecseries[ig,iktrial,:,0],Lsprvecseries[ig,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
        Bsprvseries[ig,iktrial,:]=eigvJ.copy()

        axrmu,aylmu,sigxr,sigyl,sigcov = numerical_stats(xnorm0,ynorm0,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJ,nrank,2,ppercent=Npercent)
        armu[ig,iktrial,:],almu[ig,iktrial,:] = axrmu[:,0],aylmu[:,0]
        sigrcov[ig,iktrial,:],siglcov[ig,iktrial,:]= sigxr[:,0],sigyl[:,0]
        siglr[ig,iktrial,:] = sigcov[:,0,0]
        Sigmn_num[ig,iktrial,:2] = siglr[ig,iktrial,:]*eigvAm[0]
        Sigmn_num[ig,iktrial,2] = Sigmn_num[ig,iktrial,0]*Npercent[0]+Sigmn_num[ig,iktrial,1]*Npercent[1]
           
        
        ## Rank One Approximation, Nullclines of \kappa
        xnorm0, ynorm0 = Rsprvecseries[ig,iktrial,:,0].copy(),Lsprvecseries[ig,iktrial,:,0]
        xnorm0, ynorm0 = np.reshape(xnorm0,(N,1)),np.reshape(ynorm0,(N,1))
        xmu,ymu = armu[ig,iktrial,:].copy(),almu[ig,iktrial,:].copy()
        xsig,ysig = sigrcov[ig,iktrial,:].copy(),siglcov[ig,iktrial,:].copy()
        yxcov = siglr[ig,iktrial,:].copy()

        ''' Sparse Connectivity -- Dynamics '''
        Jpt = J.copy()
        xinit = np.random.normal(0,1E-2,(1,N))
        xinit = np.squeeze(xinit)
        xtemporal = odesimulation(tt,xinit,Jpt,0)
        xfpseries_Sparse[ig,iktrial,:,:] = xtemporal.T.copy()
        ## 2 KAPPA_0
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_Sparse[ig,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        kappaintersect_Sparse[ig,iktrial,:3]=kappanum[:].copy()

   

In [ ]:
EPEP = 0.2
IPSP = 1.2

KE,KI=100,100
p=1-KE/NE
forJEE=EPSP**2*KE*eta*(1-p)**2-2*EPSP**2*KE*(1-eta)*(1-p)*p+EPSP**2*p**2*(NE-KE*2+KE*eta)## sigma^E
forJEI=-EPSP*IPSP*KI*eta*(1-p)**2+2*EPSP*IPSP*KI*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NI-KI*2+KI*eta)## sigma^I

# forJIE=-EPSP*IPSP*KE*eta*(1-p)**2+2*EPSP*IPSP*KE*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NE-KE*2+KE*eta)## ERROR!!!
forJIE=-EPSP*IPSP*KE*eta*(1-p)**2+EPSP*IPSP*KE*(1-eta)*(1-p)*p+EPSP*IPSP*KE*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NE-KE*2+KE*eta)## signa^E
forJII=IPSP**2*KI*eta*(1-p)**2-2*IPSP**2*KI*(1-eta)*(1-p)*p+IPSP**2*p**2*(NI-KI*2+KI*eta)
totallambda=(forJEE+forJEI)*N*JE0/NE*NE/N-(forJIE+forJII)*JI0

In [ ]:
'''
with symmetry, subcircuit
eigvJ0
'''
xticks = np.linspace(-1.0,2.0,1)
xlims = [-1.5,1.5]
yticks = np.linspace(-1,1,2)
ylims = [-1.5,1.5]
theta = np.linspace(0, 2 * np.pi, 200)
idxc=0
cm='rbg'
print(ntrial)
gEI =gEI
Bsprvseries_theo = np.zeros((ngEI,3),dtype=complex)
for ig,eta in enumerate (etaseries):
    
    figtspt,axtspt=plt.subplots(figsize=(6,3))
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    KE,KI=kE*Cs,kI*Cs
    p=KE/NE
    forJEE=EPSP**2*KE*eta*(1-p)**2-2*EPSP**2*KE*(1-eta)*(1-p)*p+EPSP**2*p**2*(NE-KE*2+KE*eta)## sigma^E
    forJEI=-EPSP*IPSP*KI*eta*(1-p)**2+2*EPSP*IPSP*KI*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NI-KI*2+KI*eta)## sigma^I

    # forJIE=-EPSP*IPSP*KE*eta*(1-p)**2+2*EPSP*IPSP*KE*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NE-KE*2+KE*eta)## ERROR!!!
    forJIE=-EPSP*IPSP*KE*eta*(1-p)**2+EPSP*IPSP*KE*(1-eta)*(1-p)*p+EPSP*IPSP*KE*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NE-KE*2+KE*eta)## signa^E
    forJII=IPSP**2*KI*eta*(1-p)**2-2*IPSP**2*KI*(1-eta)*(1-p)*p+IPSP**2*p**2*(NI-KI*2+KI*eta)
    totallambda=(forJEE+forJEI)*N*JE0/NE*NE/N-(forJIE+forJII)*JI0
    Janalytical = JE0*EPSP**2*p*(eta-p)*(1+KI/KE*gEI**2)*NE
    Janalytical -= gEI*JE0*EPSP**2*p*(eta-p)*(1+KI/KE*gEI**2)*NI
    print('total:',totallambda,Janalytical)
    B2=totallambda
    roots = Cubic([1,-BAmvseries[0,0],0,-B2])
    Bsprvseries_theo[ig,:] = roots[:3]
    # roots = np.ones(2)*totallambda/BAmvseries[ig,0]**2+BAmvseries[ig,0]
    ## variance
    gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs/NI
    gE2,gI2 = gE2*N,gI2*N
    alphaNEI = Npercent.copy()
    gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
    eigvgm,eigvecgm=la.eig(gMmat) 
    r_g2=np.max(eigvgm)
    r_g = np.sqrt(r_g2)
    print(r_g)
    for idxtrial in range(ntrial):
        axtspt.scatter(np.real(Bsprvseries[ig,idxtrial,:]),np.imag(Bsprvseries[ig,idxtrial,:]),s=5,c='blue',alpha=0.25) # >>>>>>>>>>>>>>
    axtspt.scatter(np.real(Bsprvseries[ig,:,0]),np.imag(Bsprvseries[ig,:,0])-0.2*idxc,s=50,c='red',marker='^',alpha=0.5) # >>>>>>>>>>>>
    # axtspt.scatter(np.real(Bsprvseries[idxgavg,:,1]),np.imag(Bsprvseries[idxgavg,:,1])-0.2*idxc,s=20,c='darkblue',marker='^',alpha=0.5) # >>>>>>>>>>>>
    axtspt.set_aspect('equal')
    axtspt.plot(np.ones(2)*BAmvseries[0,0],np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='grey')
    axtspt.plot(np.ones(2)*(roots[0]),np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='green')
    axtspt.plot(np.ones(2)*np.mean(np.real(Biidvseries[ig,:,0])),np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='blue')

    # axtspt.plot(np.ones(2)*np.mean(np.real(Bsprvseries[ig,:,0])),np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='blue')
    print('eigv:',BAmvseries[ig,0],roots)
    axtspt.spines['right'].set_color('none')
    axtspt.spines['top'].set_color('none')
    axtspt.xaxis.set_ticks_position('bottom')
    axtspt.spines['bottom'].set_position(('data', 0))
    # axtspt.set_xlim(xlims)
    # axtspt.set_ylim(ylims)
    # axtspt.set_xticks(xticks)
    # axtspt.set_yticks(yticks)
    axtspt.set_aspect('equal')
    longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
    xr = longaxis*np.cos(theta)
    yr = shortaxis*np.sin(theta)
    axtspt.plot(xr, yr, color="red", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>

In [ ]:
fig,ax2 = plt.subplots(figsize=(5,2))

for iktrial in range(ntrial):
    ax2.scatter(etaseries,Bsprvseries[:,iktrial,0],c='black',s=10,alpha=1.0)
ax2.plot(etaseries,Bsprvseries_theo[:,0],'gray',linewidth =2.5,label=r'theoretical')
ax2.plot(np.array([-0.1,1.1]),np.ones(2)*BAmvseries[0,0],linestyle='--',linewidth=1.5,color='grey',alpha=0.5)

xticks = np.linspace(0.0,1.0,3)
xlims = [-0.1,1.1]
yticks = np.linspace(1.0,2.,2)
ylims = [0.5,2.5]
# ax2.set_xlim(xlims)
# ax2.set_ylim(ylims)
# ax2.set_xticks(xticks)
# ax2.set_yticks(yticks)




In [ ]:
fig,ax2 = plt.subplots(figsize=(5,2))
for iktrial in range(ntrial):
    ax2.scatter(etaseries-0.01,Sigmn_num[:,iktrial,2],c='blue',s=5,alpha=0.5)
ax2.plot(etaseries,np.mean(Sigmn_theo[:,:,2],axis=1),'green',linewidth =2.5,label=r'theoretical')

for iktrial in range(ntrial):
    ax2.scatter(etaseries+0.01,Bsprvseries[:,iktrial,0]-BAmvseries[0,0],c='black',s=10,alpha=1.0)
ax2.plot(etaseries,Bsprvseries_theo[:,0]-BAmvseries[0,0],'gray',linewidth =2.5,label=r'theoretical')

fig,ax2 = plt.subplots(figsize=(4,4))
ax2.scatter(np.squeeze(Sigmn_num[:,:,2]).flatten(),np.squeeze(Bsprvseries[:,:,0]).flatten()-BAmvseries[0,0],s=10,c='gray')

xticks = np.linspace(-1,1.0,2)
xlims = [-1.2,1.2]
yticks = np.linspace(-1,1.,2)
ylims = [-1.2,1.2]
ax2.set_xlim(xlims)
ax2.set_ylim(ylims)
ax2.set_xticks(xticks)
ax2.set_yticks(yticks)
ax2.axhline(c='grey', lw=1)
ax2.axvline(c='grey', lw=1)

In [ ]:
fig,ax2 = plt.subplots(figsize=(5,2))
for iktrial in range(ntrial):
    ax2.scatter(etaseries,Sigmn_num[:,iktrial,2],c='r',s=5,alpha=0.5)
ax2.plot(etaseries,np.mean(Sigmn_num[:,:,2],axis=1),'red',linewidth =1.5,label=r'numerical')
ax2.plot(etaseries,np.mean(Sigmn_theo[:,:,2],axis=1),'orange',linewidth =1.5,label=r'theoretical')

xticks = np.linspace(0.0,1.0,3)
xlims = [-0.1,1.1]
yticks = np.linspace(-2,2,3)
ylims = [-2.,2]
ax2.set_xlim(xlims)
ax2.set_ylim(ylims)
ax2.set_xticks(xticks)
ax2.set_yticks(yticks)
ax2.axhline(c='grey', lw=1)

fig,ax2 = plt.subplots(figsize=(5,2))
for iktrial in range(ntrial):
    ax2.scatter(etaseries,Sigmn_num[:,iktrial,0],c='r',s=5,alpha=0.5)
ax2.plot(etaseries,np.mean(Sigmn_num[:,:,0],axis=1),'red',linewidth =1.5,label=r'numerical')
ax2.plot(etaseries,np.mean(Sigmn_theo[:,:,0],axis=1),'orange',linewidth =1.5,label=r'theoretical')

yticks = np.linspace(-5,5,3)
ylims = [-6.,6]
ax2.set_xlim(xlims)
ax2.set_ylim(ylims)
ax2.set_xticks(xticks)
ax2.set_yticks(yticks)
ax2.axhline(c='grey', lw=1)

fig,ax2 = plt.subplots(figsize=(5,2))
for iktrial in range(ntrial):
    ax2.scatter(etaseries,Sigmn_num[:,iktrial,1],c='r',s=5,alpha=0.5)
ax2.plot(etaseries,np.mean(Sigmn_num[:,:,1],axis=1),'red',linewidth =1.5,label=r'numerical')
ax2.plot(etaseries,np.mean(Sigmn_theo[:,:,1],axis=1),'orange',linewidth =1.5,label=r'theoretical')

yticks = np.linspace(-15,5,3)
ylims = [-16.,6]
ax2.set_xlim(xlims)
ax2.set_ylim(ylims)
ax2.set_xticks(xticks)
ax2.set_yticks(yticks)
ax2.axhline(c='grey', lw=1)

In [ ]:
#%%
'''
loading patterns
for ensemble-averaging results
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
xticks = np.linspace(-80,20,3)
xlims = [-90,40]
yticks = np.linspace(0,2,2)
ylims = [-2,3]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''

iktrial=0
ig=9
alphaval=0.10
edgv='blue'
cm='br'
fig,ax=plt.subplots(figsize=(5,2))  
nrank=1
ntrial = np.shape(Rsprvecseries)[1]
NE=int(NE)

'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)

mEvec,mIvec,nEvec,nIvec=np.squeeze(Rsprvecseries[ig,:,:NE,0]),np.squeeze(Rsprvecseries[ig,:,NE:,0]),np.squeeze(Lsprvecseries[ig,:,:NE,0]),np.squeeze(Lsprvecseries[ig,:,NE:,0])
eigVforn = np.mean(Bsprvseries[ig,:,0])
nEvec,nIvec = nEvec*eigVforn,nIvec*eigVforn
mEvec,mIvec,nEvec,nIvec=mEvec.flatten(),mIvec.flatten(),nEvec.flatten(),nIvec.flatten()

scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NI*ntrial,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    print('shape:',np.shape(mEvec),np.shape(meanmE))
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

# ax.scatter(nIvec,mIvec,s=5.0,c='b',alpha=0.2)#alphaval)#cm[1],alpha=alphaval)
# ax.scatter(nEvec,mEvec,s=5.0,c='r',alpha=0.2)#alphaval)#cm[0],alpha=alphaval)

ampdir = 8
ax.plot([meannE[0],meannE[0]+ampdir*dirvecE[0,0]],[meanmE[0],meanmE[0]+ampdir*dirvecE[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+ampdir*dirvecE[0,1]],[meanmE[0],meanmE[0]+ampdir*dirvecE[1,1]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+ampdir*dirvecI[0,0]],[meanmI[0],meanmI[0]+ampdir*dirvecI[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+ampdir*dirvecI[0,1]],[meanmI[0],meanmI[0]+ampdir*dirvecI[1,1]],color=edgv,linestyle='--',linewidth=1.5)

# confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=edgv)
# confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=edgv)


ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
ax.set_aspect('equal')

## Gaussian approximation

In [ ]:
'''
SYMMETRY, 
* increase g0 from zero to considerable
* constant JE,JI,a,b
'''
# generate mean matrix
print('sigmm')
nrank=1
nn = 200
gnn = 4

Nt=np.array([gnn*nn,nn])

NE,NI=Nt[0],Nt[1]
N=NE+NI
## useful for determining g
Nparams=np.array([NE,NI])
Npercent = Nparams/N
nrank,ntrial,neta,nvec,nmaxFP=1,10,1,2,3
''' Network Setting  '''
## heterogeneous random gain
ngEIH=3
ngEI = ngEIH*2
gEIseries1 = np.linspace(2.5,4.0,ngEIH)
gEIseries2 = np.linspace(5.,6.5,ngEIH)
gEIseries = np.zeros(ngEI)
gEIseries[:ngEIH] = gEIseries1
gEIseries[ngEIH:] = gEIseries2
print(gEIseries)

Cs = 60
kE,kI = gnn,1
## heterogeneous degree of symmetry: amplitudes and signs
coeffetaEsub=np.array([1.0,1.0,1.0])#
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)
signetaTotal[0]=-1

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]
## E->total ppercentEsub[0]/2.0,ppercentEsub[1]/2.0, (I) 1/2.0

'''  Recording Variables. '''
## mean connectivity
RAmvecseries,LAmvecseries = np.zeros((ngEI,N)),np.zeros((ngEI,N))
BAmvseries = np.zeros((ngEI,N),dtype=complex)

## perturbed + symmetric eigenvectors
ZSparserandom = np.zeros((ngEI,ntrial,N,N))
## eigenvectors, decomposing Sparse Matrix
Rsprvecseries,Lsprvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RsprvecTseries,LsprvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Bsprvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
Radiusseries = np.zeros(ngEI)
## Gaussian iid random connections
gYbasexee=np.zeros((ngEI,N,N)) 
Riidvecseries,Liidvecseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
RiidvecTseries,LiidvecTseries=np.zeros((ngEI,ntrial,N,nvec*2)),np.zeros((ngEI,ntrial,N,nvec*2))
Biidvseries=np.zeros((ngEI,ntrial,N),dtype=complex)
## corresponding statistics \mu and \sigma \cov
armu,sigrcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2)) # 2 for E and I
almu,siglcov = np.zeros((ngEI,ntrial,2)),np.zeros((ngEI,ntrial,2))
siglr = np.zeros((ngEI,ntrial,2))
Sigmn_num = np.zeros((ngEI,ntrial,3))
Sigmn_theo= np.zeros((ngEI,ntrial,3))


## block-like eta
etaMat =np.zeros((ngEI,ntrial,2,2))
gRandom = np.zeros((ngEI,ntrial,2))

''' Nullclines parameters  '''
## compare three models' dynamcis
kappaintersect_Sparse = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_Full   = np.zeros((ngEI,ntrial,nmaxFP*2))
kappaintersect_R1     = np.zeros((ngEI,ntrial,nmaxFP*2))

''' simulating parameters '''
tt = np.linspace(0,150,500)
dt = tt[2]-tt[1]
ntt = len(tt)
## compare three neuron population activities
xfpseries_Sparse = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_Full   = np.zeros((ngEI,ntrial,N,ntt))
xfpseries_R1     = np.zeros((ngEI,ntrial,N,ntt))
''' All random matrices for each trials '''
''' ## Three \bar{J} cases '''
EPSP,a,b=0.025,0,0# JE PER EPSP 


gEI = 3.0
etaseries = np.linspace(0.5,1.0,ngEI)
etassrecord = np.zeros((ngEI,3))
etauserecord = np.zeros((ngEI,3))
## Am is fixed 
## mean matrix, reference eigenvalue and eigenvectors
Am = np.zeros((N,N))
IPSP = gEI*EPSP
JE0=EPSP*kE*Cs
JI0=IPSP*kI*Cs
Am[:,:NE],Am[:,NE:] = JE0/NE,-JI0/NI
# meigvecAm,neigvecAm,eigvAm = decompunperturbedEI(Am)
# xAm,yAm=np.reshape(meigvecAm[:,0],(N,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(N,1))
xAm,yAm = np.zeros((N,1)),np.zeros((N,1))
eigvAm = np.zeros(N)
eigvAm[0]=(JE0-JI0)
xAm[:NE,0],xAm[NE:,0] = 1/np.sqrt(N),1/np.sqrt(N)
yAm[:NE,0],yAm[NE:,0] = N*JE0/NE/np.sqrt(N)/eigvAm[0],-N*JI0/NI/np.sqrt(N)/eigvAm[0]
RAmvecseries[0,:],LAmvecseries[0,:] = xAm[:,0],yAm[:,0]
BAmvseries[0,:] = eigvAm[:]
print('eigvAm:',eigvAm[0])
## basic preparation

''' Full Gaussian iid model -- Dynamics '''
## variance
gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs*N/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs*N/NI
# print("ratio gE/gI ",np.sqrt(gE2/gI2),', ',1/gEI)
alphaNEI = Npercent.copy()
gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
eigvgm,eigvecgm=la.eig(gMmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
# print("RG:",r_g)
Xsym =iidGaussian([0,1.0/np.sqrt(N)],[N,N])
XsymT = Xsym.copy().T
Xgiid = Xsym.copy()
Xgiid[:NE,:NE]*=np.sqrt(gE2)
Xgiid[:NE,NE:]*=np.sqrt(gI2)
Xgiid[NE:,:NE]*=np.sqrt(gE2)
Xgiid[NE:,NE:]*=np.sqrt(gI2)
Jgiid = Xgiid.copy()+Am.copy()
''' Original Sparse Network '''
eigvJiid,leigveciid,reigveciid,xnorm0iid,ynorm0iid=decompNormalization(Jgiid,xAm,yAm,xAm,yAm,nparams = Nparams,sort=0)
# print('Full:',eigvJiid[0])
''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
xnorm0iid,ynorm0iid = xnorm0iid*np.sqrt(N),ynorm0iid*np.sqrt(N)

''' Iterative Processing '''
for iktrial in range(ntrial):
    print('---- simulating neuronal activity #trial-',iktrial,'------')
    ## EE-trial
    n1,n2,prob=NE,NE,kE*Cs/NE
    subProb = randbin(1,n1*n2,1-prob)
    JEEsub= np.reshape(subProb,(n1,n2))

    ## EI-trial
    n1,n2,prob=NI,NE,kE*Cs/NE
    subProb = randbin(1,n1*n2,1-prob)
    JEIsub= np.reshape(subProb,(n1,n2))

    ## II-trial
    n1,n2,prob=NI,NI,kI*Cs/NI
    subProb = randbin(1,n1*n2,1-prob)
    JIIsub= np.reshape(subProb,(n1,n2))
    ## dilute/sparse matrix /attic
    for ig, eta in enumerate (etaseries):                
        Jbinary = np.zeros((N,N))
        ### GENERATE SYMMETRIC, SPARSE SUBMATRIX.
        p = kE*Cs/NE
        # for NE
        etaEE  = eta#1.0
        # for NI
        etaII  = 0.0#eta#1.0
        # for EI
        etaEI  = 0.0#eta#1.0
        etassrecord[ig,0],etassrecord[ig,1],etassrecord[ig,2] = (etaEE)/(1),(etaEI)/(1),(etaII)/(1)
        # gaussian 
        etaset = np.zeros(3)
        etaset[0],etaset[1],etaset[2] = (etaEE-p)/(1-p),-(etaEI-p)/(1-p),(etaII-p)/(1-p)
        etauserecord[ig,:]=etaset[:]
        ## negative
        signeta_use = np.ones(3)
        # signeta_use[1]=-1
        for i in range(3):
            if etaset[i]<0:
                signeta_use[i] *=(-1)

        probEE = kE*Cs/NE
        JbEE,JbEET_=generate_SymSpr_Mat_trial(JEEsub,etaEE,kE*Cs/NE,np.array([NE,NE]))
        JbEE_=JbEET_.copy().T
        ## combine two to get EE sub-matrix
        JbEEt = np.zeros((NE,NE))
        for i in range(NE):
            JbEEt[i,i]=JbEE[i,i]
        idxEEd, idyEEd = np.where(JbEE>0)
        iddown = np.where(idxEEd>idyEEd)
        idxEEd,idyEEd=idxEEd[iddown],idyEEd[iddown]
        JbEEt[idxEEd,idyEEd]=JbEE[idxEEd,idyEEd]
        idxEEu, idyEEu = np.where(JbEET_>0)
        idup = np.where(idxEEu<idyEEu)
        idxEEu,idyEEu=idxEEu[idup],idyEEu[idup]
        JbEEt[idxEEu,idyEEu]=JbEET_[idxEEu,idyEEu]
        # print('symmetryEE:',np.sum(np.sum(JbEEt*JbEEt.T))/kE/Cs/NE,etaEE)
        # print('probEE:',np.sum(np.sum(JbEEt))/NE**2,kE*Cs/NE)

        
        probII = kE*Cs/NE
        JbII,JbIIT_=generate_SymSpr_Mat_trial(JIIsub,etaII,kI*Cs/NI,np.array([NI,NI]))
        JbII_=JbIIT_.copy().T
        ## combine two to get EE sub-matrix
        JbIIt = np.zeros((NI,NI))
        for i in range(NI):
            JbIIt[i,i]=JbII[i,i]
        idxIId, idyIId = np.where(JbII>0)
        iddown = np.where(idxIId>idyIId)
        idxIId,idyIId=idxIId[iddown],idyIId[iddown]
        JbIIt[idxIId,idyIId]=JbII[idxIId,idyIId]
        idxIIu, idyIIu = np.where(JbIIT_>0)
        idup = np.where(idxIIu<idyIIu)##
        idxIIu,idyIIu=idxIIu[idup],idyIIu[idup]
        JbIIt[idxIIu,idyIIu]=JbIIT_[idxIIu,idyIIu]

        #
        JbIE,JbIET_=generate_SymSpr_MatEI_trial(JEIsub,etaEI,np.array([kE*Cs/NE,kI*Cs/NI]),np.array([NI,NE]))
        ## combine two to get EE sub-matrix
        JbIEt,JbEIt = np.zeros((NI,NE)),np.zeros((NE,NI))
        JbIEt,JbEIt = JbIE.copy(),JbIET_.copy()

        Jbinary[:NE,:NE],Jbinary[NE:,NE:]=JbEEt,-JbIIt
        Jbinary[NE:,:NE],Jbinary[:NE,NE:]=JbIEt,-JbEIt

        ## from attic to sparse connectivity
        JE,JI = EPSP,IPSP
        Jdilute = Jbinary.copy()
        Jdilute[:,:NE] *= JE
        Jdilute[:,NE:] *= JI
        # print('varEE:',np.std(np.squeeze(Jdilute[:,:NE]).flatten())**2,kE*Cs/NE*(1-kE*Cs/NE)*EPSP**2)
        ## lambda_change
        KE,KI=kE*Cs,kI*Cs
        p=KE/NE
        # p=1-KE/NE
        forJEE=EPSP**2*KE*eta*(1-p)**2-2*EPSP**2*KE*(1-eta)*(1-p)*p+EPSP**2*p**2*(NE-KE*2+KE*eta)## sigma^E
        forJEI=-EPSP*IPSP*KI*eta*(1-p)**2+2*EPSP*IPSP*KI*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NI-KI*2+KI*eta)## sigma^I

        # forJIE=-EPSP*IPSP*KE*eta*(1-p)**2+2*EPSP*IPSP*KE*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NE-KE*2+KE*eta)## ERROR!!!
        forJIE=-EPSP*IPSP*KE*eta*(1-p)**2+EPSP*IPSP*KE*(1-eta)*(1-p)*p+EPSP*IPSP*KE*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NE-KE*2+KE*eta)## signa^E
        forJII=IPSP**2*KI*eta*(1-p)**2-2*IPSP**2*KI*(1-eta)*(1-p)*p+IPSP**2*p**2*(NI-KI*2+KI*eta)
        sigmaE,sigmaI = forJEE*N*JE0/NE*NE+forJIE*(-N*JI0/NI*NI),forJEI*N*JE0/NE*NE+forJII*(-N*JI0/NI*NI)
        Sigmn_theo[ig,iktrial,0],Sigmn_theo[ig,iktrial,1]=sigmaE/NE/eigvAm[0]**2, sigmaI/NI/eigvAm[0]**2
        Sigmn_theo[ig,iktrial,2] = Sigmn_theo[ig,iktrial,0]*Npercent[0]+Sigmn_theo[ig,iktrial,1]*Npercent[1]
        # etaEE,etaEIset,etaII = blocklikeEtasymmetry(Jdilute,Nparams,EPSP,IPSP)
        # etaMat[ig,iktrial,0,0],etaMat[ig,iktrial,1,0],etaMat[ig,iktrial,1,1]=etaEE,np.mean(etaEIset),etaII
        # print("eta:",etaEE,np.mean(etaEIset),etaII)

        # overall
        J = Jdilute.copy()
        JT = Jdilute.copy().T
        ## calculate the block-like degree of symmetry
        # EE 

        ## subtract mean value
        # X = Jdilute.copy() - Am.copy() # random matrix, with zero mean 
        X = Jdilute.copy() # random matrix, with zero mean 
        X[:NE,:NE] = X[:NE,:NE] - np.mean(X[:NE,:NE].flatten())
        X[:NE,NE:] = X[:NE,NE:] - np.mean(X[:NE,NE:].flatten())
        X[NE:,:NE] = X[NE:,:NE] - np.mean(X[NE:,:NE].flatten())
        X[NE:,NE:] = X[NE:,NE:] - np.mean(X[NE:,NE:].flatten())
        ''' Original Sparse Network '''
        eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xnorm0iid/np.sqrt(N),ynorm0iid/np.sqrt(N),xAm,yAm,nparams = Nparams,sort=1)
        # eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xAm,yAm,xAm,yAm,nparams = Nparams,sort=1)
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0,ynorm0 = xnorm0*np.sqrt(N),ynorm0*np.sqrt(N)
        Rsprvecseries[ig,iktrial,:,0],Lsprvecseries[ig,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
        Bsprvseries[ig,iktrial,:]=eigvJ.copy()

        axrmu,aylmu,sigxr,sigyl,sigcov = numerical_stats(xnorm0,ynorm0,xAm*np.sqrt(N),yAm*np.sqrt(N),eigvJ,nrank,2,ppercent=Npercent)
        armu[ig,iktrial,:],almu[ig,iktrial,:] = axrmu[:,0],aylmu[:,0]
        sigrcov[ig,iktrial,:],siglcov[ig,iktrial,:]= sigxr[:,0],sigyl[:,0]
        siglr[ig,iktrial,:] = sigcov[:,0,0]
        Sigmn_num[ig,iktrial,:2] = siglr[ig,iktrial,:]*eigvAm[0]
        Sigmn_num[ig,iktrial,2] = Sigmn_num[ig,iktrial,0]*Npercent[0]+Sigmn_num[ig,iktrial,1]*Npercent[1]
           
        
        ## Rank One Approximation, Nullclines of \kappa
        xnorm0, ynorm0 = Rsprvecseries[ig,iktrial,:,0].copy(),Lsprvecseries[ig,iktrial,:,0]
        xnorm0, ynorm0 = np.reshape(xnorm0,(N,1)),np.reshape(ynorm0,(N,1))
        xmu,ymu = armu[ig,iktrial,:].copy(),almu[ig,iktrial,:].copy()
        xsig,ysig = sigrcov[ig,iktrial,:].copy(),siglcov[ig,iktrial,:].copy()
        yxcov = siglr[ig,iktrial,:].copy()

        ''' Sparse Connectivity -- Dynamics '''
        Jpt = J.copy()
        xinit = np.random.normal(0,1E-2,(1,N))
        xinit = np.squeeze(xinit)
        xtemporal = odesimulation(tt,xinit,Jpt,0)
        xfpseries_Sparse[ig,iktrial,:,:] = xtemporal.T.copy()
        ## 2 KAPPA_0
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_Sparse[ig,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))*Npercent[0]-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))*Npercent[1]
        kappanum[0]=kappanum[0]*np.sqrt(N)*eigvAm[0]
        kappaintersect_Sparse[ig,iktrial,:3]=kappanum[:].copy()

        ## Gaussian Full EI 

        # p = kE*Cs/NE
        # etaGau = (eta-p)/(1-p)
        # etaset=etaGau*np.ones(3)
        # ## negative
        # signeta_use = np.zeros(3)
        # signeta_use[0],signeta_use[1],signeta_use[2]=signetaEsub[0],signetaTotal[0],signetaTotal[2]
        # if (etaGau<0):
        #     signeta_use *=(-1)
        # etaset[1]   = -etaset[1]

        Xinit = Xsym.copy() 
        # signeta_use[2]=-1
        ### >>>>>>>>.Subcircuit Exc sym >>>>>>>>>.
        ## remaining EE
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:NE,:NE]=signeta_use[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
        ### >>>>>>> Total >>>>>
        # EI
        asqr=(1-np.sqrt(1-(etaset[1])**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=signeta_use[1]*aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta_use[1]*aamp*XsymT[:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signeta_use[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()

        Xinit[:NE,:NE]*=np.sqrt(gE2)
        Xinit[:NE,NE:]*=np.sqrt(gI2)
        Xinit[NE:,:NE]*=np.sqrt(gE2)
        Xinit[NE:,NE:]*=np.sqrt(gI2)
        Jfull = Xinit.copy()+Am.copy()
        ## compare
        KE,KI=kE*Cs,kI*Cs
        p=KE/NE
        forJEE=EPSP**2*KE*eta*(1-p)**2-2*EPSP**2*KE*(1-eta)*(1-p)*p+EPSP**2*p**2*(NE-KE*2+KE*eta)## sigma^E
        forJEI=-EPSP*IPSP*KI*eta*(1-p)**2+2*EPSP*IPSP*KI*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NI-KI*2+KI*eta)## sigma^I
        # etaset_use = etaset*signeta_use
        etaset_use = etaset.copy()
        forJIE=-EPSP*IPSP*KE*eta*(1-p)**2+2*EPSP*IPSP*KE*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NE-KE*2+KE*eta)## ERROR!!!
        forJII=IPSP**2*KI*eta*(1-p)**2-2*IPSP**2*KI*(1-eta)*(1-p)*p+IPSP**2*p**2*(NI-KI*2+KI*eta)
        ssigE,ssigI = (forJEE*N*JE0/NE*NE+forJIE*(-N*JI0/NI*NI))/NE,(forJEI*N*JE0/NE*NE+forJII*(-N*JI0/NI*NI))/NI
        gsigE = JE0*EPSP**2*N*p*(1-p)*etaset_use[0]-JI0*EPSP*IPSP*N*(1-p)*p*etaset_use[1]
        gsigI = JE0*EPSP*IPSP*N*p*(1-p)*etaset_use[1]-JI0*IPSP*IPSP*N*(1-p)*p*etaset_use[2]
        print('COMPARE-SPARSE:',ssigE,ssigI,ssigE*4/5.0+ssigI*1/5.0)
        print('COMPARE-gau:',gsigE,gsigI)

        ''' Original Sparse Network '''
        eigvJiid,leigveciid,reigveciid,xnormgau,ynormgau=decompNormalization(Jfull,xnorm0iid/np.sqrt(N),ynorm0iid/np.sqrt(N),xAm,yAm,nparams = Nparams,sort=1)
        # eigvJiid,leigveciid,reigveciid,xnormgau,ynormgau=decompNormalization(Jfull,xAm,yAm,xAm,yAm,nparams = Nparams,sort=1)
        # print('Full:',eigvJiid[0])
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnormgau,ynormgau = xnormgau*np.sqrt(N),ynormgau*np.sqrt(N)
        Riidvecseries[ig,iktrial,:,0],Liidvecseries[ig,iktrial,:,0]=xnormgau[:,0].copy(),ynormgau[:,0].copy()
        Biidvseries[ig,iktrial,:]=eigvJiid.copy()

   

In [ ]:
'''
with symmetry, subcircuit
eigvJ0
'''
xticks = np.linspace(-1.0,2.0,1)
xlims = [-1.5,1.5]
yticks = np.linspace(-1,1,2)
ylims = [-1.5,1.5]
theta = np.linspace(0, 2 * np.pi, 200)
idxc=0
cm='rbg'
print(ntrial)
gEI =gEI
Bsprvseries_theo = np.zeros((ngEI,3),dtype=complex)
for ig,eta in enumerate (etaseries): 
    figtspt,axtspt=plt.subplots(figsize=(6,3))
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    KE,KI=kE*Cs,kI*Cs
    p=KE/NE
    etaset = etassrecord[ig,:]
    forJEE=EPSP**2*KE*etaset[0]*(1-p)**2-2*EPSP**2*KE*(1-etaset[0])*(1-p)*p+EPSP**2*p**2*(NE-KE*2+KE*etaset[0])## sigma^E
    forJEI=-EPSP*IPSP*KI*etaset[1]*(1-p)**2+2*EPSP*IPSP*KI*(1-etaset[1])*(1-p)*p-EPSP*IPSP*p**2*(NI-KI*2+KI*etaset[1])## sigma^I

    # forJIE=-EPSP*IPSP*KE*eta*(1-p)**2+2*EPSP*IPSP*KE*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NE-KE*2+KE*eta)## ERROR!!!
    forJIE=-EPSP*IPSP*KE*etaset[1]*(1-p)**2+EPSP*IPSP*KE*(1-etaset[1])*(1-p)*p+EPSP*IPSP*KE*(1-etaset[1])*(1-p)*p-EPSP*IPSP*p**2*(NE-KE*2+KE*etaset[1])## signa^E
    forJII=IPSP**2*KI*etaset[2]*(1-p)**2-2*IPSP**2*KI*(1-etaset[2])*(1-p)*p+IPSP**2*p**2*(NI-KI*2+KI*etaset[2])
    totallambda=(forJEE+forJEI)*N*JE0/NE*NE/N-(forJIE+forJII)*JI0
    Janalytical = JE0*EPSP**2*p*(etaset[0]-p)*(1)*NE+JE0*EPSP**2*p*(etaset[1]-p)*(KI/KE*gEI**2)*NE#JE0*EPSP**2*p*(eta-p)*(1+KI/KE*gEI**2)*NE
    Janalytical -= (gEI*JE0*EPSP**2*p*(etaset[1]-p)*(1)*NI + gEI*JE0*EPSP**2*p*(etaset[2]-p)*(KI/KE*gEI**2)*NI)#gEI*JE0*EPSP**2*p*(eta-p)*(1+KI/KE*gEI**2)*NI
    print('total:',totallambda,Janalytical)
    B2=totallambda
    roots = Cubic([1,-BAmvseries[0,0],0,-B2])
    Bsprvseries_theo[ig,:] = roots[:3]
    # roots = np.ones(2)*totallambda/BAmvseries[ig,0]**2+BAmvseries[ig,0]
    ## variance
    gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs/NI
    gE2,gI2 = gE2*N,gI2*N
    alphaNEI = Npercent.copy()
    gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
    eigvgm,eigvecgm=la.eig(gMmat) 
    r_g2=np.max(eigvgm)
    r_g = np.sqrt(r_g2)
    print(r_g)
    for idxtrial in range(ntrial):
        axtspt.scatter(np.real(Bsprvseries[ig,idxtrial,:]),np.imag(Bsprvseries[ig,idxtrial,:]),s=5,c='blue',alpha=0.25) # >>>>>>>>>>>>>>
    axtspt.scatter(np.real(Bsprvseries[ig,:,0]),np.imag(Bsprvseries[ig,:,0])-0.2*idxc,s=50,c='red',marker='^',alpha=0.5) # >>>>>>>>>>>>
    # axtspt.scatter(np.real(Bsprvseries[idxgavg,:,1]),np.imag(Bsprvseries[idxgavg,:,1])-0.2*idxc,s=20,c='darkblue',marker='^',alpha=0.5) # >>>>>>>>>>>>
    axtspt.set_aspect('equal')
    axtspt.plot(np.ones(2)*BAmvseries[0,0],np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='grey')
    axtspt.plot(np.ones(2)*(roots[0]),np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='green')
    axtspt.plot(np.ones(2)*np.mean(np.real(Biidvseries[ig,:,0])),np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='blue')

    # axtspt.plot(np.ones(2)*np.mean(np.real(Bsprvseries[ig,:,0])),np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='blue')
    print('eigv:',BAmvseries[ig,0],roots)
    axtspt.spines['right'].set_color('none')
    axtspt.spines['top'].set_color('none')
    axtspt.xaxis.set_ticks_position('bottom')
    axtspt.spines['bottom'].set_position(('data', 0))
    # axtspt.set_xlim(xlims)
    # axtspt.set_ylim(ylims)
    # axtspt.set_xticks(xticks)
    # axtspt.set_yticks(yticks)
    axtspt.set_aspect('equal')
    longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
    xr = longaxis*np.cos(theta)
    yr = shortaxis*np.sin(theta)
    axtspt.plot(xr, yr, color="red", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>

In [ ]:
'''
with symmetry, subcircuit
eigvJ0
'''
xticks = np.linspace(-1.0,2.0,1)
xlims = [-1.5,1.5]
yticks = np.linspace(-1,1,2)
ylims = [-1.5,1.5]
theta = np.linspace(0, 2 * np.pi, 200)
idxc=0
cm='rbg'
print(ntrial)
gEI =gEI
Bsprvseries_theo = np.zeros((ngEI,3),dtype=complex)
for ig,eta in enumerate (etaseries):
    
    figtspt,axtspt=plt.subplots(figsize=(6,3))
    IPSP = gEI*EPSP
    JE0=EPSP*kE*Cs
    JI0=IPSP*kI*Cs
    KE,KI=kE*Cs,kI*Cs
    p=KE/NE
    forJEE=EPSP**2*KE*eta*(1-p)**2-2*EPSP**2*KE*(1-eta)*(1-p)*p+EPSP**2*p**2*(NE-KE*2+KE*eta)## sigma^E
    forJEI=-EPSP*IPSP*KI*eta*(1-p)**2+2*EPSP*IPSP*KI*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NI-KI*2+KI*eta)## sigma^I

    # forJIE=-EPSP*IPSP*KE*eta*(1-p)**2+2*EPSP*IPSP*KE*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NE-KE*2+KE*eta)## ERROR!!!
    forJIE=-EPSP*IPSP*KE*eta*(1-p)**2+EPSP*IPSP*KE*(1-eta)*(1-p)*p+EPSP*IPSP*KE*(1-eta)*(1-p)*p-EPSP*IPSP*p**2*(NE-KE*2+KE*eta)## signa^E
    forJII=IPSP**2*KI*eta*(1-p)**2-2*IPSP**2*KI*(1-eta)*(1-p)*p+IPSP**2*p**2*(NI-KI*2+KI*eta)
    totallambda=(forJEE+forJEI)*N*JE0/NE*NE/N-(forJIE+forJII)*JI0
    Janalytical = JE0*EPSP**2*p*(eta-p)*(1+KI/KE*gEI**2)*NE
    Janalytical -= gEI*JE0*EPSP**2*p*(eta-p)*(1+KI/KE*gEI**2)*NI
    print('total:',totallambda,Janalytical)
    B2=totallambda
    roots = Cubic([1,-BAmvseries[0,0],0,-B2])
    Bsprvseries_theo[ig,:] = roots[:3]
    # roots = np.ones(2)*totallambda/BAmvseries[ig,0]**2+BAmvseries[ig,0]
    ## variance
    gE2,gI2 = EPSP**2*(1-kE*Cs/NE)*kE*Cs/NE,IPSP**2*(1-kI*Cs/NI)*kI*Cs/NI
    gE2,gI2 = gE2*N,gI2*N
    alphaNEI = Npercent.copy()
    gMmat = np.array([[gE2*alphaNEI[0],gI2*alphaNEI[1]],[gE2*alphaNEI[0],gI2*alphaNEI[1]]])
    eigvgm,eigvecgm=la.eig(gMmat) 
    r_g2=np.max(eigvgm)
    r_g = np.sqrt(r_g2)
    print(r_g)
    for idxtrial in range(ntrial):
        axtspt.scatter(np.real(Biidvseries[ig,idxtrial,:]),np.imag(Biidvseries[ig,idxtrial,:]),s=5,c='blue',alpha=0.25) # >>>>>>>>>>>>>>
    axtspt.scatter(np.real(Biidvseries[ig,:,0]),np.imag(Biidvseries[ig,:,0])-0.2*idxc,s=50,c='red',marker='^',alpha=0.5) # >>>>>>>>>>>>
    axtspt.scatter(np.real(Biidvseries[ig,:,1]),np.imag(Biidvseries[ig,:,1])-0.2*idxc,s=20,c='red',marker='^',alpha=0.5) # >>>>>>>>>>>>
    axtspt.set_aspect('equal')
    axtspt.plot(np.ones(2)*BAmvseries[0,0],np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='grey')
    axtspt.plot(np.ones(2)*(roots[0]),np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='green')
    axtspt.plot(np.ones(2)*np.mean(np.real(Biidvseries[ig,:,0])),np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='blue')

    # axtspt.plot(np.ones(2)*np.mean(np.real(Bsprvseries[ig,:,0])),np.array([-2.0,2.0]),linestyle='--',linewidth=2.5,color='blue')
    print('eigv:',Biidvseries[ig,:,0],roots)
    axtspt.spines['right'].set_color('none')
    axtspt.spines['top'].set_color('none')
    axtspt.xaxis.set_ticks_position('bottom')
    axtspt.spines['bottom'].set_position(('data', 0))
    # axtspt.set_xlim(xlims)
    # axtspt.set_ylim(ylims)
    # axtspt.set_xticks(xticks)
    # axtspt.set_yticks(yticks)
    axtspt.set_aspect('equal')
    longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
    xr = longaxis*np.cos(theta)
    yr = shortaxis*np.sin(theta)
    axtspt.plot(xr, yr, color="red", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>

In [ ]:
#%%
'''
loading patterns
for ensemble-averaging results
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
xticks = np.linspace(-80,20,3)
xlims = [-90,40]
yticks = np.linspace(0,2,2)
ylims = [-2,3]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''

iktrial=0
ig=4
alphaval=0.10
edgv='red'#'blue'
cm='br'
# fig,ax=plt.subplots(figsize=(5,2))  
nrank=1
ntrial = np.shape(Rsprvecseries)[1]
NE=int(NE)

'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)

# mEvec,mIvec,nEvec,nIvec=np.squeeze(Rsprvecseries[ig,:,:NE,0]),np.squeeze(Rsprvecseries[ig,:,NE:,0]),np.squeeze(Lsprvecseries[ig,:,:NE,0]),np.squeeze(Lsprvecseries[ig,:,NE:,0])
# eigVforn = np.mean(Bsprvseries[ig,:,0])
mEvec,mIvec,nEvec,nIvec=np.squeeze(Riidvecseries[ig,:,:NE,0]),np.squeeze(Riidvecseries[ig,:,NE:,0]),np.squeeze(Liidvecseries[ig,:,:NE,0]),np.squeeze(Liidvecseries[ig,:,NE:,0])
eigVforn = np.mean(Biidvseries[ig,:,0])
nEvec,nIvec = nEvec*eigVforn,nIvec*eigVforn
mEvec,mIvec,nEvec,nIvec=mEvec.flatten(),mIvec.flatten(),nEvec.flatten(),nIvec.flatten()

scale_std=5.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NI*ntrial,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    print('shape:',np.shape(mEvec),np.shape(meanmE))
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

# ax.scatter(nIvec,mIvec,s=5.0,c='b',alpha=0.2)#alphaval)#cm[1],alpha=alphaval)
ax.scatter(nEvec,mEvec,s=5.0,c=cm[0],alpha=alphaval)

ampdir = 8
ax.plot([meannE[0],meannE[0]+ampdir*dirvecE[0,0]],[meanmE[0],meanmE[0]+ampdir*dirvecE[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+ampdir*dirvecE[0,1]],[meanmE[0],meanmE[0]+ampdir*dirvecE[1,1]],color=edgv,linestyle='--',linewidth=1.5)
# ax.plot([meannI[0],meannI[0]+ampdir*dirvecI[0,0]],[meanmI[0],meanmI[0]+ampdir*dirvecI[1,0]],color=edgv,linestyle='--',linewidth=1.5)
# ax.plot([meannI[0],meannI[0]+ampdir*dirvecI[0,1]],[meanmI[0],meanmI[0]+ampdir*dirvecI[1,1]],color=edgv,linestyle='--',linewidth=1.5)

confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=edgv)
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=edgv)


ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')